In [1]:
import matplotlib.pyplot as plt
from official.nlp import optimization  # to create AdamW optimizer
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
import os
import shutil
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

tf.get_logger().setLevel('ERROR')


In [2]:
usecols = ['url', 'category', 'text']
data = pd.read_csv('dataset.csv', usecols=usecols)
data = data.dropna()
data = data[
    (data['text'] != 'ERROR')
]
data.to_csv('prepro-dataset.csv', index=False)


In [3]:
data_train: pd.DataFrame
data_test: pd.DataFrame
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [4]:
def categorize_labels(labels: np.ndarray) -> tuple[np.ndarray, dict[str, int]]:
    label_map: dict[str, int] = {}
    unique_labels = np.unique(labels)

    count = 0
    for unique_label in unique_labels:
        label_map[unique_label] = count
        count += 1

    map_function = np.vectorize(lambda x: label_map[x])
    vals = map_function(labels)
    return np.eye(count)[vals], label_map, count


data_features = data_train['text'].to_numpy()
data_labels = data_train['category'].to_numpy()
data_labels, label_map, category_count = categorize_labels(data_labels)
train_dataset = tf.data.Dataset.from_tensor_slices(
    (data_features, data_labels))
print(label_map)


{0: 0, 1: 1, 2: 2}


2022-10-18 22:46:54.065319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
BATCH_SIZE = 18
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


In [6]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/3',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')


BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3


In [7]:
def build_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder,trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = tf.keras.layers.Dropout(0.1)(net)
    net = tf.keras.layers.Dense(category_count, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

classifier_model = build_classifier_model()


In [8]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [9]:
epochs = 30
steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [10]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

In [11]:
print(f'Training model with {tfhub_handle_encoder}')
history = classifier_model.fit(x=train_dataset,
                               epochs=epochs)

Training model with https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Epoch 1/30
9/9 [==============================] - 28s 2s/step - loss: 0.8306 - binary_accuracy: 0.5066
Epoch 2/30
9/9 [==============================] - 21s 2s/step - loss: 0.6320 - binary_accuracy: 0.6601
Epoch 3/30
9/9 [==============================] - 28s 3s/step - loss: 0.5366 - binary_accuracy: 0.7083
Epoch 4/30
9/9 [==============================] - 27s 3s/step - loss: 0.4012 - binary_accuracy: 0.8114
Epoch 5/30
9/9 [==============================] - 21s 2s/step - loss: 0.3268 - binary_accuracy: 0.8487
Epoch 6/30
9/9 [==============================] - 21s 2s/step - loss: 0.2556 - binary_accuracy: 0.8794
Epoch 7/30
9/9 [==============================] - 25s 3s/step - loss: 0.1969 - binary_accuracy: 0.9254
Epoch 8/30
9/9 [==============================] - 21s 2s/step - loss: 0.1483 - binary_accuracy: 0.9649
Epoch 9/30
9/9 [==============================] - 29s 3s/step - loss: 0.1112 - binar

In [12]:
dataset_name = 'wkc'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

classifier_model.save(saved_model_path, include_optimizer=False)

In [13]:
dataset_name = 'wkc'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))
reloaded_model = tf.saved_model.load(saved_model_path)

In [17]:
examples = [
    '''
    Global management consulting | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content We’re helping build an economy that works for all. We’re helping build an economy that works for all. 7 in 10 people live in societies with growing inequality. That’s why we’re taking action. See how we’re helping Preorder the #TitaniumEconomyBook Global M&A market slows in 2022 first half—but shows signs of strength Americas are most active while technology/media/telecom dominates other sectors. Capture the value Explore more insights on M&A Survey results: Expectations for company performance, by industry Across industries, only respondents working in energy and materials and financial services report more upbeat expectations for their companies’ profits or demand for their goods or services than in the previous survey. See the results Explore more insights on strategy and corporate finance Author Talks: How industrial technology is creating a more inclusive economy McKinsey reveals 35 cutting-edge companies in the industrial tech sector, highlighting a robust “titanium economy” that creates American jobs and fuels innovation through sustainable, inclusive growth. Drive innovation Preorder the #TitaniumEconomyBook New at McKinsey Blog October 4, 2022 Latest McKinsey tech outlook identifies 14 key trends for business leaders September 29, 2022 McKinsey launches new Europe research at firm’s largest-ever media day September 28, 2022 ‘A sense of journey’: McKinsey’s new Mumbai office designed for age of hybrid work September 27, 2022 McKinsey and AWS expand existing collaboration to tackle the next horizon of cloud transformation September 26, 2022 McKinsey experts bring sustainability and inclusion to the skies Read more on our blog What does it mean to accelerate sustainable and inclusive growth? McKinsey partners from around the world weigh in Making the case for a new kind of growth Join McKinsey Come to McKinsey to do the best work, with the best teams and truly be at your best. Careers Search and apply How can we help you? Get in touch with us or find an office closest to you. McKinsey experts bring sustainability and inclusion to the skies | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog McKinsey experts bring sustainability and inclusion to the skies Sustainability Inclusive Growth September 26, 2022 The Farnborough International Air Show, one of the world’s largest aerospace and defense tradeshows, returned to England for the first time in four years this summer, providing aeronautics enthusiasts and industry leaders alike a chance to connect and consider the state of an industry facing critical challenges across sustainability, talent, technology, and growth. This year also marked the first time McKinsey was an official air show partner. Throughout the week, at a purpose-built chalet overlooking the event’s airstrip, colleagues from McKinsey’s Aerospace and Defense Practice hosted hundreds of client executives and industry leaders to explore topics like the future of air mobility, sustainable aviation, diversity and inclusion, and resilience. Video Partner Emma Loxton on Farnborough 2022. “Our presence at Farnborough this year is a perfect expression of the way our practice has grown and expanded,” says senior partner Varun Marya , who helps lead McKinsey’s aerospace and defense work globally. “Today, working in areas from general aviation to space, we help our clients set bold agendas, implement technologies, and develop capabilities to improve their businesses in substantial and sustainable ways.” This work is informed by our expertise and perspective in the sector, which was on full display at Farnborough. Partner Jesse Klempner moderated a panel on “The Role of Space in Driving Sustainability and Security,” where he was joined by representatives from the European Space Agency, Lockheed Martin, the World Economic Forum, and the satellite internet provider OneWeb. Elsewhere, partner Robin Riedel , who co-leads the McKinsey Center for Future Mobility, spoke on the event’s mainstage about what it will take to coordinate sector efforts and reach net zero by 2050. Later, along with leaders from the Mission Possible Partnership and the World Economic Forum, Robin convened an executive session on sustainable aviation at our McKinsey chalet and interviewed six industry disruptors whose ideas and innovations are shaping the future of aerospace and defense. Partner Emma Loxton also spoke about the road to net-zero aviation during a CFO panel that focused on building sustainable business models. Held on the event’s mainstage, the panel included executives from Rolls Royce, Shell, Virgin Atlantic, and ZeroAvia, a renewable-hydrogen aircraft developer. In another event, Emma held a reception for women working in aerospace at the McKinsey chalet, bringing together dozens of women executives and their allies to discuss inclusive talent. Clean skies for tomorrow: Delivering on the global power-to-liquid ambition Read the article That topic was the subject of one of our most widely talked-about reports of the week, Debugging the software talent gap in aerospace and defense . Published on one of the event’s busiest days, the report generated several client conversations and earned coverage in the print edition of Aviation International News’s air show daily. In it, the authors describe the skills deficit facing the industry, particularly on the tech front. They illustrate the opportunity companies have to “think differently” about their talent pipelines, which can be seized through several tactics, including increasing recruitment among traditionally underrepresented populations. Women, for example, are a third more underrepresented in aerospace and defense than they are in other sectors, and women of color make up just 10 percent of entry-level employees. Video The Air League 2022 Innovators’ Challenge. One path to improving those figures lies in creating and nurturing enthusiasm for the industry among diverse groups of young people. Late in the week, we hosted the final round of the Air League’s 2022 Global Innovators Challenge. The Air League is a UK charity focused on changing lives through aviation, and this year’s challenge asked participants to propose ideas for accelerating the sector’s transition to zero-carbon emissions. Six finalists presented their ideas in front of a judging panel that included representatives from leading industry organizations; McKinsey was represented on the panel by Emma. The winning team, “Flush to Fly,” presented its concept to source sustainable aviation fuels from sewage waste, had the opportunity to present their concept at the event’s mainstage at the close of the week. Related Blog Are flying cars finally ready to take off? August 31, 2021 Two partners talk about the latest in air mobility and explain how we’re helping clients begin to make science fiction a reality. Company performance by industry: A global survey | McKinsey Skip to main content Survey results: Expectations for company performance, by industry September 30, 2022 | Survey LinkedIn Twitter Facebook Email Survey results: Expectations for company performance, by industry LinkedIn Twitter Facebook Email Across industries, only respondents working in energy and materials and financial services report more upbeat expectations for their companies’ profits or demand for their goods or services than in the previous survey. In the latest McKinsey Global Survey on economic conditions, respondents’ expectations for their companies’ performance are more somber than they have been since early in the COVID-19 pandemic. This pessimism comes as companies feel the impact of cost increases. Nine in ten respondents report cost increases in the past six months—particularly the effects of rising energy prices and material costs . Yet respondents in sectors such as energy and materials and financial services report brighter spots compared with the June 2022 survey. Profits: Responses reveal a continued pessimism toward future company growth. Just over half of all respondents expect profits to increase, down from 65 percent six months ago. While in most industries, expectations for profits have become less positive since December 2021, respondents working in the energy and materials sector have a rosier view of their profits over the next six months than they did last quarter. Customer Demand: Overall, just 51 percent of respondents expect demand for their companies’ goods or services to increase over the next six months, the smallest share since July 2020. While respondents in consumer goods and retail and technology, media, and telecommunications are much more pessimistic about demand than they have been since mid-2020, respondents in financial services and advanced industries are more hopeful now than in the previous survey. Workforce Size: Nearly four in ten respondents expect their companies’ workforce size to expand in the months ahead. While respondents’ expectations for hiring in many industries remain mostly aligned with the previous quarter’s, respondents in financial services are much more likely now than in June to expect their employers to increase their headcount. About the author(s) The survey content and analysis were developed by Jeffrey Condon , a senior knowledge expert in McKinsey’s Atlanta office; Krzysztof Kwiatkowski and Vivien Singer , both capabilities and insights experts at the Waltham Client Capabilities Hub; and Sven Smit , the chair and director of the McKinsey Global Institute and a senior partner in the Amsterdam office. This article was edited by Heather Hanselman, an editor in the Atlanta office. Explore a career with us Search Openings Related Articles Article McKinsey Global Economics Intelligence Survey Economic conditions outlook, September 2022 Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Economic growth for the good of all: Sustainable and inclusive | McKinsey Skip to main content McKinsey Quarterly Our future lives and livelihoods: Sustainable and inclusive and growing October 26, 2021 | Article By Bob Sternfels , Tracy Francis , Anu Madgavkar , and Sven Smit LinkedIn Twitter Facebook Email Article (8 pages) Our future lives and livelihoods: Sustainable and inclusive and growing LinkedIn Twitter Facebook Email Article (8 pages) Growth for all, growth for good. Here, we offer a proposal for business, government, and society leaders. Audio DOWNLOADS Article (8 pages) As the world economy starts to emerge from the COVID-19 crisis, the time will soon come for leaders to look beyond safeguarding lives and livelihoods and to set their sights on a more profound challenge: bettering them. This societal challenge might be  ten times as big as the pandemic and last ten times as long. The three goals we have in mind—growth, sustainability, and inclusion—buttress one another yet don’t always pull in the same direction; we see powerful reinforcing as well as counteracting loops among them (exhibit). And so, while many might broadly agree on the aspiration, there’s a very tough question lurking in the background: How do we  go about building a future that delivers growth and sustainability and inclusion? Exhibit We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Full disclosure: we’re not going to offer an answer. Instead, we propose a way for changemakers in business, government, and society to explore the problem, a mental model that might offer the  best chance to reach the answer. It starts with this: we believe the ands are crucial and that they are  in fact the means to the end . The three elements of growth, sustainability, and inclusion are deeply connected and cannot be viewed as trade-offs. Consider this: without growth, how could we achieve prosperity and well-being or pay for the transitions needed to make the economy more sustainable  and inclusive? Without sustainability, how could we fashion growth for the current generation and the ones to follow? Without inclusion—an opportunity for productive work and a satisfying life for all citizens—how could we ensure the demand needed to propel growth? Indeed, getting to and —moving to a world in which growth and sustainability and inclusion form a powerful dynamic—is the imperative for the next era of business. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes But before we get to the challenge of and , let’s  face facts: hastening growth, sustainability, and inclusion are incredibly difficult challenges in  their own right. Fortunately, thinkers, strategists, activists, and many others around the world—dreamers and doers—are working on it. We are too. In our view, the world will need to confront three problems simultaneously: Growth is elusive. In the mature G-7 economies, GDP growth has halved to 1 percent per year  on average since the 2008 global financial crisis. 1 1. World Economic Outlook Database, October 2021, International Monetary Fund, imf.org. It’s the same story in emerging economies: despite some exceptions, such as China and India, growth in emerging economies overall has been lower recently than in the early 2000s. Poverty is still endemic, despite the progress made. More than 600 million people still lived in extreme poverty as of 2017. And in 2020, another 100 million or so people joined them as  a result of the COVID-19 pandemic. This will persist unless today’s leaders create sufficient jobs with decent wages, as well as a robust social contract that ensures access to affordable housing, healthcare, and energy for the bottom one to three quintiles of the population, depending on the country. Meanwhile, a new threat to personal income is mounting: the  rise of technology-driven changes in the ways we work, which the pandemic has accelerated. We estimate that more than 100 million people will need to make occupational transitions by 2030 in a set of eight advanced and emerging economies. Ensuring a sustainable future will require massive investment. For example, the International Energy Agency estimates that net-zero emissions might require investments of almost $5 trillion each year by 2030, and $4.5 trillion per year by 2050. 2 2. Net Zero by 2050 , International Energy Agency, July 2021, iea.org. The annual bill equates to about half of global corporate profits in 2019,  or about one and a half times the annual increase in public debt over the preceding 15 years. Additional investments needed  for decarbonization in agriculture, transportation, and other sectors could nearly double the bill. While many of these investments would produce a return, their financing or pricing is not yet  set up. And that’s just the start: as we explain in this article, even if the global economy were to get these three goals notionally right, there are contingencies among them that, if left unresolved, could wreck any progress made. Here, we seek to frame the debate about achieving sustainable, inclusive growth in a clear-eyed  way, laying out the aspiration but also the toughest problems that need to be solved to achieve this growth, with some illustrations as to their size. Good strategy should always start with asking the  right questions. For today’s leaders, the questions are vast and profound— and soluble. Good strategy should always start with asking the right questions. For today’s leaders, the questions are vast, profound— and soluble. The virtuous cycle starts with growth What do we mean by sustainable, inclusive growth? There are many ideas associated with these words. We aim for broad rather than narrow interpretations: In growth, we include the ambition of increased prosperity and well-being, including economic-profit growth for companies, GDP growth  for nations—as well as measures such as life satisfaction for citizens—derived in part from dignity of work (while recognizing that measurable definitions of well-being are  still evolving). In inclusion, we consider equality of opportunity and broad-based progress of outcomes for  all—especially sufficiency of living standards—and the narrowing of inequalities among genders, ages, ethnicities, family backgrounds, and places of residence. In sustainability, we aim for environmental resilience, which starts with reducing climate risk but also includes much broader preservation of natural capital as well as intergenerational fairness, all considered in terms of economic and societal costs and benefits. These three goals are daunting. Fortunately, they can strengthen and reinforce one another: Growth supports inclusion, part 1: Creating meaningful jobs and lifting incomes. High-growth emerging economies have delivered powerful proof that growth supports inclusion, by reducing the global share of those living  in extreme poverty by two-thirds—to less than 10 percent of the world’s population—and by welcoming hundreds of millions to the middle class. This applies in advanced economies too: from the early 1990s to 2005, before the global financial crisis, GDP per capita rose by 2 to 4 percent per year and real median household market incomes also rose. Growth supports inclusion, part 2: Correcting labor-market inadequacies. In growing economies, government transfers and tax policies can help support incomes for large swaths  of the population. Research from the McKinsey Global Institute found that real market incomes were flat or fell for just 20 to 25 percent of households, after taxes and transfers; before these transfers, some 60 to 70 percent of households saw incomes decline. During the pandemic, while US median household  income dropped 2.9 percent in 2020, the  share of people living in poverty declined , after accounting for government aid. 3 3. Income, poverty and health insurance coverage in the United States: 2020, US Census Bureau, September 14, 2021, census.gov. Growth enables sustainability by encouraging investment. Economic growth strengthens consumer confidence, spending, and demand, all vital elements of a healthy investment climate—which the energy transition is going to need. And as our research on outperforming emerging economies has shown, the capital deepening that results from greater investment spurs productivity and, with it, wages  and growth. Greater inclusion and sustainability promote growth through new demand and investment opportunities. Sustainability drives new business opportunities in domains such as clean technologies. India, for example, could more than quadruple its renewable-energy capacity by 2030 ; we estimate that this could generate some $90 billion in GDP and support about two million jobs in 2030. And inclusion has similarly powerful effects on growth. We estimate that more inclusive access to healthcare could add 0.4 percent to the world’s GDP growth by  2040. More broadly, inclusion spurs demand, as a burgeoning middle class is a key driver  of consumption. Africa has about 200 million young people of working age and will have  close to a billion by 2050. Youth training and development, especially of digital skills,  can vault this group into the middle class—and help close skill gaps in the rest of the world. Sustainability reinforces both inclusion and growth through the ‘energy prize.’ The energy transition will yield a prize of two cross-cutting benefits: lower costs that make energy more accessible, and more productive lives. Over the past ten years, the cost of electricity from renewables fell about 50 to 85 percent . 4 4. “Majority of new renewables undercut cheapest fossil fuel on cost,” International Renewable Energy Agency, June 22, 2021, irena.org. Renewables are now gaining ground in developing economies. In sub-Saharan Africa,  a region with the lowest energy-access rate in the world, decentralized renewable solutions such as rooftop solar are taking root. 5 5. Jan Corfee-Morlot et al., Achieving clean energy access in sub-Saharan Africa , Organisation for Economic Co-operation and Development, January 31, 2019, oecd.org. Lower emissions and reduced air pollution can improve health and allow more people to participate productively in the economy. History has some instructive lessons: after passage of the Clean Air Act in the United States, in 1970, reduced pollution increased the labor-force participation rate for affected individuals and had a positive long-run impact on wages . 6 6. Adam Isen, Maya Rossin-Slater, and Reed Walker, “Every breath you take, every dollar you’ll make: The long-term consequences of the Clean Air Act of 1970,” VoxEU, February 19, 2014, voxeu.org. These three goals—sustainability, inclusion, growth—are daunting. Fortunately they can strengthen and reinforce each other. Squaring the circle If only each element of the circle of sustainable, inclusive growth created purely positive reinforcements to the others, the way forward would be  clear. But the reality is that sustainability, inclusion, and growth also counteract. Squaring this circle means combating three sets of potential counterforces, which could be just as powerful  as the reinforcing loops. Watch A McKinsey Live event on 'Charting a sustainable, inclusive, and growing future' Watch here Growth’s counteractions Growth imposes two major challenges. First is the persistent rise in inequality, which could worsen with growth. Already, 70 percent of the global population live in countries where inequality is mounting. Second is rising resource consumption and emissions. Growth affects inclusion through skill-biased inequality—and its magnitude is set to rise with trends accelerated by the COVID-19 pandemic. Growth in the knowledge-based economy  has stoked demand for higher-level cognitive, technological, and socioemotional skills — a demand not matched by the supply of workers with such skills. As a result, a skill-biased inequality in many countries has sprung up. In the United States, for example, wages for middle-income jobs grew by 1.1 percent between 2000 and 2018, whereas wages for high-pay and low-pay workers grew much faster , at 7.3 and 5.3 percent, respectively. As the pandemic accelerates digitization and automation, almost all growth in labor demand could occur in high-wage occupations . The number of workers who would need to make occupational transitions by 2030 in order to stay employed would increase by up to 25 percent , including—for the first time—many low-wage workers. Growth counteracts sustainability through greater resource consumption. The global “material footprint” —that is, the raw materials used to make the goods that we consume—rises in correlation with GDP growth. 7 7. The Sustainable Development Goals Report 2019 , United Nations, 2019, unstats.un.org. As growth expands in emerging countries, the problem of an increasing global material footprint could get worse. According to the World Bank , about  10 percent of the world’s people still have  no access to electricity, and 2.6 billion people lack access to clean cooking solutions. 8 8. “Report: Universal access to sustainable energy will remain elusive without addressing inequalities,” World Bank press release, June 7, 2021, worldbank.org. If increased demand from a globally expanding consuming class is not accompanied by improvements in resource efficiency, this will put an even heavier burden on the planet. Sustainability’s counteractions Trillions in capital are needed for energy investment to achieve the goal of net-zero emissions by 2050. If consumers and businesses shoulder the burden, near-term growth and inclusion could suffer,  even though the longer-term benefits are clear. If costs are passed on to consumers, energy prices could rise well before the gains are eventually reaped, and if costs are passed on to businesses, the profitability of whole sectors could suffer. This dynamic sets up the potential for two counteractions: uneven distribution of impact and  a challenge to the goal of inclusion. The energy transition could affect some countries and sectors severely. Oil- and natural-gas-producing economies could see their annual per capita income from these products fall by about 75 percent by the 2030s , according to the International Energy Agency. 9 9. Net Zero by 2050 , International Energy Agency, July 2021, iea.org. Lower-income countries will be disproportionately exposed because they have a higher share of emissions-intensive sectors and will also  need to make higher investments relative to their GDP. Those sectors include power, automotive, construction, and others, all of which will be intensely affected, as will supply chains. Lower-income households are disproportionately vulnerable. In Europe,  recent rises in energy prices—the vanguard of the energy transition—are falling heavily  on low-income households, prompting some governments, including in Spain, to provide subsidies . 10 10. “Spain targets energy firms as European bills surge,” BCC, September 14, 2021, bbc.com. And while the transition could lead  to some 18 million more jobs in the green economy , according to International Labour Organization estimates, many people,  especially lower-income workers, will need to retrain to qualify for these new jobs. 11 11. Greening with jobs—World Employment and Social Outlook 2018 , International Labour Organization, May 14, 2018, ilo.org. Inclusion’s potential counteractions The positive spillovers of inclusion are indisputable and well documented: greater workforce participation, higher creativity, more capital allocated to children’s needs. However, poorly conceived measures to boost inclusion can have unintended negative consequences that can include distorted product markets, reduced investment, or faster environmental depletion. For example, in developing economies, free or highly subsidized nonvolumetric pricing of electricity used to pump water can lead to groundwater depletion . 12 12. Bekele Shifraw, “Addressing groundwater depletion: Lessons from India, the world’s largest user of groundwater,” World Bank Independent Evaluation Group, August 23, 2021, ieg.worldbankgroup.org. Efforts to achieve equality can also backfire if they become a box-ticking exercise, or a quota-driven program, which may fail to address the root causes of inequality. As a  result, the goal of achieving a fairer workplace or society may not be achieved, and outcomes may even worsen for certain groups. As in the pandemic, we will need multiple experiments, unprecedented speed in scaling successful ones, and broad participation across actors. Starting here, starting now: A proposal Achieving a future that is sustainable and inclusive and growing is so compelling an idea that today’s leaders owe it to future generations to act immediately. Such a feat cannot be left to enlightened self-interest: if it were that easy, the problem would already have been solved. We see six key challenges that will need to be tackled—with success or failure hinging on how effectively these challenges are met. How to unlock growth supported by higher productivity of an additional 1.0 to 1.5 percentage points of GDP per year, at a global scale with  the same urgency that we have seen during the COVID-19 pandemic? How to reduce the transition costs of decarbonization by, say, $500 billion to $1 trillion of the $5 trillion in required spending per year through technological innovation and smart-portfolio choices? How to finance and smooth the cost of the energy transition , country by country and sector by sector, in a way that won’t stifle growth? How to reskill and re-employ more than 100 million workers who are in stagnating or shrinking occupations as a result of technological change, including the many millions who will likely be displaced by energy transitions? How to strengthen the social contract by achieving basic needs for median households , including affordable housing, healthcare, and energy—needs that are unmet for many of these families in both advanced and developing countries—in a way that attracts private-sector innovation and supply? How to support the most vulnerable population segments —for example, the poorest one-fifth of the global population—that struggle with access and affordability in areas such as nutrition, water, energy, education, and financial capital? Answering these six questions would negate  the counterforces mentioned earlier and allow the virtuous cycle to flow unimpeded. But important obstacles, linked to incentives, stand in the way. First is what Mark Carney has called “the tragedy of horizons” : today’s leaders collectively need to take action today for returns that will accrue only over time. 13 13. “Breaking the tragedy of the horizon—climate change and financial stability—speech by Mark Carney,” Bank of England, September 29, 2015, bankofengland.co.uk. Second is the tragedy of the commons: for collective action, especially on environmental sustainability, all invested parties must look past their parochial interests and fight for the  common good. No stakeholder can solve all these problems on  their own. A clear road map, with buy-in from others, is paramount, as is a framework of incentives that balance short- and long-term horizons and interests across value-chain elements, economic sectors, countries, and regions. As in the case of the pandemic, tackling these challenges successfully will require multiple experiments, unprecedented speed in scaling successful ones, and broad participation across actors. Governments will need to orchestrate a resilient transition—to manage risks, smooth costs, and avoid cascading crises in response to actions taken. On the business side, more companies and CEOs will need to enter the arena, to engage deeply in the design of policies, and to contribute their market knowledge. They will need to be open and realistic about the challenges, while also setting ambitious goals to create positive impact for their customers, workforces, societies, and the environment. Their capacity for innovation can and must be harnessed to shift the frontier of what’s possible and to help achieve what may seem unachievable. If companies don’t engage well and honestly, younger generations of workers will hold them accountable. When it comes to achieving sustainable, inclusive growth, it is crucial first to fully recognize both  the reinforcing as well as the counteracting loops. Then the conversation must move from agreeing  on the targets—for who would not agree to such a tantalizing vision—to understanding how to solve the tough problems that stand in the way. For our part, we have put our hypotheses on those problems at the top of our research agenda and look to learn even more from the leaders of the global organizations we work with who are “making a dent in the universe” through sustainable, inclusive growth. We hope that the ways in which we’ve sketched out the forces and counterforces here contributes  to our collective understanding. With that, it may be possible to start to move toward a sustainable and inclusive and growing global economy. If we don’t focus on the and , we won’t achieve  the end. About the author(s) Bob Sternfels Global Managing Partner, Bay Area Tracy Francis Senior Partner, São Paulo Anu Madgavkar McKinsey Global Institute Partner, New Jersey LinkedIn Sven Smit Chairman and Director, McKinsey Global Institute, and Senior Partner, Amsterdam LinkedIn The authors wish to thank Peter Gumbel and Daniel Pacthod for their contributions to this article. This is the first in a series of articles devoted to sustainable and inclusive growth. This article was edited by Mark Staples, an executive editor in the New York office. Explore a career with us Search Openings Related Articles Article Driving Holistic Impact Report The case for inclusive growth Article - McKinsey Quarterly Climate math: What a 1.5-degree pathway would take Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Locations |  | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Our Offices McKinsey has offices in 130+ cities in 65+ countries Africa Angola Luanda Avenida Lenine, Torres Oceano, Torre A, 21ºA Ingombota Luanda, Angola Tel: +244 226 425 400 Fax: +244 226 425 401 Map it McKinsey Angola (Português) McKinsey Angola (English) Egypt Cairo Nile City Towers, North Tower, Floor 23 P.O.Box 11624 Cairo, Egypt Tel: +2 (02) 21292800, 801 Map it English عربى Ethiopia Addis Ababa Bole Medhanealem Street OASIS Building 1st floor right Bole sub-city, Kebele:03/05 House No. New Addis Ababa Ethiopia Map it Kenya Nairobi 2nd Floor, Mzima House, Mzima Springs Road PO Box 66747-00800 Nairobi Kenya Tel: +254 709 818 000 Map it Morocco Casablanca McKinsey & Company Morocco Tour CFC Immeule 58 Quartier Casa-Anfa 7e étage Bloc A et B 20036 Casablanca- Morocco Tel: +212 5 22 88 68 00 Fax: +212 5 22 88 67 00 Map it Nigeria Lagos Heritage Place 21 Lugard Avenue 6th Floor Ikoyi, Lagos, Nigeria Tel: +234 1 46 33 500 Fax: +234 1 46 33 570 Map it South Africa Johannesburg 140 West Street Sandown Sandton South Africa 2196 Tel: +27 (11) 506 8000 Fax: +27 (11) 506 9561 Map it Johannesburg Client Capabilities Hub 140 West Street Sandown Sandton South Africa 2196 Tel: +27 (11) 506 8000 Fax: +27 (11) 506 9561 Map it Asia-Pacific Australia Brisbane Suite 4, L20 10 Eagle Street Brisbane, Queensland, 4000 Australia Tel: +61 (7) 3221-2564 Map it Canberra Unit 7 and 8, Level 2, 11 National Circuit, Barton 2600 ACT, Australia Tel: +61 2 8273 6451 Map it Melbourne Level 31, South Tower 80 Collins Street Melbourne, Victoria 3000 Australia Tel: +61 (3) 9659 3100 Fax: +61 (3) 9659 3200 Map it Perth Level 3 Newspaper House 129 St. Georges Terrace Perth, Western Australia, 6000 Australia Tel: +61 (8) 9323 7400 Fax: +61 (8) 9323 7500 Map it Sydney 88 Phillip Street, 35th Floor, Sydney, New South Wales, 2000 Australia Tel: +61 (2) 8273 1600 Fax: +61 (2) 8273 1700 Map it Azerbaijan Baku 153 Neftchiler Avenue, AZ 1010 Port Baku Tower, 8th Floor Baku Azerbaijan Map it Greater China Beijing 20/F South Tower Beijing Kerry Centre 1 Guanghua Road Chaoyang District Beijing 100020 Tel: +86 (10) 6561 3366 Fax: +86 (10) 8529 8038 Map it McKinsey China (English) McKinsey China (中文) Chengdu Room 1508, Tower 2 IFS 3 Duan Hong Xing Road Chengdu, 610021 Tel: +86 (28) 6279 6000 Fax: +86 (28) 6632 6811 Map it McKinsey China (English) McKinsey China (中文) Hong Kong 40 Floor, ICBC Tower 3 Garden Road Hong Kong SAR Tel: +852 2868 1188 Fax: +852 2845 9985 Map it McKinsey China (English) McKinsey China (中文) Shanghai 21F, 3 Corporate Avenue 168 Hubin Road Shanghai 200020 Tel: +86 (21) 6385 8888 Fax: +86 (21) 6386 2000 Map it McKinsey China (English) McKinsey China (中文) Shanghai Client Capabilities Hub 21F, 3 Corporate Avenue 168 Hubin Road Shanghai 200020 Tel: +86 (21) 6385 8888 Fax: +86 (21) 6386 2000 Map it Shenzhen Floor 39, Tower 3, Kerry Plaza, Zhongxinsi Road, Futian Area, Shenzhen 518048 Tel: 86 (755) 3397 3300 Fax: 86 (755) 3397 3355 Map it McKinsey China (English) McKinsey China (中文) Taipei 47/F, 7 Xin-Yi Road Section 5 Taipei 101 Tower Taipei 110 Tel: +886 (2) 8758 6700 Fax: +886 (2) 8758 7700 Map it McKinsey China (English) McKinsey China (中文) India Bengaluru 6th Floor, 1 SOBHA No: 50, St. Marks Road, Bengaluru 560001 India Tel: +91 (80) 66744000 Fax: +91 (80) 66744400 Map it Bengaluru - New Ventures Plot No. 148, EPIP Phase II Whitefield Bengaluru 560 066 Karnataka, India Map it Bengaluru - Service Center Nalapad Brigade Center Whitefield Rd, Mahadevapura ITPL Road Bengaluru, Kamataka 560048 India Map it Chennai Alamelu Terrace, 5th Floor, 163, Anna Salai, Chennai 600 002 Tamil Nadu, India Tel: +91 (44) 6641 3000 Fax: +91 (44) 6641 3333 Map it Chennai Client Capabilities Hub World Trade Center 5/142, Tower B, 17th-19th Floor, Rajiv Gandhi Salai, OMR, Perungudi, Chennai - 600 096 Tel: +91 (44) 42265000 Map it Chennai - Service Center World Trade Center 5/142, Tower B, 17th-19th Floor, Rajiv Gandhi Salai, OMR, Perungudi, Chennai 600 096 Tel: +91 (44) 42265000 Map it Gurugram Plot No. 4, Echelon Institutional Area, Sector 32, Gurugram 122 001 Haryana, India Tel: +91 (124) 661 1000 Fax: +91 (124) 661 1400 Map it Gurugram Client Capabilities Hub 3rd Floor Block III Vatika Business Park Sector 49 Sohna Road Gurugram 122018 India Tel: +91 (124) 333 1000 Map it Mumbai 21st Floor, Express Towers, Nariman Point, Mumbai 400 021 Maharashtra, India Ground floor, North Avenue 1, Maker Maxity, Bandra Kurla Complex Rd, BKC, Bandra East, Mumbai 400 051 Maharashtra, India Tel: +91 (22) 6630 2000 Fax: +91 (22) 6630 2444 Map it Thiruvananthapuram - Service Center Carnival Technopark Unit 5B 5th Floor Technopark Road, Trivandrum 695 581 Kerala, India Map it Indonesia Jakarta Wisma GKBI 40th Floor Jl. Jend. Sudirman 28 Jakarta 10210 Indonesia Tel: +62 (21) 5798 6600 Fax: +62 (21) 574 1540 Map it Japan Kansai Grand Front Osaka, Tower B 3-1 Ofuka-cho, Kita-ku, Osaka 530-0011 Tel: +81 (6) 6563 7200 Fax: +81 (6) 6563 7201 Map it McKinsey Japan (日本語) McKinsey Japan (English) Tokyo 32F Ark Hills Sengokuyama Mori Tower 9-10, Roppongi 1-chome, Minato-ku Tokyo 106-0032 Japan Tel: +81 (3) 5562 2100 Fax: +81 (3) 5562 2200 Map it McKinsey Japan (日本語) McKinsey Japan (English) Kazakhstan Almaty Esentai Tower 77/7 Al-Farabi Avenue, 15th floor | A15E3H4 050000 Almaty, Kazakhstan Tel: +7 7172 69 1277 Map it Astana Business Center "Q" Kabanbay Batyr Ave 15 A, Block A, 2nd floor 01000 Astana Republic of Kazakhstan Tel: +7 717 2 691200 Fax: +7 717 2 691250 Map it Malaysia Kuala Lumpur Level 27, Menara 3 Petronas, Persiaran KLCC, 50088 Kuala Lumpur Tel: +60 (03) 2 382 5500 Fax: +60 (03) 2 382 5600 Map it Myanmar Yangon New Zealand Auckland Level 13 188 Quay Street Auckland 1010 New Zealand Tel: +64 9 357 5800 Map it Pakistan Karachi Ocean Tower, 24th floor G-3, Khyaban-e-Iqbal Block 9 Clifton Karachi 75600 Pakistan Tel: +92 21 3581 1000 Fax: +92 21 3581 1800 Map it English عربى Philippines Manila 7/F Zuellig Building Makati Avenue Corner Paseo de Roxas Makati City, 1225 Philippines Tel: + 63 (2) 8459 3900 Fax: +63 (2) 8459 3920 Map it Singapore Singapore One Raffles Quay Level 24, South Tower Singapore 048583 Singapore Tel: +65 6586 4900 Fax: +65 6836 9200 Map it South Korea Seoul McKinsey, Incorporated 54Fl. Three IFC 10, Gukjegeumyung-ro Yeongdeungpo-gu Seoul, 07326, Republic of Korea Tel: +82 (2) 398 2500 Fax: +82 (2) 2128 3500 Map it Sri Lanka Colombo One Galle Face Tower Level 24, Unit 2406 & 2407 1A Center Road, Galle Face, Colombo 02 Tel: +94 117 610 200 - 399 Map it Thailand Bangkok M. Thai Tower, 24th floor All Seasons Place 87 Wireless Road Bangkok 10330 Thailand Tel: +66 (2) 694 8500 Fax: +66 (2) 694 8501 Map it Vietnam Hanoi 24th Floor, Capital Place Building 29 Lieu Giai Street Ba Dinh District Hanoi Tel: +84 24 3941 6000 Map it Central & South America Argentina Buenos Aires Av. Leandro N. Alem 855, 24th floor C1001AAD Buenos Aires Argentina Tel: +54 (11) 5 776 3900 Fax: +54 (11) 5 776 3870 Map it McKinsey Argentina (Español) McKinsey Argentina (English) Brazil Rio de Janeiro Praia de Botafogo, 300 Centro Empresarial Botafogo, 13th floor 22250-905 Rio de Janeiro, RJ Brazil Tel: +55 (21) 2559 5000 Fax: +55 (21) 2559 5050 Map it McKinsey Brazil (Português) McKinsey Brazil (English) Salvador Av. Tancredo Neves, 620 Ed. Mundo Plaza, 12th floor 41820-020 Salvador, BA Brazil Tel: +55 (71) 3273-6300 Fax: +55 (71) 3273 6308 Map it McKinsey Brazil (Português) McKinsey Brazil (English) São Paulo Avenida Presidente Juscelino Kubitschek, 1.909 Condomínio São Paulo Corporate Towers, Torre Sul, 21st, 22nd, 23rd floors Vila Nova Conceição 04543-907 São Paulo, SP Brazil Tel: +55 (11) 5189 1400 Fax: +55 (11) 5189 1700 Map it McKinsey Brazil (Português) McKinsey Brazil (English) Chile Santiago Av Apoquindo 2929, 20th floor Las Condes, Santiago - Chile Tel: +56 (22) 353 1800 Fax: +56 (22) 353 1801 Map it McKinsey Chile (Español) McKinsey Chile (English) Colombia Bogotá Carrera 7 No 71-21 Torre B Oficina 1606 11001 Bogotá Colombia Tel: +57 (1) 313 7000 Fax: +57 (1) 313 7079 Map it McKinsey Colombia (Español) McKinsey Colombia (English) Medellín Carrera 42 # 3 Sur – 81 Edificio Milla de Oro, Torre 1 Oficina 404 Medellín 050022 Tel: +57(4) 319 6200 Map it McKinsey Colombia (Español) McKinsey Colombia (English) Costa Rica San José Client Capabilities Hub Zona Franca Ultrapark II, Edificio 10 From Jardines del Recuerdo, 800m West and 300m South Lagunilla de Heredia Costa Rica Tel: +506 2562 1300 Map it Dominican Republic Santo Domingo 95, Winston Churchill Ave. Piantini, Distrito Nacional Blue Mall Tower Fl. 21 Ste. P14-01 Santo Domingo, 10148, Dominican Republic Tel: +1 829-735-0500 Map it Ecuador Quito Av. Simon Bolivar SN y Via Nayon Edificio Ekopark, torre 2 – piso 6 Quito, Ecuador Tel: +593 987330431 Map it Panama Panama City PH Davivienda Tower, Piso 21, Avenida Balboa., Panama, Panama Tel: +507 68151057 Map it McKinsey Panama (Español) McKinsey Panama (English) Peru Lima Las Begonias 415 Piso 16 Oficina.1601 A Torre Begonias Lima 27 Peru Tel: +51 (1) 712 9300 Fax: +51 (1) 712 9339 Map it McKinsey Peru (Español) McKinsey Peru (English) Uruguay Montevideo Luis Bonavita 1294 WTC FZ Torre I, oficina 704 Tel: (+598) 626 4000 Map it McKinsey Uruguay (Español) McKinsey Uruguay (English) Europe Azerbaijan Baku 153 Neftchiler Avenue, AZ 1010 Port Baku Tower, 8th Floor Baku Azerbaijan Map it Austria Vienna - Lixto Falkestraße 1 1010 Vienna Austria Tel: +43 15370 6245 Fax: +43 15370 6244 Map it Lixto (English) Lixto (Deutsch) Vienna Schottenring 19 1010 Vienna Austria Tel: +43 (1) 53 70 60 Fax: +43 (1) 53 70 6700 Map it Belgium Brussels 2 Rue Brederode/Brederodestraat 1000 Brussels Belgium Tel: +32 (2) 645 4211 Map it Brussels Client Capabilities Hub 2 Rue Brederode/Brederodestraat 1000 Brussels Belgium Tel: +32 (2) 645 4211 Map it Brussels - Risk Dynamics 2 Rue Brederode/Brederodestraat 1000 Brussels Belgium Map it Croatia Zagreb Radnička cesta 37/A Zagreb Croatia Tel: +385 (1) 4891 400 Fax: +385 (1) 4891 490 Map it Czech Republic Prague McKinsey & Company, Inc. Prague - organizační složka Milevská 5, 140 00, Praha 4 VAT ID: CZ 272 14 869 Tel: +420 (2) 2141 4111 Fax: +420 (2) 2141 4100 Map it Denmark Copenhagen Ved Stranden 14 1061 Copenhagen K Denmark Tel: +45 3393 3030 Map it Finland Helsinki Eteläesplanadi 18 00130 Helsinki Finland Tel: +358 (9) 6157 100 Fax: +358 (9) 6157 1200 Map it France Lyon - Orphoz 158 Avenue Thiers 69006 Lyon France Map it English Français Paris - Experience Studio 90 Av. des Champs-Élysées, 7th floor 75008 Paris Tel: +33 (0) 1 40 69 14 00 Fax: +33 (0) 1 40 69 93 93 Map it Lyon 24, rue Childebert Lyon France 69002 Tel: +33 (0) 4 72 77 27 00 Fax: +33 (0) 4 72 77 27 27 Map it Paris 90 avenue des Champs-Elysées 75008 Paris France Tel: +33 (0) 1 40 69 14 00 Fax: +33 (0) 1 40 69 93 93 Map it Germany Berlin Budapester Straße 46 10787 Berlin Germany Tel: +49 (30) 88 45 20 Fax: +49 (30) 88 45 2700 Map it Cologne Christophstraße 17 50670 Köln Germany Tel: +49 (221) 20 8 70 Fax: +49 (221) 20 8 7700 Map it Düsseldorf Kennedydamm 24 40476 Düsseldorf Germany Tel: +49 (211) 136 40 Fax: +49 (211) 136 4700 Map it Frankfurt TaunusTurm Taunustor 1 60310 Frankfurt Germany Tel: +49 (69) 71 62 0 Fax: +49 (69) 71 62 5700 Map it Hamburg Am Sandtorkai 77 20457 Hamburg Germany Tel: +49 (40) 3612 10 Fax: +49 (40) 3612 1700 Map it Munich Sophienstraße 26 80333 München Germany Tel: +49 (89) 55 94 0 Fax: +49 (89) 55 94 8700 Map it Munster - 4Tree Am Mittelhafen 10 7th & 8th Floors, Building Section C 48155 Münster Germany Map it Nuremberg - Orpheus Gleissbuehlstr. 2 90402 Nuremberg Germany Map it Stuttgart Dorotheenstraße 6 70173 Stuttgart Germany Tel: +49 (711) 255 35 Fax: +49 (711) 255 3700 Map it Greece Athens 6, Othonos Street 105 57 Athens Greece Tel: +30 (210) 36 72 400 Fax: +30 (210) 36 72 444 Map it Hungary Budapest Andrassy str 23. H-1061 Budapest Hungary Tel: +36 (1) 373 8500 Fax: +36 (1) 373 8600 Map it Ireland Dublin 4th floor 80 Harcourt Street Dublin 2 Ireland D02 F449 Tel: +353 (1) 478 5500 Map it Israel Tel Aviv HaArba`a Towers, South tower,23 fl 30 HaArba`a st. Tel-Aviv 6473926 Israel Tel: +972 (9) 971 8500 Fax: +972 (9) 971 8501 Map it Italy Milan Piazza del Duomo, 31 20122 Milan Italy Tel: +39 (02) 72406 1 Fax: +39 (02) 72001440 Map it Rome Piazza del Popolo, 18 00187 Rome Italy Tel: +39 (06) 420871 Fax: +39 (06) 42013313 Map it Kazakhstan Almaty Esentai Tower 77/7 Al-Farabi Avenue, 15th floor | A15E3H4 050000  Almaty, Kazakhstan Tel: +7 727 355 15 20 Map it Astana Business Center "Q" Kabanbay Batyr Ave 15 A, Block A, 2nd floor 01000 Astana Republic of Kazakhstan Tel: +7 717 2 691200 Fax: +7 717 2 691250 Map it Luxembourg Luxembourg 40 Avenue Monterey 2163 Luxembourg Luxembourg Tel: +352 (26) 56 46 11 Map it Netherlands Amsterdam Amstel 344 1017 AS Amsterdam, The Netherlands Tel: +31 (20) 551 3777 Map it Norway Oslo Dronning Mauds gate 1 PO Box 1683, Vika 0120 Oslo Norway Tel: +47 22 86 25 00 Map it Poland Poznan - Service Center Malta Business Park ul. Abpa A. Baraniaka 88D Building E 61-131 Poznań Poland Tel: +48 61 842 1000 Map it McKinsey Service Center - Poznan (Polski) McKinsey Service Center - Poznan (English) Warsaw Sp. z o.o. sp.k. Budynek Metropolitan Plac Piłsudskiego 2 00-073 Warszawa NIP: 526-00-21-533 Tel: +48 (22) 820 5700 Map it Wroclaw Client Capabilities Hub Plac Nowy Targ 28 50-141 Wroclaw Poland Tel: +48 (71) 393 7100 Map it Portugal Lisbon Client Capabilities Hub Praça Marquês de Pombal, 3A-8 1250-161 Lisboa Portugal Tel: +351 (21) 312 3200 Fax: +351 (21) 315 4365 Map it Lisbon Praça Marquês de Pombal, 3A-8 1250-161 Lisbon Portugal Tel: +351 (21) 312 3200 Map it Romania Bucharest 6 Matei Millo Street, 1st District Millo Building Fl. 7 Bucharest Romania Tel: +40 21 207 83 00 Map it Serbia Belgrade Mutapova 7 Green Square Building, Gallery Belgrade Republic of Serbia Map it Slovakia Bratislava McKinsey & Company, Inc. Slovakia - organizačná zložka Bajkalská 29/C Bratislava 821 01 Tel: 00421 220 906 421 Fax: 00421 220 906 422 Map it Spain Barcelona Avenida Diagonal 579 – 4th floor – Door A 08014 Barcelona Spain Tel: +34 (93) 363 43 00 Fax: +34 (93) 322 20 33 Map it Madrid Sagasta, 33 28004 Madrid Spain Tel: +34 (91) 346 5800 Fax: +34 (91) 310 1076 Map it Sweden Gothenburg Kungsportsavenyn 1 S-411 36 Gothenburg Sweden Tel: +46 (31) 725 40 00 Map it McKinsey Design, Stockholm Studio Missionsvägen 24 167 33 Bromma Sweden Tel: +46 (8) 506 672 00 Map it Stockholm Klarabergsviadukten 70 8th Floor World Trade Center Box 70371 S-107 24 Stockholm Sweden Tel: +46 (8) 700 64 00 Map it Switzerland Geneva Place de Cornavin 7 1201 Geneve Switzerland Tel: +41 (22) 744 20 00 Fax: +41 (22) 744 20 01 Map it Zurich Bleicherweg 30 8002 Zurich Switzerland Tel: +41 (44) 876 80 00 Fax: +41 (44) 876 90 00 Map it Turkey Ankara Tepe Prime İş ve Yaşam Merkezi Mustafa Kemal Mah. Dumlupınar Bulvarı No: 266 A Blok Kat 6 No: 50-51 06800 Çankaya Ankara Turkey Tel: +90 (312) 287 36 60 Map it Istanbul Ferko Signature Büyükdere Caddesi No: 175 Kat: 28-29 Levent – 34394 Turkey Tel: +90 (212) 339 49 00 Fax: +90 (212) 357 04 04 Map it Ukraine Kyiv 17th Floor, Leonardo Business Center Bohdana Khmel'nyts'koho St, 19/21 01030 Kyiv Ukraine Tel: +380 (44) 393-1200 Fax: +380 (44) 393-1201 Map it United Kingdom London The Post Building 100 Museum Street London WC1A 1PB United Kingdom Tel: +44 (20) 7839 8040 Map it London - Finalta Kinnaird House 1 Pall Mall E London United Kingdom SW1Y 5AU Tel: +44 20 7851 9100 Map it London - QuantumBlack The Post Building 100 Museum Street London WC1A 1PB United Kingdom Tel: +44 20 3763 6360 Map it Middle East Bahrain Manama Financial Harbour (West), Level 41 Office No. 4123 King Faisal Highway Manama, Bahrain Tel: +973 (17) 502 830 Fax: +973 (17) 502 832 Map it English عربى Egypt Cairo Nile City Towers, North Tower, Floor 23 P.O.Box 11624 Cairo, Egypt Tel: +2 (02) 21292800, 801 Map it English عربى Kuwait Kuwait C/o Regus Kuwait Al Hamra Tower, Office No. 31, 35th Floor East Maqwa, Kuwait City. Kuwait Tel: +965 222 03 538 Fax: +965 222 03 511 Map it English عربى Pakistan Karachi Ocean Tower, 24th floor G-3, Khyaban-e-Iqbal Block 9 Clifton Karachi 75600 Pakistan Tel: +92 21 3581 1000 Fax: +92 21 3581 1800 Map it English عربى Qatar Doha Office 1203, 12th Floor Tower 2 – The Gate Maysaloun St., WestBay Doha, Qatar Tel: +974 4041 5200 Fax: +974 4041 5300 Map it English عربى Saudi Arabia Riyadh Kingdom Tower 54th Floor P.O. Box 54517 Riyadh 11524 Saudi Arabia Tel: +966 (11) 212 3600 Fax: +966 (11) 212 3800 Map it English عربى Riyadh Client Capabilities Hub Kingdom Tower 54th Floor P.O. Box 54517 Riyadh 11524 Saudi Arabia Tel: +966 (11) 212 3600 Fax: +966 (11) 212 3800 Map it United Arab Emirates Abu Dhabi 2304 International Tower, Capital Center Al Karamah Street , P.O Box 52093 Abu Dhabi United Arab Emirates Tel: +971 2 6942500 Fax: +971 2 6942600 Map it English عربى Dubai Client Capabilities Hub Dubai International Financial Center Gate Precinct 4, Level 4 P.O. Box 33538 Dubai United Arab Emirates Tel: +971 (4) 389 9000 Fax: +971 (4) 389 9500 Map it Dubai Dubai International Financial Center Gate Precinct 4, Level 4 P.O. Box 33538 Dubai United Arab Emirates Tel: +971 (4) 389 9000 Fax: +971 (4) 389 9500 Map it English عربى North America Canada Calgary Royal Bank Building 335 - 8th Avenue SW, Suite 2210 Calgary, AB T2P 1C9 Canada Tel: +1 (403) 444 8000 Fax: +1 (403) 444 8050 Map it McKinsey Canada (English) McKinsey Canada (Français) Montréal 1250, boulevard René-Lévesque Ouest Bureau 4430 Montréal, QC H3B 4W8 Canada Tel: +1 (514) 939 6800 Fax: +1 (514) 939 6810 Map it McKinsey Canada (English) McKinsey Canada (Français) Toronto 110 Charles Street West Toronto, ON M5S 1K9 Canada Tel: +1 (416) 313 3700 Fax: +1 (416) 313 2999 Map it McKinsey Canada (English) McKinsey Canada (Français) Toronto - PriceMetrix 155 Wellington Street West Suite 2901 Toronto, ON M5V 3H6 Canada Tel: +1 (416) 955 0514 Fax: +1 (416) 955 0501 Map it Vancouver 2840-650 W. Georgia St Vancouver, BC V6B 4N8 Canada Tel: +1 (604) 235 7800 Fax: +1 (604) 235 7816 Map it McKinsey Canada (English) McKinsey Canada (Français) Mexico Mexico City Pedregal 24, Torre Virreyes Piso 23° Molino del Rey 11040 Mexico City Mexico Tel: +52 (55) 53 51 77 00 Fax: +52 (55) 53 51 78 00 Map it Monterrey Av. Ricardo Margain Zozaya 444 Torre Norte Piso 4° Pte. Col. Valle del Campestre 66265 San Pedro Garza Garcia, NL Mexico Tel: +52 (81) 4646-9000 Map it United States Atlanta 725 Ponce de Leon Ave Suite 700 Atlanta, GA 30306 United States Tel: +1 (404) 335 3000 Fax: +1 (404) 521 1743 Map it Atlanta Client Capabilities Hub 725 Ponce de Leon Ave Suite 700 Atlanta, GA 30306 United States Tel: +1 (404) 335 3000 Fax: +1 (404) 521 1743 Map it Austin 1221 S. Congress Ave. Bldg 1; Suite 200 Austin, TX 78704 United States Tel: +1 (737) 348-2400 Map it Bay Area - San Francisco 555 California Street Suite 4700 San Francisco, CA 94104 United States Tel: +1 (415) 981-0250 Fax: +1 (415) 318-5200 Map it Bay Area - Silicon Valley 889 Winslow St, Suite 300 Redwood City, CA 94063 Tel: +1 (650) 494 6262 Fax: +1 (650) 842 5800 Map it McKinsey Design, San Francisco Studio 2012 16th Street San Francisco, CA 94103 United States Tel: +1 (415) 252.4388 Fax: +1 (415) 252.4389 Map it Boston 280 Congress Street Suite 1100 Boston, MA 02210 United States Tel: +1 (617) 753 2001 Map it Carolinas - Charlotte 300 South Tryon Street Suite 1100 Charlotte, NC 28202 United States Tel: +1 (704) 954 5000 Fax: +1 (704) 376 8191 Map it Chicago 300 East Randolph Street Suite 3100 Chicago, IL 60601 United States Tel: +1 (312) 551 3500 Fax: +1 (312) 551 4200 Map it Chicago - Design to Value Lab 1229 West Washington Blvd. 3rd floor Chicago, IL 60607 United States Map it Chicago - Service Center 5600 North River Road Suite 450 Rosemont, IL 60018 United States Map it McKinsey Design, Chicago Studio 1229 West Washington Blvd. 4th Floor Chicago, IL 60607 United States Tel: +1 (312) 929-2750 Fax: +1 (312) 265-0595 Map it Cleveland 950 Main Avenue Suite 1200 Cleveland OH 44113 United States Tel: +1 (216) 274 4300 Fax: +1 (216) 274 4500 Map it Dallas McKinney & Olive 2021 McKinney Ave #1800 Dallas, TX 75201 United States Tel: +1 (214) 665 1200 Fax: +1 (214) 871 7472 Map it Dallas Client Capabilities Hub 2021 McKinney Ave #1800 Dallas, TX 75201 Tel: +1 (214) 665 1200 Fax: +1 (214) 871 7472 Map it Denver 1800 Larimer St Suite 2200 Denver, CO 80202 United States Tel: +1 (303) 723 6300 Fax: +1 (303) 723 6350 Map it Detroit 500 Woodward Avenue Suite 2850 Detroit, MI 48226 United States Tel: +1 (313) 202 5200 Fax: +1 (313) 202 5300 Map it GCI Analytics 3111 Clairmont Road Suite A, Atlanta, GA 30329 United States Tel: +1 (770) 300 9400 Map it Houston 609 Main Street Suite 2300 Houston, TX 77002 United States Tel: +1 (713) 751 4000 Fax: +1 (713) 650 1050 Map it Miami Two Brickell City Centre 78 SW 7th Street Suite 1000 Miami, FL 33130 Tel: +1 (305) 961 8600 Fax: +1 (305) 372 3981 Map it Minneapolis 800 LaSalle Avenue, Suite 2350 Minneapolis, MN 55402 United States Tel: +1 (612) 371 3100 Fax: +1 (612) 371 3131 Map it Miramar - Service Center 3350 SW 148th Avenue Suite 403 Miramar, FL 33027 United States Map it New Jersey 1 Deforest Avenue Suite 300 Summit, NJ 07901 United States Tel: +1 (973) 549 6600 Fax: +1 (973) 549 6300 Map it New York 3 World Trade Center 175 Greenwich Street New York, NY 10007 United States Tel: +1 (212) 446 7000 Map it New York  - Experience Studio 195 Broadway, 19th Floor New York, NY 10007 Map it New York - New Ventures 195 Broadway 19th Floor New York, NY 10007 United States Tel: +1 (212) 446 7000 Map it New York - Service Center 711 Third Avenue 4th Floor New York, NY 10017 United States Tel: +1 (212) 415 1387 Map it Philadelphia The Cira Centre 2929 Arch Street Suite 1400 Philadelphia, PA 19104-2892 United States Tel: +1 (215) 594 4500 Fax: +1 (215) 594 4478 Map it Pittsburgh One PPG Place Suite 2350 Pittsburgh, PA 15222 United States Tel: +1 (412) 804 2700 Fax: +1 (412) 804 2800 Map it Seattle 1420 Fifth Avenue Suite 4300 Seattle, WA 98101 United States Tel: +1 (206) 667 0100 Fax: +1 (206) 667 0111 Map it Southern California 2000 Avenue of the Stars, Suite 800N Los Angeles, CA 90067 United States Tel: +1 (424) 249 1000 Fax: +1 (424) 249 1005 Map it St Louis 7733 Forsyth Blvd Suite 825 Saint Louis, MO 63105 United States Map it Stamford Two Harbor Point Square 100 Washington Boulevard Stamford, CT 06902 United States Tel: +1 (203) 977 6700 Map it Tampa - Service Center 140 Fountain Parkway Floor 3 & Floor 8 St. Petersburg FL, 33716 United States Map it Waltham Client Capabilities Hub 404 Wyman Street Suite 300 Waltham MA–02451 United States Tel: +1 (781) 522 3200 Map it Washington DC 1200 Nineteenth Street NW Suite 1000 Washington, D.C., 20036 United States Tel: +1 (202) 662 3100 Fax: +1 (202) 662 3175 Map it McKinsey launches new Europe research at firm’s largest-ever media day | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog McKinsey launches new Europe research at firm’s largest-ever media day From left, Fabian Billing, managing partner, McKinsey Germany and Austria; Clarisse Magnin, managing partner, McKinsey France; Tunde Olanrewaju, managing partner, McKinsey UK, Ireland and Israel Hide Show Europe Innovation September 29, 2022 McKinsey recently held its second Europe Media Day in our Brussels office. Our largest media day ever, the event drew dozens of journalists, including bureau chiefs, columnists, and editors from 15 countries, who spent the day moderating panels and participating in discussions and interviews. McKinsey partners and experts representing 12 countries shared insights with the standing-room-only audience on topics that reflected the wide range of our research, from inflation to decarbonization, innovation, security, and economic inclusion. It was also an opportunity to preview our latest perspective on Europe, “ Securing Europe’s competitiveness: Addressing its technology gap ,” published with McKinsey Global Institute (MGI). Video Lifting Europe’s Ambition McKinsey also used the event to launch its new, pan-European publishing series, Lifting Europe's Ambition . As part of his opening remarks, Europe managing partner Magnus Tyreman introduced the flagship series, explaining to journalists that the series will be a multi-year, multi-media project covering the most important issues on European business leaders’ and policymakers’ minds. It features a dedicated microsite where visitors can find all of McKinsey’s European insights in one place, along with a video interview series featuring McKinsey leaders and external experts. Magnus continued with his opening remarks by noting the meeting had been planned and cancelled twice due to the pandemic; for this third time, the moment was auspicious. “Never has there been so much uncertainty. We are experiencing a convergence of issues—war, inflation, an energy crisis, and geopolitical tensions are at their highest level,” explained Magnus Tyreman , the managing partner of McKinsey Europe in his opening remarks. “We will not return to normal. This is the end of an era.” The second panel examined how the war in the Ukraine was upending the development of net zero pathways; another discussion posited the industries in which Europe could lead; in the closing session, the managing partners from France, Germany and the UK shared findings on what keeps CEOs up at night in their respective economies. Lifting Europe’s Ambition Learn more While European leaders have shown great resolve in responding to the war  and resulting shocks, they will need to build the same momentum to face the region's ́slow-motion technology crisis, as outlined in our report. Europe has many high-performing companies, but in aggregate, they are falling seriously behind. Between 2014 and 2019, large European companies with more than a billion in revenue were 20 percent less profitable, grew revenue 40 percent more slowly, and spent 40 percent less on R&D, compared with US firms. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Solveigh Hieronimus, McKinsey senior partner “In this study, instead of looking at Europe through the lens of sectors, we look at it through the lens of 10 transversal or interlinking technologies,” explains Solveigh Hieronimus , a senior partner. “Europe leads on only two of these—next generation materials and clean tech. We have strong STEM talent…a strong bio industry…we should be able to gain excellence here. However, there are some technology areas where it is too late to lead, particularly where we have not invested in the past few years.” While this trend is not new, the issue now is that technology is overtaking almost every sector, and reshaping competitive dynamics. Eric Hazan , a senior partner, outlined the reasons for this slow rate of growth: “The region leads in so many areas: cleantech, materials, the bio revolution, advanced industries, and more. Our human capital is second to no one. But there’s the question of scale, particularly when it comes to technology investment. There are few established ecosystems for sharing expertise and data, and our regulatory environment and scarce risk capital don’t encourage disruption and innovation.” We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Tunde Olanrewaju, managing partner, McKinsey UK, Ireland and Israel “It's not surprising that in such a challenging environment, 'resilience' comes up again and again as business concern,” points out Maria del Mar Martinez , a senior partner. “For businesses, this means having the ability and all of the levers—resources, flexibility, redundancy—to deal with adversity and shocks and continually adapt. Companies have to prepare not only for the coming recession, but also how to recover after it.” At times throughout the day, the discussion turned existential. “Are we being too pessimistic? too focused on fragmentation and regulations? Maybe we should be looking more at our mindsets, correcting our self-image, and raising our levels of ambition?”  These were just some of the questions raised by the audience. Everyone agreed that it will take bold action to solve the challenges facing Europe’s business leaders and policymakers and to lift the continent’s ambitions. Related Blog Partnering with the world’s leading tech companies to accelerate innovation March 7, 2022 McKinsey’s open ecosystem of alliances and acquisitions helps power our clients’ transformations across areas like digital, technology, sustainability, and more. Home | Careers | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content My Dashboard Welcome to your McKinsey Career Search Jobs Make Your Own McKinsey A career that fits your calling Problem solvers and creative thinkers. Engineers and new business builders. Put your talents to use where opportunities are limitless and every day makes a difference. Whether you’re an experienced professional or a recent graduate, working with McKinsey could be a challenging and rewarding next step in your career. Working with us Why you should join McKinsey Hear from our leaders Some of McKinsey’s senior leaders share our purpose, mission and values and encourage you to apply today. 2021 ESG Report Our 2021 ESG Report reflects our progress to accelerate sustainable and inclusive growth – for our clients, our colleagues, and our communities. From catalyzing decarbonization, building inclusive workforces, and supporting nonprofits, we’re proud of our accomplishments. Your Career Interview preparation Learn about our interview process We are excited to get to know you and help you get to know us  through our interview process. What we look for For students McKinsey on campus We’re looking for exceptional people from colleges and universities around the world. Find your university MEANINGFUL WORK Accelerating Sustainable and Inclusive Growth We’ve always been big believers in the growth potential of our clients. Today, we’re privileged to work with organizations that are on a new growth journey, one that pursues sustainability, inclusion, and economic growth, all at the same time Inclusive Economy 7 in 10 people live in societies with growing inequality. That’s why we’re taking action to create a more inclusive economy through our work with clients, and in local communities. Careers blog At the end of the day, we all want to know who we'll work with and what makes someone successful at McKinsey. Meet some of our people here. New at McKinsey blog Read a collection of stories about our people, our capabilities, our research, and the ever-changing face of our firm. Reach and Impact Learn more about how we are making change happen around the world. Learn more Disability accommodations in the application process McKinsey invites all interested and qualified candidates to apply for employment opportunities. If you have a disability that prevents or limits your ability to use or access the site, or if you require any other accommodation in the application process due to a disability, you may request a reasonable accommodation. To make a request, please contact our recruiting team and we will be happy to assist you. For further details, visit our FAQs on how to navigate McKinsey with a disability. Important fraud alert McKinsey has become aware of scams involving false offers of McKinsey employment. The scams and false offers use imposter websites, email addresses, and text messages. None of these offers are legitimate, and McKinsey's recruiting process never involves interviewing via instant message, nor requires candidates to purchase products or services, or process payments on our behalf. Equal Opportunity Employment Statement McKinsey & Company is an equal opportunity employer. We are committed to a work environment that supports, inspires, and respects all individuals and in which personnel processes are merit-based and applied without discrimination on the basis of race, color, religion, sex, sexual orientation, gender identity, marital status, age, disability, national or ethnic origin, military service status, citizenship, or other protected characteristic. US federal Transparency in Coverage Rule The McKinsey health plan in the United States is in compliance with the federal Transparency in Coverage Rule. Machine-readable files (MRFs) are available in response to the federal Transparency in Coverage Rule and include negotiated service rates and out-of-network allowed amounts between health plans and healthcare providers. The machine-readable files are formatted to allow researchers, regulators, and application developers to more easily access and analyze data. Learn more Find your ideal job Already started your application? Sign in here. Find your ideal job 500+ Roles Available Jobs Already started your application? Sign in here. McKinsey and AWS expand existing collaboration to tackle the next horizon of cloud transformation | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog McKinsey and AWS expand existing collaboration to tackle the next horizon of cloud transformation Digital September 27, 2022 There’s a $1 trillion prize up for grabs today in cloud technology, and McKinsey and AWS have teamed up to help the boldest leaders secure a large share of it. Building on a proven history of client impact, our organizations have now blended select Cloud by McKinsey offerings with AWS Professional Services , giving clients new ways to identify the right cloud transformation opportunities from day one—and deliver them at greater speed and scale than ever before. “From day one, the essence of our collaboration with AWS has always been to help clients tackle their toughest business and technical problems through cloud transformations,” says Peter Dahlstrom , McKinsey senior partner and global head of Alliances and Acquisitions. “But over the last three years, it has proven to be so much more than that. Today, we’ve become true impact partners to our clients as we drive full end-to-end, cloud-enabled business transformations that build capabilities clients need to fuel resilience and drive growth.” McKinsey and AWS first started working together to help organizations accelerate transformations through digital-native, disruptive capabilities enabled by cloud technology. Together, we help clients ‘innovate’—through advanced analytics, IoT, automation at scale, and more—and ‘rejuvenate’ by using cloud to lower traditional legacy IT costs and risks. By combining McKinsey’s expertise and impact approach with AWS’ growing cloud innovations and ‘learn by doing’ model, our relationship has already delivered impact in nearly 100 client engagements to date. Video A winning collaboration A look at McKinsey and AWS’ work for Emirates Team New Zealand. The collaboration is part of McKinsey’s open ecosystem of alliances and acquisitions and leverages the AWS Partner Network of more 100,000 members. Working together, McKinsey and AWS have helped a pharmaceutical company transform the patient experience by enabling remote triage through a virtual agent that is expected to handle more than one million calls annually. We also helped an automotive manufacturer consolidate real-time data from 30,000 locations in the supply chain, driving a 30 percent efficiency improvement by 2025 and $1 billion in anticipated supply-chain savings. And using Amazon SageMaker, we helped Emirates Team New Zealand successfully defend their America’s Cup title by creating a new sailor for their team: an AI training bot . By introducing a new suite of joint offerings with AWS Professional Services, we have a new way to help our clients solve their toughest technical problems and build capabilities over time. Our new offerings include SpeedLab for Customer Experience and SpeedLab for Supply Chain. Additionally, Jumpstart Cloud helps organizations set up the right technical foundations and tools for rapid cloud adoption, and Jumpstart Mainframe helps organizations unlock business value when migrating from their legacy systems. “Our work at AWS is guided by our leadership principles, with customers at the core of everything we do. Combined with McKinsey's values and industry expertise this partnership presents an opportunity to raise the bar by bringing the best of AWS’ offerings and capabilities to customers. We are excited about the possibilities ahead in unlocking business value for our customers” said Al Opher , vice president, Worldwide Head of AWS Professional Services. While the purpose of our collaboration remains the same—to help business leaders build the technology and capabilities they need to drive enduring change—we now bring in the deepest and broadest range experts in McKinsey and AWS to enable end-to-end execution from day one at greater scale. “We are highly committed to our partnership with McKinsey and look forward to expanding across multiple industries to solve complex problems and drive significant business impact,” said Ruba Borno , vice president, Worldwide Channels & Alliances at AWS. “McKinsey’s expertise in key technologies and industry knowledge, coupled with AWS’ role as the leading cloud provider, is what customers are looking for to innovate and achieve their business ambitions.” As the capabilities from McKinsey and AWS continue to grow—at pace and alongside the evolving needs of customers—we know we are only in the beginning stages of what will be a long, impactful, and enduring alliance. Related Impact Story Flying across the sea, propelled by AI March 17, 2021 To help Emirates Team New Zealand successfully defend their America’s Cup title, we introduced them to a new sailor: a McKinsey-built AI bot. Blog McKinsey acquires Candid Partners, a leader in cloud consulting January 15, 2021 With the recent acquisition of Candid Partners, we welcome 100+ senior cloud experts—developers, engineers, agile coaches—into our firm. New at McKinsey Blog | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to About Us New at McKinsey Blog A collection of stories about our people, our capabilities, our research, and the ever-changing face of our firm. Browse All Posts Browse by: Search Jobs | Careers | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content My Dashboard Latest McKinsey tech outlook identifies 14 key trends for business leaders | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog Latest McKinsey tech outlook identifies 14 key trends for business leaders Innovation Analytics Sustainability October 4, 2022 The McKinsey Technology Council— a global group of over 100 scientists, entrepreneurs, researchers, and business leaders—has published its second annual Technology Trends Outlook . By assessing metrics of innovation, interest, investment, and adoption, the council has prioritized and synthesized 40 technologies into 14 leading trends. Following on from last year, applied AI once again earned the highest score for innovation in the report. Sustainability, meanwhile, emerged as a major catalyst for tech around the world, with clean energy and sustainable consumption drawing the highest investment from private-equity and venture-capital firms. And five new trends were added to this year’s edition:  industrializing machine learning, Web3, immersive-reality technologies, the future of mobility, and the future of space. In this post, McKinsey senior partner Lareina Yee , expert partner Roger Roberts , and McKinsey Global Institute partner Michael Chui share their thoughts about what the findings may mean for leaders over the next few years. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com From left, Michael Chui, MGI partner; Lareina Yee, McKinsey senior partner; Roger Roberts, McKinsey expert partner There are a number of tech trend reports published each year. What’s unique about this one? Lareina: It provides a bird’s eye view of the full spectrum of technology trends, from the bio revolution to advanced connectivity to space exploration. It also shows how all of these trends are threading together so leaders can understand the opportunities they create. For example, the metaverse is not a single trend but includes cloud computing, advanced connectivity, applied AI, and also immersive reality technologies and Web3, all of which allow for the creation of truly new experiences and modes of interaction. What this means is that creativity—how these technologies can be used together—may be the ‘constraining’ resource now, more than just computing or storage or network bandwidth. What surprised you most about the findings? Roger: One thing that I think is under appreciated is the potential in next-generation software development , the tools and capabilities which allow software developers to better write, develop, and test code faster and to therefore innovate in much quicker timeframes. That has huge power, so if I looked at which of these trends that people should be pushing faster and harder, that would be one. Also, over the next year or two, I think we’re going to see a lot more clarity emerging about the potential impact of quantum computing , which has been moving from science to engineering and will start to really scale in certain use cases. Michael: I am excited about the huge leap forward that industrializing machine learning (a form of AI) has taken, even over the past year, moving from pilots into production scale for real-world business use. Machine learning has an incredible variety of applications, creating value across multiple sectors. Some examples include predicting when customers are at risk of “churning” and dropping a service, identifying potential financial fraud more accurately, and assuring autonomous vehicles can navigate the streets of our cities safely. What should business leaders take away from this tech report? Roger: If a company can better understand those technologies which have the most momentum—by which we mean research, venture funding, use-case development—they can identify those that they need to act on in the next several years, and those for which they can take on a posture of watchful waiting. In practical terms this means thinking about, “What tech curves am I going to ride here and then what curves am I going to try to bend?” And by “riding,” I mean utilizing outside R&D and talent and letting that lift my business. On the “bending” side, leaders would say, “I’m putting my own engineering muscle behind thinking about how to shape something to our needs or build something unique on top of these platforms.” That’s where the combination of creativity, design, engineering, and data science need to come together at the front lines to build a new consumer experience or to launch that new business. Lareina: Companies also need to think about how these tech initiatives might shape the partnerships or alliances of their ecosystem. We know that companies with stronger networks of relationships will be better positioned to win in the world that’s being reinvented by these technologies. McKinsey Technology Trends Outlook 2022 View the interactive How will this research shape the work McKinsey does with its clients? Lareina: We’re sharing the findings and partnering with industries to think through how specific trends can best help companies achieve sustainable and inclusive growth. Some examples of this include:  identifying opportunities for consumer brands to develop beauty products with biodegradable ingredients; exploring the ways that advanced connectivity can extend healthcare and education services for more inclusive access; and looking at how immersive tech can deepen training and product support, so manufacturers can enhance customer and employee experiences together. Any predictions for next year? Michael: From year to year, the names of the trends aren’t going to shift so much. But the level of investment, the way in which they’re being applied, the rate of transition from science to engineering to practice will change. I’d love to see enterprises employing the engineering and also using their talents in creativity, design, and organizational change to combine and apply these technologies in new and unexpected ways. Roger: I would highlight that it’s not just what the tech trends are, but how fast they are moving that is really extraordinary.  We have seen this during the pandemic with the unprecedented pace of development and delivery of mRNA vaccines. This happened at a rate that many experienced people did not think would be possible. It shows that there are a range of players—governmental, environmental, technological—combining to accelerate technology change on many dimensions.  Leaders need to prepare to move at this pace. Related Blog Three experts offer an inside look at the state of AI January 13, 2022 McKinsey’s 4th annual survey on the state of AI reveals best practices that can deliver outsized results. Blog Microsoft’s Wright joins McKinsey as chief technology and platform officer September 21, 2022 Jacky Wright, who follows other recent senior hires, will bolster our firm’s technology capabilities and help our clients innovate and scale new ideas. Contact us | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Contact Us Thank you for your interest in McKinsey. Please fill out the form below to ask a question or report a technical problem. Please note: while we appreciate your questions, we are unable to respond to all inquiries. Required fields are marked with an asterisk (*). I have a question about: My member account Consulting services McKinsey Publishing Applying to McKinsey Other business inquiries Name Email Company or School Subject Message REF Security validation Captcha is Inavlid. McKinsey & Company is committed to protecting your information in accordance with its privacy policy . Learn more McKinsey & Company is committed to protecting your information. Your information will be used in accordance with the applicable data privacy law, our internal policies and our privacy policy . As McKinsey is a global organisation, your information may be stored and processed by McKinsey and its affiliates in countries outside your country of residence, but wherever your information is processed, we will handle it with the same care and respect for your privacy. Media inquiries Visit our media center to find contact details for our media relations team in your region. Reprints To request permission to republish an article, or if you have any questions about the usage of content published by McKinsey & Company visit our FAQ page. Inclusive Economy | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content We’re helping build an economy that works for all. 7 in 10 people live in societies with growing inequality. That’s why we’re taking action to create a more inclusive economy through our work with clients, and in local communities. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Insights on an inclusive economy Podcast US manufacturing: The next frontier for sustainable, inclusive growth April 21, 2022 – US manufacturing has an outsize impact on the nation’s economy. How can business leaders harness innovation to revitalize the manufacturing sector and drive inclusive economic growth? Report Race in the workplace: The frontline experience July 30, 2022 – Three of four frontline workers want to be promoted, but less than one in four achieves it. Companies can take targeted actions... to better support these workers. Article Rural rising: Economic development strategies for America’s heartland March 30, 2022 – There is no one-size-fits-all economic development strategy for rural communities. How can local leaders—including governments,... businesses, and individuals—put rural regions on track to thrive? Article A sustainable, inclusive, and growing future for the United States November 8, 2021 – While the challenges the United States faces are significant, it can build on core strengths to grow, broaden, and sustain prosperity. 1. Job creation statistic is based on an initial analysis of the change in employee headcount by more than 1,000 McKinsey clients globally that are public companies, as reported by those companies in corporate filings over the period from 2015-2019. This data does not include employee headcount data from clients that are privately-held companies, government entities, or non-profits as such data is not consistently available." Accelerating Sustainable and Inclusive Growth | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content What does it mean to accelerate sustainable and inclusive growth? McKinsey partners from around the world weigh in. In the media Wall Street Journal: McKinsey’s Top Executive Wants to Change How the Firm Operates Read Wall Street Journal article (registration required) Bloomberg: Asia’s Challenge and the World’s: A Future that’s Sustainable, Inclusive and Growing Read Bloomberg article CNN Business: The labor market is upside down. Here’s what CEOs can do about it Read CNN Business article Wired: In the age of AI, here’s how to unlock sustainable inclusive growth in the Middle East’s job market Read Wired article New York Business Journal: Three ideas to accelerate New York City’s path to sustainability Read New York Business Journal article Business Insider Mexico: Volatility on the way to a low-emission energy transition Read Business Insider article Featured Insights Collection Sustainable and inclusive growth: A weekly briefing September 15, 2022 – Updated weekly, this digest explores sustainable, inclusive growth and a new era in which growth and societal benefits can be complementary, reinforcing goals. Video Sustainable and inclusive growth: A new paradigm for Asia March 15, 2022 – Can growth be sustainable and inclusive? Leaders share their perspectives on how Asian businesses can make this a reality. Report The case for inclusive growth April 28, 2021 – US leaders across sectors have an opportunity to improve economic performance—and bolster individual and community well-being—through investments that embed equity into the development process. Podcast The path to sustainable and inclusive growth April 6, 2022 – Can collaboration between business, government, and society spur economic growth that benefits everyone without destroying the planet? 1. Numbers are based on our ongoing analysis of publicly held companies between 2015 –2021. We assess the percentage of global gross domestic product (GDP) growth driven by our clients, relative to total global GDP growth. In the same time period, we look at total Scope 1 and Scope 2 CO 2 emission reductions reported by
these publicly held companies, and the percentage of those achieved by our clients. We also establish the average net new jobs created by our publicly held clients during this period. Global M&A market slows in 2022 first half--but shows signs of strength | McKinsey Skip to main content Global M&A market slows in 2022 first half—but shows signs of strength September 30, 2022 | Article By Margaret Loeb LinkedIn Twitter Facebook Email Article (3 pages) Global M&A market slows in 2022 first half—but shows signs of strength LinkedIn Twitter Facebook Email Article (3 pages) Americas are most active while technology/media/telecom dominates other sectors. DOWNLOADS Article (3 pages) After soaring to an all-time peak in 2021, the global M&A market has hit the pause button. Early 2022 saw the value of large deals (more than $25 million) fall 24 percent from a year earlier, on a 12 percent drop in deal volume. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes But the 2022 numbers match healthy, prepandemic levels and are especially notable in a time of great uncertainty. Geopolitical instability, spiking inflation, supply chain issues, skittish capital markets, regulatory changes—all these factors, and more, are fueling uncertainty. And as Andy West , global coleader of McKinsey’s M&A Practice, says, “Uncertainty always weighs on decision making, and M&A is a big decision for deal makers. So naturally we’re seeing a bit of a slowdown.” According to the McKinsey M&A Practice review of the global M&A market, 2022 activity has declined, but only slightly. Deal makers in the Americas have been the most active traders, delivering almost half of worldwide deal value (48 percent, versus 52 percent for all of 2021). Europe, the Middle East, and Africa’s share is up slightly (28 percent, versus 26 percent), as is Asia–Pacific’s share (24 percent, versus 22 percent). The heavy hitters of 2021 remain the dominant deal makers of 2022. The technology, media, and telecommunications sector (TMT) has outperformed other industries—accounting for 30 percent of total deal value (lagging its 32 percent in 2021 but nearly matching its 2020 performance of 31 percent). The next two largest sectors in M&A trail far behind—real estate at 13 percent and industrials at 11 percent. Logic suggests that an uncertain market would encourage more small and fewer large deals. But 2022 has so far seen as many mega deals like the TMT acquisitions as 2021. Mega deals account for 28 percent of 2022 deal value versus 23 percent a year earlier. Private equity (PE) firms are likewise outpacing their record-setting 2021 performance. Their activity in the first half of 2022 delivered 26 percent of deal value (up from 25 percent for all of last year), with no slowdown in sight. As Oliver Engert , senior partner and a leader of McKinsey’s M&A Practice, notes, “PE invested over $2 trillion in 2021. Yet they still have trillions in dry powder left to spend, and that will be a really important contributor to the M&A market.” That firepower may be influencing the 2022 rise in premiums to 41 percent, up from 35 percent for all of 2021. While M&A activity has slowed from last year’s torrid pace, Oliver finds a positive note in this change in tempo. “With a slightly slower market, you can take your time to really understand the drivers of the asset that you’re considering. You can cultivate the management team and the employees that you are about to embrace and create something that’s far better than merely a ‘transaction.’” The slower pace also creates space to recharge, rethink, and renew. Many deal teams ended 2021 frazzled by the nonstop action. And many companies need to reflect on the impact of today’s uncertain market on their business, from their strategy to their supply chain, their balance sheet, their access to capital, and their portfolio, Andy says, adding, “In the context we’re in today, for most companies those things have changed. And the real risk is not recognizing it. So, my advice is to take a hard look at your strategy, understand what’s different today than it might have been a month, a quarter, or a year ago, and ask yourself, does that change either how I do M&A or where I’m going to focus my M&A strategy?” Andy and Oliver remain bullish on the programmatic strategies that have weathered more than a decade of M&A cycles. They express confidence that these strategies will continue to outperform in 2022. But they also highlight the importance of active portfolio management and vehicles like joint ventures and alliances to cope with the uncertain market. With these conditions, getting the fundamentals of deal execution right is more important than ever. “Value capture is really important in a market like this because we have to mitigate the headwinds that companies face: rising commodity prices, rising energy prices, increasing labor costs,” Oliver says. “As a result, we have to get far more specific, far more accurate, and probably go far deeper in value creation.” Capturing maximum value requires having the right organization design, operating model, talent, and culture. Value capture is really important in a market like this because we have to mitigate the headwinds that companies face: rising commodity prices, rising energy prices, increasing labor costs. Oliver Engert Treating the slower pace of M&A activity as an opportunity to revisit and refresh the fundamentals should place deal makers in the best position in 2022 and beyond. About the author(s) Margaret Loeb is a senior adviser in McKinsey’s New York office. The author would like to thank Roerich Bansal and Rohan Singh for their contributions of research and analysis for the Mergers & Acquisitions Practice’s 2022 first-half report, upon which this article is based. Explore a career with us Search Openings Related Articles Article Global M&A market defies gravity in 2021 second half Article The flip side of large M&A deals Article - McKinsey Quarterly How one approach to M&A is more likely to create value than all others Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. M&A Insights | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content M&A Insights Article The flip side of large M&A deals March 25, 2022 – Research shows there is a 50-50 chance that companies pursuing large M&A deals will outperform industry peers. Here are the four actions executives can take to increase those odds. Article Global M&A market slows in 2022 first half—but shows signs of strength September 30, 2022 – Americas are most active while technology/media/telecom dominates other sectors. Interview How an acquisition invigorated an asset management leader December 16, 2021 – Jenny Johnson, president and CEO of Franklin Templeton, explains how the firm’s acquisition of Legg Mason positions it for... the next phase of growth. Article Global M&A market defies gravity in 2021 second half March 16, 2022 – But what are the odds that M&A remains buoyant in the months ahead? Article
            
                    - McKinsey Quarterly How one approach to M&A is more likely to create value than all others October 13, 2021 – Two decades of research show that, while large deals still have their place, programmatic M&A strategies continue to create... gains in excess total returns to shareholders, at lower levels of risk. Want to learn more about how we help clients in M&A? Learn more M&A Strategy & Due Diligence Article The flip side of large M&A deals March 25, 2022 – Research shows there is a 50-50 chance that companies pursuing large M&A deals will outperform industry peers. Here are the... four actions executives can take to increase those odds. Article
            
                    - McKinsey Quarterly How one approach to M&A is more likely to create value than all others October 13, 2021 – Two decades of research show that, while large deals still have their place, programmatic M&A strategies continue to create... gains in excess total returns to shareholders, at lower levels of risk. Podcast Agile business portfolio management October 1, 2021 – Companies that regularly refresh their portfolios tend to outperform, but deciding when and how to divest a business may be the... most challenging part of M&A. Article The power of through-cycle M&A April 30, 2020 – How companies can use deal making to accelerate strategic shifts during the COVID-19 crisis. Integration Article Post-close excellence in large-deal M&A June 29, 2021 – The most successful large-deal transactions follow four key practices during integration execution. Article Eight basic beliefs about capturing value in a merger April 2, 2019 – To maximize deal value in a merger, focus on critical principles. Article Organizational culture in mergers: Addressing the unseen forces March 26, 2019 – Understanding culture, and proactively managing it, is critical to a successful integration. This requires a comprehensive approach. Article How the best acquirers excel at integration January 1, 2016 – The same handful of integration challenges vex companies year after year. New survey data suggest how high performers stay on... top. Separations & IPOs Article Divesting with agility November 11, 2020 – Research shows that active, efficient reallocation of resources creates better returns for companies than simply standing pat... does. Here’s how to make portfolio decisions faster. Article Through a different lens: A McKinsey perspective on separations May 28, 2020 – To succeed, companies need to view divestitures in a way that goes beyond the traditional M&A approaches. Podcast Active portfolio management: Interview with Andy West July 10, 2019 – Senior partner Andy West discusses the latest trends in M&A in light of several recent high-profile divestitures. Article Going, going, gone: A quicker way to divest assets August 6, 2018 – Speedy separations create more value than those that lumber along, our research finds. Preparation is the key. Joint Ventures & Alliances Article Checking the health of your business partnerships May 29, 2020 – Frequent, systematic assessments of joint ventures and alliances can reveal hidden problems and opportunities to create more value. Article Improving the management of complex business partnerships March 21, 2019 – Adhering to four key principles can help companies increase the odds that their collaborations will create more value over their... life cycles. Article Negotiating a better joint venture April 27, 2016 – As important as it is to secure the right terms for a shared enterprise, it is just as critical to form a sustainable relationship. Article Avoiding blind spots in your next joint venture January 1, 2014 – Even joint ventures developed using familiar best practices can fail without cross-process discipline in planning and implementation. Learning & Capability Building Survey Practice makes perfect: What sets programmatic acquirers apart November 12, 2019 – While there is no shortcut to outperformance in deal making, a new survey finds that expert acquirers take a different approach... to strategy, deal execution, and integration. Article Organizational culture in mergers: Addressing the unseen forces March 26, 2019 – Understanding culture, and proactively managing it, is critical to a successful integration. This requires a comprehensive approach. Article Equipping leaders for merger integration success July 9, 2018 – Leadership capability building is crucial during mergers. Tailoring a program for three cohorts of integration leaders can make... a big difference. Article
            
                    - McKinsey Quarterly The four building blocks of change April 11, 2016 – Four key actions influence employee mind-sets and behavior. Here’s why they matter. Connect with M&A Contact Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. ‘A sense of journey’: McKinsey’s new Mumbai office designed for age of hybrid work | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog ‘A sense of journey’: McKinsey’s new Mumbai office designed for age of hybrid work September 28, 2022 McKinsey’s presence in India has grown and changed with the country. We opened our first office there in 1992, in what was then Bombay, as the country’s economy opened up. The expansion of our work, in hand with growth in India’s economy, led to office openings in Delhi, Chennai, and Bangalore, India’s tech capital. But the latest change in our physical presence in India is one of innovation in addition to expansion, with a new Mumbai office opening in early July. When it came time to expand our footprint in Mumbai’s burgeoning financial and business district, Bandra Kurla Complex, our Mumbai office managing partner Kanika Kalra knew it had to be a space that reflected the drastic changes in work brought about by COVID-19. McKinsey Global Institute’s The future of work after COVID-19 report found that the bar for the appeal of an office had gone up dramatically since the pandemic. “We started designing this office during the pandemic, and we were sure work would not be what it was before,” Kanika says. “The office would have to be a special place that fosters a sense of belonging to a larger organization.” We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com The space is spread over six floors of an office tower, each with its own theme, brought to life through artwork—all made by Indian artists, such as a map of Mumbai made of computer keys, and natural design elements such as a fishpond. The materials were largely sourced within India, including Athangudi (native to the Chettinad area of Tamil Nadu) tiles in the second-floor bistro and Indian motifs on the carpets. “We wanted to encourage people to move around, to explore,” says Kanika, “to incorporate a sense of journey into an office day.” Ishita Kayastha , associate partner in Mumbai, was slow to return to the office as the pandemic eased, but the opening of this new office changed her outlook. “It’s just so bright and vibrant, with lots of sunlight,” she says. “The various themes, seating, colors; we get to explore different moods and sides of ourselves depending on what we want.” The underlying goal of creating an in-office journey worthy of journeying to is to build and strengthen connectivity among employees. “Work happens no matter where you are,” says Kanika, “but collaboration and the elevation of work doesn't. So, this is all about making that magic of collaboration.” As employees make their way through workspaces and floors, from the couch to the café to the roof, they might strike gold and catch a chance to pick someone’s brain who they’d normally not have reason to set up a meeting with. Ishita says she’s received impromptu help from firm experts a few times this way. “It’s different than just running into someone in a hall or an elevator. The space encourages you to talk,” Ishita says. “I can feel the increase in energy and feeling of connectedness between teams.” It really came together with this new office, which became a natural center of gravity to gather people together around shared senses of purpose and meaning. Shivanand Sinha, McKinsey partner Staff have also been proactive about building connectivity. Shivanand Sinha , a partner in Mumbai, helped set up the Mumbai chapter of Fab Clubs, a dozen activity groups for employees with common interests, such as hiking, films, biking, board games and music, and attributes their success to the office. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Shivanand Sinha and Kanika Kalra “We had the idea to do something like this to encourage people to come back after COVID, but it was slow to start,” says Shiv. “It really came together with this new office, which became a natural center of gravity to gather people together around shared senses of purpose and meaning.” McKinsey is currently not requiring employees to come into the office daily, giving people the freedom to create work conditions that work best for them. The challenge for many companies using remote and hybrid models is to blend this freedom with the cohesion attained by physical gatherings. Kanika believes rethinking the office is key and likens this new era to how workplace attire has evolved. “There used to be formal wear and casual wear and then this wonderful mix of the two that evolved from the tech revolution,” she says. “I think now there’s the office, there’s home, and there’s a little mix of the two that is happening, something warmer and more comfortable to be at.” Related Interview The rebirth of workspace design: An interview with Gensler co-CEO Diane Hoskins February 28, 2022 As offices consider how to reopen after two years of disruption from COVID-19, Hoskins sees an opportunity to create better, more human-centric workspaces. The Titanium Economy | McKinsey on Books | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content The authors Asutosh Padhi Asutosh is a senior partner and the managing partner for McKinsey in North America, leading the firm across the United States, Canada, and Mexico. He also serves as part of McKinsey’s 15-person global leadership team, and is a member of McKinsey’s Shareholders Council, the firm’s equivalent to a board of directors. Since joining the firm, Asutosh has helped high-performing, iconic industrial companies with functional, business-unit, and enterprise transformations, drawing on his strategic, organizational, and operations expertise. Gaurav Batra Gaurav Batra previously co-led McKinsey & Company’s Advanced Electronics Practice in the Americas. He advised clients on strategy and margin improvement issues, with a particular focus on commercial excellence, pricing, and distributor and channel management. Gaurav also previously led McKinsey’s Advanced Data Analytics (ADA) initiative in the Advanced Electronics Practice to help drive sustainable performance improvement for the firm’s clients. Prior to joining McKinsey, Gaurav worked as a senior business manager of corporate development with Capital One, a leading diversified financial services company in the United States. Nick Santhanam Nick Santhanam is the CEO & President of Fernweh Group. He was formerly a Senior Partner at McKinsey & Company’s Palo Alto Office, where he led North America’s industrial practice. He has co-authored several pieces of McKinsey’s leading industrials research: McKinsey on Industrials, McKinsey on Food Processing and Handling, and McKinsey on Packaging. He is also on the Advisory Board of Smithsonian Libraries. Praise for The Titanium Economy “A brilliant and incisive look at what could be the real key to America’s future economic growth—its industrial manufacturing base.” —David Rubenstein, co-founder and co-chairman, the Carlyle Group “In their fascinating new book, The Titanium Economy , Asutosh Padhi, Gaurav Batra, and Nick Santhanam show that manufacturing is in the midst of a renaissance in the US. The stage is set for a resurgence in America’s industrial prowess, but getting there will require some critical changes in both business and government. The Titanium Economy provides the necessary playbook.” —Alan Murray, CEO, Fortune Media “ The Titanium Economy formally recognizes a sector that has long punched above its weight—and one that should be at the vital core of the US economy for years to come. This book should be required reading for anyone passionate about the future of manufacturing. An excellent roadmap for how to win in the twenty-first century.” —Blake Moret, chairman and CEO, Rockwell Automation “Industrial technology is crucial to sustaining American innovation and competitiveness. The Titanium Economy provides a roadmap for how to breathe new life into the sector and assure its growth long term.” —Cindy Niekamp, board member and former senior vice president, PPG Industries “This book not only amplifies how important manufacturing is to national success, but it also inspires the reader to innovate for customer and stockholder value while advocating for a more adaptive, sustainable planet.” —David Burritt, president and CEO, US Steel “The Titanium Economy illustrates the tremendous possibilities advanced technology brings to our world. The sector presents a once-in-a-lifetime opportunity to redefine what it means to deliver value to our customers, communities, employees, and stakeholders. As this book proves, tech-enabled industrial businesses have an opportunity to lay a new cornerstone as we build America’s economic future. We’ve only begun to scratch the surface of our collective potential to make communities more livable, strengthen national security, and advance human discovery on Earth, and beyond.” —Kathy Warden, chair, CEO and president, Northrop Grumman “America, take note: it turns out the invisible, mid-cap industrial technology sector is an engine for jobs, innovation, growth, and high returns. The Titanium Economy makes a compelling case and provides a prescription for regaining US-manufacturing leadership. It is a call to action and a great read!” —Linda Fayne Levinson, chair, Clearpath Robotics, and former director, Jacobs, NCR, Hertz, Western Union, Ingram Micro “Innovation in manufacturing has been the backbone of the American economy. This is a timely, compelling, and practical exploration of the underappreciated and arguably misunderstood sector—one that is critical to driving inclusive economic growth for the future.” —Michael Lamach, former chairman and CEO, Trane Technologies “Technology is more than headline-grabbing Silicon Valley founders, viral smartphone apps, and sleek products. Technology is the driving force behind a renaissance in many sectors—including manufacturing. The Titanium Economy artfully explores the ways technology and manufacturing are intertwined, and the investments needed—especially in a diverse workforce—to help the sector actualize its potential.” —Reshma Saujani, founder, Girls Who Code, and founder and CEO, Marshall Plan for Moms “Manufacturing could become the next great driver of economic growth. The Titanium Economy offers a wonderful recounting of the opportunity at hand and a step-by-step guide for turning potential into reality.” —Susan Doniz, chief information officer and senior vice president, information technology and data analytics, The Boeing Company Author Talks: How industrial technology is creating a more inclusive economy McKinsey reveals 35 cutting-edge companies in the industrial tech sector, highlighting a robust “titanium economy” that creates American jobs and fuels innovation through sustainable, inclusive growth. Read the full interview Related articles Article Delivering the US manufacturing renaissance August 29, 2022 – The stage is set for a manufacturing resurgence in the United States. Can the country’s producers make it happen? Podcast US manufacturing: The next frontier for sustainable, inclusive growth April 21, 2022 – US manufacturing has an outsize impact on the nation’s economy. How can business leaders harness innovation to revitalize... the manufacturing sector and drive inclusive economic growth? Commentary Want to drive inclusive economic growth? Start with manufacturing January 20, 2022 – The decline of the US manufacturing industry has contributed to rising inequality and hurt the country’s global competitiveness—a... revitalization could contribute to sustainable and inclusive growth. Article The semiconductor decade: A trillion-dollar industry April 1, 2022 – The global semiconductor industry is poised for a decade of growth and is projected to become a trillion-dollar industry by 2030. Article Capturing the true value of Industry 4.0 April 13, 2022 – With digital transformations notoriously difficult to scale up across factory networks, manufacturers may need to slow down to... get ahead in the race to implement Industry 4.0. Article How industrial and aerospace and defense OEMs can win the obsolescence challenge April 14, 2022 – Complex equipment can last for decades, but internal components such as semiconductors have much shorter life cycles. Navigating... that disparity requires a systematic approach. Preorder now The Titanium Economy How Industrial Technology Can Create a Better, Faster, Stronger America By Asutosh Padhi , Gaurav Batra, and Nick Santhanam Amazon Apple Books Barnes & Noble Google Play eBooks.com Kobo Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Strategy & Corporate Finance Insights | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Strategy & Corporate Finance Insights How effective boards approach technology governance As technology’s strategic importance to the business expands, management needs stronger board guidance. Four engagement models have proven useful. Read more Survey The coronavirus effect on global economic sentiment September 30, 2022 – In the latest survey, inflation and geopolitical conflicts remain the top perceived economic risks, while concerns about energy... volatility predominate in Europe. Survey Economic conditions outlook, September 2022 September 29, 2022 – In stormy weather, survey respondents maintain realism about the global economy. While geopolitical conflicts and inflation remain... top of mind, concerns about energy volatility predominate in Europe. Report
            
                    - McKinsey Global Institute Securing Europe’s competitiveness: Addressing its technology gap September 22, 2022 – Even amid war in Europe and the energy and cost of living crisis that has resulted, the region needs to pay attention to its slow-motion... corporate and technology crisis. Article Global Economics Intelligence executive summary, August 2022 September 9, 2022 – Growth moderates globally in an inflationary environment; the US Federal Reserve emphasizes its tightening course; Europe faces... near-term energy volatility. Article
            
                    - McKinsey Quarterly Strategic courage in an age of volatility August 29, 2022 – Today’s challenging environment requires business leaders to hone their edge in three critical areas: insights, commitment, and... execution. Article How can CFOs rebrand themselves as innovation allies? July 19, 2022 – They can take five actions to improve objective-setting, performance measurement, and cultural factors associated with successful... innovation projects. Survey Five ways to increase your board’s long-term impact July 15, 2022 – Directors are spending more time on their work, yet few say their boards are better at creating long-term value. Survey data highlight... the operating models that correlate most with value creation. Article Global Economics Intelligence executive summary, June 2022 July 6, 2022 – High energy prices stoke inflation; central banks chart tightening course; supply challenges persist and industry struggles to... meet demand. Article How capital expenditure management can drive performance June 29, 2022 – Want to reduce project costs and timelines while increasing returns? Undertake a top-to-bottom reassessment of your capital investments... at every stage of the life cycle. Survey Three new mandates for capturing a digital transformation’s full value June 15, 2022 – Most organizations achieve less than one-third of the impact they expected from recent digital investments. What can companies... learn from the best performers about how to beat the odds today? Article Fear factor: Overcoming human barriers to innovation June 3, 2022 – Worries about failure, criticism, and career impact hold back many people from embracing innovation. Here’s how to create... a culture that accounts for the human side of innovation. Article Global Economics Intelligence executive summary, May 2022 June 7, 2022 – Central banks move against inflation; US industry expands while China’s economy contracts amid COVID-19 measures; supply... challenges persist. FEATURED PUBLICATIONS Collection Beat the odds with a bold business strategy Learn what it takes to unlock the big moves that really matter for exceptional performance. Book - Practices Capabilities Valuation: Measuring and Managing the Value of Companies, 7th edition At the crossroads of corporate strategy and finance lies valuation. This book enables everyone, from the budding professional to the seasoned manager, to excel at measuring and maximizing shareholder and company value. McKinsey & Company Books CEO Excellence From McKinsey & Company, an insight-packed, revelatory look at how the best CEOs do their jobs based on extensive interviews with today’s most successful corporate leaders—including chiefs at Netflix, JPMorgan Chase, General Motors, and Sony. #CEOExcellenceBook PODCASTS Expert voices Interviews and conversations with our partners on how senior management thinks about strategy and how they confront their most pressing challenges. Latest topics: Podcast Four front-foot strategies to help create value in the net-zero transition September 2, 2022 – Companies can identify green growth opportunities and move boldly to take advantage of them. Podcast Does your idea have the ‘voltage’ to scale? August 31, 2022 – Economist John List discusses the science of determining whether a product or service can become a blockbuster. Podcast Charles Lowrey on putting purpose at the heart of Prudential Financial’s transformation August 19, 2022 – The chairman and CEO of Prudential Financial talks about what it takes to keep a transformation on track. Podcast Voices of CEO excellence: Morgan Stanley’s James Gorman July 22, 2022 – The chair and CEO of the financial services giant explains how he reshaped the firm in the wake of the financial crisis. Podcast MassMutual’s Roger Crandall on disrupting your own industry July 11, 2022 – The long-time CEO of the US insurer reflects on the changes in the industry, the risk environment, and the CEO role. Podcast The path to sustainable and inclusive growth April 6, 2022 – Can collaboration between business, government, and society spur economic growth that benefits everyone without destroying the... planet? Podcast The role of the board in preparing for extraordinary risk March 22, 2022 – Risks that threaten a company’s existence require unique interventions from the board. Podcast Programmatic M&A: Winning in the new normal March 21, 2022 – How to successfully source and execute a string of deals that lead to the creation of a new business. Subscribe and listen to the series The Strategy & Corporate Finance blog Fresh takes on strategic growth and the latest trends in business. Read the latest Special collections Collection McKinsey on Finance 20th anniversary Reflections on sustainable, inclusive growth. Collection Cash Excellence This collection of articles and videos explores how to enhance an organization’s performance and resilience through the... application of cash management best practices, mindsets, and capabilities. Collection CEO perspectives How do the best CEOs think and act? What wisdom can they share? Explore a special collection of CEO perspectives that touches... on everything from corporate strategy to personal working norms. Global theme Business Resilience Resilience is the ability of a business to withstand, adapt, and thrive in the face of shocks that are internal and external,... as well as known and unanticipated. Want to learn more about how we help clients in Strategy & Corporate Finance? Learn more More insights Blog Post Prime Numbers In this blog series, we provide a refresher on how to interpret the metrics underlying companies’ performance, resource-allocation... decisions, and growth strategies. Survey The coronavirus effect on global economic sentiment September 30, 2022 – In the latest survey, inflation and geopolitical conflicts remain the top perceived economic risks, while concerns about energy... volatility predominate in Europe. Report
            
                    - McKinsey Global Institute The rise and rise of the global balance sheet: How productively are we using our wealth? November 15, 2021 – Net worth has tripled since 2000, but the increase mainly reflects valuation gains in real assets, especially real estate, rather... than investment in productive assets that drive our economies. Podcast Boards, talent, and culture August 30, 2021 – Boards need to ensure that management walks the talk on culture and values. Report
            
                    - McKinsey Global Institute Will productivity and growth return after the COVID-19 crisis? March 30, 2021 – Bold innovation by some firms under pressure of the pandemic could deliver a productivity dividend, but that depends on corporate... action broadening and robust demand. Article The great acceleration July 14, 2020 – The COVID-19 crisis has intensified existing trends, widening the  gap between those at the top and bottom of the power curve... of  economic profit. Will your strategy keep you ahead of the accelerated pace of change? Article Coronavirus: 15 emerging themes for boards and executive teams June 2, 2020 – Board directors and executives can pool their wisdom to help companies grapple with the challenge of a lifetime. Survey Survey results: Expectations for company performance, by industry September 30, 2022 – Across industries, only respondents working in energy and materials and financial services report more upbeat expectations for... their companies’ profits or demand for their goods or services than in the previous survey. Article Looking back: What does the ‘long term’ really mean? September 29, 2022 – Stock markets can be volatile, and some years they decline. But the ups far outnumber the downs—and returns are in line with two... centuries of performance. Interview
            
                    - McKinsey Quarterly ‘If you’re going to build something from scratch, this might be as good a time as in a decade’ September 26, 2022 – In the second episode of our series The Quarterly Interview: Provocations to Ponder , venture capitalist Bill Gurley explains... the promise and perils facing start-ups at a moment of economic uncertainty and reveals why hybrid work may be the most interesting technology of all. Podcast To slow down attrition, pay closer attention to what workers really need September 22, 2022 – The Great Attrition continues, but if companies understand the differences among five common employee personas, they may be able... to find and retain talent more effectively. Sorry, we couldn't find any results. Try removing some filters. View more Connect with our Strategy & Corporate Finance Practice Contact LinkedIn @McKStrategy @McKonFinance Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Author Talks: Industrial technology and the inclusive economy | McKinsey Skip to main content Author Talks: How industrial technology is creating a more inclusive economy September 8, 2022 | Interview By Asutosh Padhi , Gaurav Batra, and Nick Santhanam LinkedIn Twitter Facebook Email Author Talks: How industrial technology is creating a more inclusive economy LinkedIn Twitter Facebook Email McKinsey reveals 35 cutting-edge companies in the industrial tech sector, highlighting a robust “titanium economy” that creates American jobs and fuels innovation through sustainable, inclusive growth. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes In this edition of Author Talks , McKinsey Global Publishing’s Lucia Rahilly chats with McKinsey senior partner Asutosh Padhi and coauthors Gaurav Batra and Nick Santhanam about their forthcoming book, The Titanium Economy: How Industrial Technology Can Create a Better, Faster, Stronger America (Public Affairs, October 2022). You’ve probably never heard of the companies they cover — under-the-radar makers of a range of industrial products. But these companies make up a burgeoning sector that Padhi, Batra, and Santhanam say is just beginning its best years, and that has the potential to reinvigorate American manufacturing—sustainably and inclusively—in the process. An edited version of the conversation follows. What problem were you hoping to solve with this book? Video Asutosh Padhi: I believe titanium economy companies are not well understood and not well appreciated. I was hoping to shine a spotlight on what I think is truly an amazing set of companies that drive a lot of innovation and therefore job creation. Ultimately, these companies are going to be critical in creating a more inclusive economy in the United States . Nick Santhanam: The problem we were trying to solve is lack of awareness. If you look at the industrial sector, it is truly a diamond in the rough. I would even go further and say it’s truly a diamond—maybe not even in the rough. When you talk to folks, you realize that there is a huge difference between what people believe about the industrial sector and what is reality. We wanted to fix that perception gap. Gaurav Batra: When I was growing up, and even in the early years of my career, the industrial sector was not always viewed as a rosy picture, but my personal experience over the past two decades with the industrial sector is nowhere close to that perception. In reality, there is immense value being added to companies, to employees, and to communities—it’s really the heartbeat of an economy. My personal hope from this book is that we are able to bridge the gap between perception and reality and have folks see the value of the sector. What surprised you in the research or writing? Video Asutosh Padhi: Eighty percent of the titanium economy companies are private; therefore they tend to have a longer-term orientation. But the thing that surprised us was the high degrees of employee retention in these companies in the middle of the Great Resignation . As we looked underneath, we could understand why: the compensation levels are different. They’re much higher in these companies. The amount of care and concern for the employees is different. People actually get access to different development opportunities, and I think there’s a culture and a purpose that comes from being part of that setup. It’s a combination of factors that brings it all together. The importance of the talent proposition and how well the titanium economy companies intuitively understand that—not just today but for the past 50 to 100 years—was one of the things that really struck us. The importance of the talent proposition and how well the titanium economy companies intuitively understand that—not just today but for the past 50 to 100 years—was one of the things that really struck us. Asutosh Padhi We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Asutosh Padhi Nick Santhanam: Two things surprised me the most. I spent almost north of 30 years in the industrial sector, and I knew it was a great sector, but I did not realize how many companies out there were doing so well in the stock market. When Gaurav showed me the list, I fell out of my chair. I said, “This can’t be true. You can’t have this many companies who are doing better than S&P 500, who are doing better than Nasdaq-100. Wow.” The second thing, which in a lot of ways provoked us to write the book, is that this is not an industry that is dead. We knew it was a good sector, but we went in thinking “this was a sector that was stolen. This was a sector that was given away. This was a sector that was taken away from us.” We realized all of that was false—this sector was not stolen, it was not taken away, and it was not given away. It’s a great, vibrant sector that creates a lot of jobs and a lot of livelihoods. As we talk about it in the book, the best of days are yet to come. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Gaurav Batra I thought we were going to be telling a story of the past or, at least, saying, “Hey, these are our best days.” As we finished the book, we walked away saying, “Boy, we are in the first innings.” Gaurav Batra: We discussed the book with about 50 to 70 CEOs in the ecosystem, and they all agreed with the messages. The surprising thing for me was, if you agreed with them, then why didn’t you act on them? We also talked about how so many of the titanium economy companies are private, so they’re not really worried about quarter-over-quarter or how investors are looking at them. I would say that was the biggest surprise for me, how the alignment with the key messages of the book—the potential of titanium economy—was pretty widespread. What in your own background inspired you to undertake this research? We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Nick Santhanam Asutosh Padhi: I’m a mechanical engineer by background, but I never practiced mechanical engineering. I went to business school and then joined McKinsey directly out of campus, but deep inside me there’s always been a mechanical engineer. The thing that’s always fascinated me is the importance of innovation and how much of it resides in places that we would not associate with innovation. I saw that in the course of my work at McKinsey. I saw a set of companies that everyone talks about, that is known for innovation, and that is great at innovation: I saw software, I saw technology. The thing that surprised me was that there is an equally great set of companies that are great at innovation, that are manufacturing oriented , yet no one associated innovation with this set of companies. Gaurav Batra: A couple of personal experiences were transformational in terms of my own thinking about the sector. I’m a mechanical engineer by training, and one of my first jobs was as a manufacturing manager for a detergent powder factory back in India. The India factory was in a tribal area—the government used to give tax incentives—so my workforce was made up of about 200 people who probably couldn’t even write. They were coming to the factory daily, producing tons of detergent powder, which was getting distributed all through the country. Video That was the first time I really saw the human value of what a healthy manufacturing organization can do: 200 people were relying on it for their daily needs. It was a vehicle for social mobility for them. They came from really underprivileged backgrounds and being able to earn a healthy salary gave them a chance to take their families to the next level, put their kids through education, give them proper training, and have better lives themselves. That was one of the first personal experiences that transformed my thinking about the industrial sector and what it could do for not just for the companies but also for a broad swath of people who were involved with that ecosystem. Nick Santhanam: I’m a chemical engineer. I came to this country in 1991, and my very first job was for a company called Johnson Matthey, where I did a lot of work on the manufacturing line. Then, I worked for a company called Arlon, which was a printed circuit board company, and then a company called Taconic, which was also in that space. This was the time of Y2K, when everybody was a computer programmer. I believed that I added more value because products were needed. These were real products being shipped, affecting people’s real livelihoods, and yet people did not know what Johnson Matthey did, or Arlon, or Taconic. That, in a lot of ways, provoked me to say, “This is a great story. It’s a great truth, and I think it’s worthwhile for people to know. Then people can make their own decisions, but let’s get the facts out there.” Tell us about the companies you profile in The Titanium Economy . Video Nick Santhanam: We talked to about 35 companies, which we have profiled in the book, as well as an additional 50 companies. I believe every one of them is an unsung hero on their own merits, on the value they have created—for the ecosystem and for their customers. As much as I wish I could say it’s company A versus company B, all of the 35 companies we profiled in the book and the 50-odd companies we talked to are unbelievably great companies. Asutosh Padhi: You walk into one of these companies, and the places look clean; the machines are new; the people are working together in teams; the place is inviting. You get the sense that there is something happening that is much more modern and oriented around precision manufacturing, whether it’s using new materials, new technologies, or different types of equipment . You also get a sense of purpose: Why are these people so excited to be here? It is because they believe that they’re driving innovation, that they’re doing things that are important to our future. What will it take to get the titanium economy to where it needs to be to enable a more inclusive economy? Video Asutosh Padhi: My dream is that the titanium economy in the United States not only continues to flourish but continues to accelerate. Making that happen will require individual companies to step up further. In my conversation with CEOs we’re always asking the question, “How high is high? How much more innovation could you do? Which other customers could you be serving? What does that imply about new products you can launch?” The other part of it is the enabling factor: it’s the role that the government and policy can take to enable investing in infrastructure, investing in people, and finding a way to drive a closer connection between corporate America—especially the titanium economy companies—and the places where talent is available, like vocational colleges and community colleges. Anything that we can enable will help grow the titanium economy. My hope is that the titanium economy continues to scale, and that as it scales, it will continue to drive job creation, innovation, and exports. It’ll drive an amplification effect, where it will create jobs in adjacent industries, and it will help local communities thrive. It’s also my hope that the titanium economy becomes more diverse. My hope is that the titanium economy continues to scale, and that as it scales, it will continue to drive job creation, innovation, and exports. It’ll drive an amplification effect, where it will create jobs in adjacent industries, and it will help local communities thrive. Asutosh Padhi More importantly, I hope that the debate shifts. Today the debate around manufacturing in the US is one that is very pessimistic. It always has a tinge of “this is yesterday.” I’m hoping the debate now shifts to optimism and possibilities, and the fact that this is our future, not yesterday. Gaurav Batra: I’ve served the sector in my capacity as a partner at McKinsey over the past 12 years. In every instance, you could draw a direct correlation between how well a company does and what kinds of opportunities it starts affording its own employees. I have personal examples where I have supported my clients as they turn into better operational companies, and folks from the front line have done well; they’ve changed the way they work; they’ve risen in the company and become managers, directors, and VPs. I think that’s a great example of the potential that the titanium economy company might offer, not just for the executives but for the employees who are working there, in terms of their own social mobility and social stature. What do you think the hardest part of achieving that aspiration will be? Video Asutosh Padhi: The toughest part of realizing that aspiration will be addressing the lack of awareness around who these titanium economy companies are. These companies are all around us every day, yet we don’t actually know them. They don’t make the covers of the most important magazines we read, or the newspapers we read, or the news channels we watch. As a result, over a period of time, they could easily have a talent deficit—people may not want to work there. Part of the reason we wrote this book is to highlight the importance, the purpose, and the role that these companies play in our country and therefore find a way to ensure that there’s a flow of talent, a flow of innovation, and a continued focus on enabling these companies to be successful. Gaurav Batra: I think it’ll be a true team effort. The key members of the team would be the executives in the industrial ecosystem today who are running small, mid-cap, and large-cap companies—from the frontline managers up to the CEOs. They have to realize and understand the urgency of achieving their company’s true potential. They have to understand what their true potential is and create a path to it. Second, investors need to start thinking about industrial as a good place to start investing, not just from an altruistic perspective but also from a financial perspective. If you look at examples in the book—HEICO is a great one, Trex is a great one—they’re real diamonds in the rough that perform better than any index you can come up with over decades. Finally, the third members of the team would be public-policy experts, particularly focused on the talent gap coming up between supply and demand. That’s where we need some public-policy-level decisions, which will help close that gap in the coming few years. These three types of folks have to put in a team effort over the next few years to make this happen. Who did you write this book for? Video Asutosh Padhi: We have written the book from the standpoint of a student who is graduating today, and who’s considering different choices. We’re saying, “Where do you want to spend the rest of your career? Do you want to go and work in the services industry, which you absolutely can do? Or do you instead choose to do something different? You may come back after reading the book and say, ‘I actually want to work in manufacturing.’” The old image of manufacturing—which was around factories that were greasy and dirty— doesn’t exist anymore . Will this start to attract a more diverse set of people to come work with these companies? Nick Santhanam: My daughter is 24. Folks who have a 24-year-old daughter realize that dads are usually not the person who does anything right. Once I walked into her room and she was hiding something, and I said, “What are you hiding?” She said, “Nothing.” Then I realized that she was reading the book, and she was halfway through. She sort of sheepishly said, “You know, this book is not that bad.” For people who know the father–daughter lingo, that means it’s actually pretty good. This is when it resonated with me that this is a book for pretty much any audience, because it brings what the industrial sector is to life. It’s not about pontificating; it is not about victimizing or blaming somebody. It’s about what the sector is, what the sector has done, and what the sector could be—based on facts, in the classical McKinsey fashion. We’ve used data to lay out the story, and people can make their own decisions. We are not telling people, “This is the answer.” We don’t want to tell the answer. We have laid out the facts. Whether you’re an investor, whether you’re a 24-year-old employee joining the workforce, or you’re the CEO of an industrial company, we believe this is a good book for them, for all of them. Why haven’t industrial companies invested more in storytelling about their successes? Video Asutosh Padhi: The titanium economy companies are under the radar because they are, in some sense, everywhere, so you can’t find them at a single place. They are private, so you don’t find them listed on the stock market. They don’t have business-to-consumer brands, so they don’t have the need to advertise. Because they tend to be smaller from the standpoint of revenues, they don’t make headlines in the news media. Titanium economy companies have been going about doing what they’re actually good at, which is delivering great returns for their shareholders, taking care of customers, taking care of employees, doing the job around communities, and cleaning the environment—but they have never really found an opportunity to pull it all together and tell the story. Gaurav Batra: Industrial companies haven’t invested more in storytelling for the same reason we are excited about this sector: because it’s not on a technology treadmill. You don’t see an industrial company coming up and then disappearing in three or six years. On the flip side, this also brings a sense of complacency. You may think, “I’m making 10 percent margin, 15 percent margin, that’s not bad. It’s not great, but it’s not bad either.” There’s limited reason to complain. The urgency to change, the need to change, the burning platform don’t necessarily exist. This is where I think we get to the crux of the problem: the narrative, the mindset about the sector overall, has to change. It should be less about “are we comfortable?,” because we are comfortable; I think it’s a very healthy sector today in the US, but it can be a lot healthier. So, we find the true potential of the sector for the country and frame all our discussions through that lens. I think it’s a very healthy sector today in the US, but it can be a lot healthier. So, we find the true potential of the sector for the country and frame all our discussions through that lens. Gaurav Batra Nick Santhanam: The industrial sector works hard for their money. Every time they’ve gotten an improvement, it’s because they have driven a true performance improvement, versus just getting valued more, which is, as we call it, the multiples. The analogy is, if you’re buying a house, you buy it for a dollar per square foot. Three years later, you’d pay more for the same house because you’d pay more dollars per square feet. Industrial is a sector where you continue to pay the same, so you only get more if you build more houses or make more earnings. I think that’s because the sector has not been good at storytelling. It hasn’t gotten its message around about how great it is, and I think fixing that is going to solve a trillion-dollar question, which is creating their trillions of dollars in valuation. What has been the experience of industrial companies during this period of profound disruption, including COVID and the energy transition? Video Asutosh Padhi: One of the things that struck me is that we’ve been having a lot of discussion about the energy transition , and I think there’s widespread concern about the fact that companies are doing a lot of greenwashing. I was skeptical too, but then I started to look at what these companies are doing. They’re inventing technologies that will actually help with carbon capture and sequestration. Take a company like Trex: they’re recycling plastics to make outdoor wooden decks that are 97 percent recycled. There are waste management companies we talk about in the book that are using advanced analytics. Normally when you throw out the garbage in your house, if you’re like me, you are not particularly good at figuring out what is garbage and what should be recycled. Companies are now using artificial intelligence to sort through trash and figure out how to optimize waste management. They’re not talking much about it, but they’re making it happen. Nick Santhanam: The industrial sector is extremely resilient. It’s probably one of the very few sectors that follows “live and let live.” Unlike other sectors—retail or tech, for example—where companies have a meteoric rise and a meteoric fall, you don’t have that here, which is a greatness of the industrial sector: it’s very resilient. Industrial companies make great products. They make great margin. They’ve been wanted. Yes, they’ve gone through rough spots, but even through the rough spots, they end up coming out all right. They’re very resilient. Unlike other sectors—retail or tech, for example—where companies have a meteoric rise and a meteoric fall, you don’t have that here, which is a greatness of the industrial sector: it’s very resilient. Nick Santhanam What are the top lessons that you would like readers to take away from reading this book? Video Asutosh Padhi: The most important lessons are around the importance of recognizing innovation and technology. We have a very strong foundation in the United States to build from; we have a set of about 5,000 companies that have been driving the titanium economy’s growth, not just for the past five to ten years, but for the past 100 years. I think we are still early on that journey to the full potential of the titanium economy. Over time it could perhaps be two or three times the scale of what it is today. It could help us create a lot more jobs, and it could have the amplification effect we talked about. When it scales, it will help us create a much cleaner planet. It will help us create a most inclusive economy, and my personal hope is that it also becomes more diverse. Video How industrial technology is creating a more inclusive economy Visit Author Talks to see the full series. Learn more About the author(s) Asutosh Padhi is a senior partner and the North America managing partner, based in the Chicago office. Gaurav Batra is an alumnus of McKinsey’s Washington, DC, office. Nick Santhanam is an alumnus of the Bay Area office. Lucia Rahilly is deputy publisher and global editorial director of McKinsey Global Publishing, based in the New York office. Comments and opinions expressed by interviewees are their own and do not represent or reflect the opinions, policies, or positions of McKinsey & Company or have its endorsement. Explore a career with us Search Openings Related Articles Article The Titanium Economy Article Delivering the US manufacturing renaissance Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Middle East | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Welcome to McKinsey in the Middle East We partner with leaders to accelerate growth, build businesses and organizations, and nurture young talent. Since 1957, we have helped the region unlock growth and development, build champions in major industries, and nurture the talent of the next generation. The work of all eight of our locations— Abu Dhabi , Cairo , Doha , Dubai , Karachi, Kuwait, Manama , and Riyadh —is underpinned by our deep local roots, global reach, and unrivaled investment in knowledge. 50+ languages spoken in the Middle East offices 600+ consultants across our Middle East locations 3000+ projects completed across the region since 2000 Learn more about McKinsey Featured video Qiyada Emerging Leaders Qiyada Emerging Leaders is an initiative that aims to empower promising young professionals in Abu Dhabi to achieve their full leadership potential. Find out more about Qiyada More videos Serving Society Social responsibility in the Middle East Our mission is grounded in supporting the new generation of young leaders, entrepreneurs, engineers, academics, and artists in the Middle East. Giving back in 2020 Our purpose is to help create positive, enduring change in the world. We are committed to doing all we can to help youth in this region get opportunities they deserve to pursue their dreams. Learn more about how we are developing future leaders Download the report (PDF) One million saved meals In 2020, we started a collaboration with the Mohammed Bin Rashid Al Maktoum Global Initiatives to tackle hunger and food waste, supporting the biggest food donation drive in the region. Learn more about the one million saved meals campaign Featured insights Article Growth opportunities for digital health in KSA and UAE June 16, 2022 – Digital health can improve patient services and well-being in the Kingdom of Saudi Arabia and the United Arab Emirates. The sector offers growth opportunities for existing players and new market entrants. Article Reimagining education in MENAP Article Ten ways to accelerate the benefits of digital health in Saudi Arabia Podcast What’s fueling Pakistan’s emerging start-up ecosystem Interview Partnering for health equity: ALJ Health’s Akram Bouchenaki Mckinsey Bel Arabi Compendium McKinsey Bel Arabi is a compendium of articles curated and translated into Arabic for your convenience. We compile articles under a specific theme that relates to major issues and subjects affecting governments and businesses in the region. Winter 2022 McKinsey Bel Arabi Issue 6 Download Issue 6 View all issues More insights Meet our people Gassan Al-Kibsi Senior Partner and Managing Partner EEMA, 
                                        Middle East Supports public, private, and social-sector leaders across the region to improve opportunities for the youth and create global... Hasan Muzaffar Senior Partner and Managing Partner Middle East, 
                                        Dubai Helps leading conglomerates and sovereign and principal investors shape corporate strategy, optimise investments, and strengthen... Rima Assi Senior Partner and Managing Partner, 
                                        Abu Dhabi Leads the Abu Dhabi office and advises leaders, policy makers, and governments on the transformation of fiscal-management and... Dr. Mazen Ramsay Najjar Senior Partner and Managing Partner, 
                                        Kuwait Leads our Kuwait office as well as the Risk & Resilience Practice in Eastern Europe, the Middle East, and Africa, is a member... Jawad Khan Senior Partner and Managing Partner, 
                                        Karachi Leverages deep experience to help financial groups achieve sustainable tech-enabled growth Jalil Bensouda Senior Partner & Managing Partner, 
                                        Cairo Supports public and private clients on strategy, large scale transformations, and digital programs Tarek El Sayed Senior Partner and Managing Partner, 
                                        Riyadh Supports CEOs, senior executives, and government leaders on a range of strategic and organizational topics. Samvit Kanoria Senior Partner and Managing Partner Qatar, 
 Saudi Arabia Our Client Capabilities Hub locations Atlanta Brussels Chennai Dallas Dubai Gurugram Johannesburg Lisbon Riyadh San José Shanghai Waltham Wroclaw Connect with the McKinsey Client Capabilities Network Contact Interviewing | Careers | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content My Dashboard Interviewing We want you to succeed At McKinsey, we help our clients tackle the world’s most important problems, while providing our people with unrivaled professional development experiences. Joining us means you’ll have unparalleled opportunities to learn and grow. And you won’t tackle any of these challenges alone. Hear from firm members Get ready for your interviews Interviewing is a two-way process—it helps us learn about you as a potential colleague, and helps you learn about McKinsey and what you could do here. Overall, we look for personal impact, entrepreneurial drive, inclusive leadership, and problem solving, and we recognize there are many ways to acquire those skills. Our values, practice cases and other resources to prepare Who we are and what we look for Our values define who we are as a firm and who we aspire to be as individual colleagues. They are the guiding principles that shape our actions and are essential to fulfilling our two-part mission to: help our clients make distinctive, lasting and substantial improvements in their performance and to build a great firm that attracts, develops, excites and retains exceptional people. While there’s no template for success at McKinsey, our people share some qualities that help make us successful–and that make working here rewarding. During your interview, we'll ask you to share examples from your personal experiences. Some interviewers will use the exact language below, while some will use their own words. Your responses to these questions allow us to understand your accomplishments and challenges beyond your CV and discover skills that will help you thrive at McKinsey. We suggest that you prepare two personal examples to demonstrate your skills in these areas. During your interview, you should speak in a conversational style about your examples and share details about the challenges, goals, and actions you took to create change. If you are interested in McKinsey careers beyond consulting, you can learn more about the assessment process for our internal roles here . We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Find your ideal role McKinsey has a many different paths, so finding a suitable role is helpful before submitting an application. Not sure where to start? Finding a role for you Take this quick 10-question quiz to discover a career path at McKinsey that best suits you Students We look for exceptional people with many academic backgrounds from colleges and universities around the world Experienced professionals We look for skilled professionals from a variety of fields and professions, including technology, finance, government, engineering, law, the military, medicine, and many more. Internal roles Thousands of talented colleagues around the world bring their skills, experience, and perspectives to McKinsey’s internal roles to make meaningful impact and keep the firm operating at its best. Get to know us Careers Blog At the end of the day, we all want to know who’ll we work with and what makes someone successful at McKinsey. Meet some of our people here. McKinsey Recruiting Podcast The McKinsey Recruiting Podcast aims to inform listeners about who we are and what we do. Tune in to get an insider's view of McKinsey, hear us answer common questions, and get to know some of our colleagues. Important fraud alert McKinsey has become aware of scams involving false offers of McKinsey employment. The scams and false offers use imposter websites, email addresses, and text messages. None of these offers are legitimate, and McKinsey’s recruiting process never involves interviewing via instant message, nor requires candidates to purchase products or services, or process payments on our behalf. Find your ideal job Already started your application? Sign in here. Find your ideal job 500+ Roles Available Jobs Already started your application? Sign in here. Hungary | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Welcome to McKinsey Hungary A global hub for knowledge and talent, McKinsey’s Budapest office works with Hungary’s leading companies to strengthen performance and unlock growth. With deep roots in Hungary, we work closely with the country’s leading companies to shape growth strategies, strengthen operations and organizations, leverage technology, and develop leadership. We serve clients across all major sectors—including financial services, energy, telecommunications, and pharmaceuticals. Our Budapest office is a global center for leading-edge research and specialist knowledge. Through our diverse pro bono work, we contribute to our local society in areas ranging from healthcare to the social sector. 20+ years of McKinsey’s presence in Hungary 160+ colleagues working in our Budapest office 30+ countries where we worked last year Our People Levente Jánoskuti Managing Partner, 
                                        Budapest Leads McKinsey's Budapest office; the Global Pricing Lab for corporate banking and McKinsey Digital initiatives in Central Europe.... Dániel Róna Partner, 
                                        Budapest Leads our agile work in Central Europe and our Enterprise Agility Center in Budapest and helps institutions across industries... Balázs Czímer Partner, 
                                        Budapest Brings a decade of experience in serving major financial institutions across Europe on strategy, large-scale transformation programs,... Csaba Paál Partner, 
                                        Budapest Works with telecommunications companies across Europe to shape bold growth strategies, strengthen their commercial positions,... Gergely Bacsó Partner, 
                                        Budapest Brings deep expertise in digital retail banking and insurance, specializing in digital transformations and setting up new digital... András Havas Partner, 
                                        Budapest Supports leading financial institutions in shaping ambitious growth strategies and driving digital transformations globally Ákos Légrádi Partner, 
                                        Budapest Supports banking and telecommunications clients across Europe and the Middle East; helps modernize organizations through large-scale... András Kadocsa Partner, 
                                        Budapest Serves automotive and industrial companies, helping them innovate their strategies, achieve transformational change in manufacturing... András Sebők Partner, 
 Poland Our Client Capabilities Hub locations Atlanta Brussels Chennai Dallas Dubai Gurugram Johannesburg Lisbon Riyadh San José Shanghai Waltham Wroclaw Connect with the McKinsey Client Capabilities Network Contact Attracting and retaining talent | McKinsey Skip to main content To slow down attrition, pay closer attention to what workers really need September 22, 2022 | Podcast By Bonnie Dowling LinkedIn Twitter Facebook Email Article (8 pages) To slow down attrition, pay closer attention to what workers really need LinkedIn Twitter Facebook Email Article (8 pages) The Great Attrition continues, but if companies understand the differences among five common employee personas, they may be able to find and retain talent more effectively. DOWNLOADS Article (8 pages) Is there any way to stop the Great Attrition? McKinsey associate partner Bonnie Dowling says yes. In this episode of The McKinsey Podcast , she speaks with editorial director Roberta Fusaro about McKinsey’s latest research on the Great Attrition . This time, the team analyzed data from multiple countries and identified five types of worker personalities that, if catered to appropriately, could help fill all those open positions. After, we’ll hear from Catherine Price, author of Power of Fun: How to Feel Alive Again , a book that was featured in our Author Talks series, who says fun is not a treat, but a tool. The following transcript has been edited for clarity and length. The McKinsey Podcast is cohosted by Roberta Fusaro and Lucia Rahilly. Quantifying the disconnect between employers and employees Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes Roberta Fusaro: Bonnie, thank you for joining the podcast. Bonnie Dowling: I’m excited to be here, Roberta. Thank you for having me. Roberta Fusaro: It seems like companies can’t find enough workers, and they can’t fill jobs fast enough. It also seems like we’re seeing a lot of people who are still leaving companies at just as great a rate. What did McKinsey’s most recent research on the Great Attrition reveal about these comings and goings? Bonnie Dowling: This is our second or even third installment of the Great Attrition/Great Attraction research . This time, we surveyed over 13,000 individuals from six different countries. We looked at Australia, Canada, India, Singapore, the UK, and the US, across 16 industries and included people across all ages, as well as incomes. We found that people are continuing to quit their jobs. They are going to different industries. They are leaving the workforce entirely. These things haven’t necessarily changed since our original research despite the fact that the economy and the world has changed, which begs a question about what’s really going on and the disconnect between employers and employees and why we haven’t been able to fix that. Roberta Fusaro: As you mentioned, we’ve been looking at labor trends since the onset of COVID-19. Are these numbers much different? Are things getting any better? Bonnie Dowling: We did this research to understand what those trends were and what was happening, especially as we saw some things happen in the broader economy, like economic uncertainty overall. However, what we’ve seen is, when it comes to worker mentality, it has not caused a change. In August 2021, when we ran this research and a very similar survey, 40 percent of employees said they were at least somewhat likely to leave their jobs in the next three to six months. When we ran this research again April 2022, we found that the same amount were at least somewhat likely to leave their jobs in the next three to six months. Roberta Fusaro: Is this true across the board? Is this true across all industries? Is it true at all levels? Bonnie Dowling: There’s some variation. What’s interesting about this fundamental shift that maybe is not a trend, is it is striking across all levels, from frontline employees to executives. It’s also striking across all industries. Yes, you see higher churn in areas like retail or hospitality that historically have always had higher churn. But in finance or insurance organizations you’re seeing high levels of churn as well. Roberta Fusaro: In the report on the Great Attrition research, you and your coauthors mentioned that “The tried and true approaches to finding and keeping employees aren’t working anymore.” Why is that? Bonnie Dowling: Compensation is table stakes. If you’re not offering a competitive wage, you’re not in the game to start with. But it’s not enough anymore. What we’re seeing is there are different types of people who are looking for different things, especially those who’ve stepped out of the workforce entirely. What’s interesting right now is we can’t afford to ignore those who have stepped out. Why? Because there are not enough people looking for jobs. There are not enough people in the workforce to fill the number of jobs we have openings for, at least not in the US. Want to subscribe to The McKinsey Podcast? Google Podcasts Apple Podcasts Spotify Stitcher RSS Listen to previous episodes Five worker personas Roberta Fusaro: What’s most fascinating about the most recent Great Attrition research is that you’ve identified these worker personas, breakdowns of what different types of workers might be looking for in the workplace, and therefore what employers need to do to attract them. How did you come up with these personas? Bonnie Dowling: We started to look at those who had quit jobs without a job in hand, and thus had taken a step out. We also looked at those who quit jobs without a job in hand but returned to the workforce and those who had quit and returned to different industries. We looked at a variety of people who had left their jobs over the course of the pandemic. And we did a cluster analysis around what they said would either bring them back into the workforce, would bring them back to a traditional job within the workforce, or what had brought them back. That’s what allowed us to start to identify these different groups. Roberta Fusaro: What are the personas? Bonnie Dowling: There are five: the traditionalists, the do-it-yourselfers, the idealists, the caregivers, and the relaxers. Roberta Fusaro: Let’s start with the traditionalists. Who are they, and what are they looking for in the workplace? Bonnie Dowling: These are the folks who didn’t quit their jobs during the pandemic. If they did quit, they moved immediately into another very traditional job. They’re the folks who are motivated by the same things that motivated us in 2017 and 2019. They’re dependable for employers looking for employees. They want a clear path to advancement. They want competitive compensation. They want a workplace where they enjoy their colleagues and have inspirational leaders. Roberta Fusaro: What’s tough about attracting and retaining this type of employee, if anything? Bonnie Dowling: What’s tough about traditionalists is there’s not enough of them left. If we think about how many people have quit their jobs over the course of the pandemic, in the US alone it’s about a third of the workforce. That means only about two-thirds are what we would classify as traditionalists. Given the number of job openings we have today, which is about double, even in our most recent numbers from June, even those ones point to there being almost double the number of job openings as there are people looking for jobs. That means you have to look beyond the traditionalist employee. They may be easy to attract, they may be easy to recruit, but if there are not enough of them, you can’t depend on filling your gaps with only them. That’s why you need to look at the other personas. Roberta Fusaro: Let’s hear more about the do-it-yourselfers. Who are they? Bonnie Dowling: The do-it-yourselfers are a group of people who are still working, but they’ve gone on to start their own businesses or become contractors or join the gig economy. They tend to be in the 25- to 45-year-old age range. They really value flexibility, meaningful work, and compensation. To get this group back, you’re competing with them, because they’ve created something for themselves that’s paying the bills, that’s providing them a sense of purpose, and gives them all of the flexibility they could ever want, because they’re their own boss. Getting them back means you’ve got a bit of an uphill battle, but it’s also probably the largest group out there. Roberta Fusaro: How do companies then compete with this new set of competitors? Bonnie Dowling: Exactly. How do you prove you’re a better boss than someone is to themselves? It’s going to be a challenge for sure. Companies need to emphasize their sense of purpose and their mission. How do you prove you’re a better boss than someone is to themselves? It’s going to be a challenge for sure. Companies need to emphasize their sense of purpose and their mission. Bonnie Dowling The other thing companies can offer that is difficult for an individual upstart is thinking through your full benefit package. Not just your compensation, although having a steady, more uncertain-economy-proof compensation package is helpful. You also have to think through what benefits you have that are different and more attractive. Beyond that, one of the key things individuals get by working for themselves is flexibility. I started hearing executives talking about employees leaving in droves when they told me they’d announced their return-to-office policy. They thought they were being flexible by offering things like “Three days in office, two days’ work from wherever you want. It just can’t be Mondays and Fridays.” They were shocked to see people quit. And the first thing I said was, “Well, you’re sending the inherent message that you don’t trust your employees not to take a four-day weekend when you put in that policy.” They were a bit confused by that but started to recognize that maybe that wasn’t the message they wanted to send to employees that had kept the lights on through the pandemic. Now, as those return-to-office policies and approaches are starting to materialize, we see more people and more organizations thinking through what that’s going to look like, even if it is going to change and evolve as the virus continues to change and evolve. If you don’t take into account flexibility—and not just flexibility of location but also the time of work, how we work, all of those pieces—you will not be able to attract the do-it-yourselfers back. Roberta Fusaro: What are caregivers looking for, and who are they? Bonnie Dowling: It probably doesn’t come as a surprise that flexibility is going to be a very important factor for caregivers. They’re predominantly women. They’re aged 18 to 44. They’re folks who have children at home or maybe elderly parents. They need flexibility to live in both their role as a caregiver and as an employee. What we see here are former employees who may want to return. They may have enjoyed that sense of purpose and the ability to advance that they got from their career. They may be traditionalists at heart, but what they found over the course of the pandemic was that it wasn’t working to both be a parent or a child with aging parents. It wasn’t working to be a caregiver and employee. If you think about things like the skyrocketing cost of childcare and how it became so unreliable over the course of the pandemic, and then you think about how many employers have punitive policies in place if you’re tardy or have an absence, it started to not have the right ROI. If I’m spending the majority of my after-tax paycheck on childcare and yet I’m also getting dinged at work because I get a call from the school at 7 in the morning that says, “We’ve gone virtual for the day because there is an exposure,” the ROI isn’t there. So people started stepping out. If we want to attract them—and many of these individuals would like to return, it seems—we need to think about how we can truly create flexibility and how we can align the benefit packages we offer to their needs. So you see the increased provision of childcare, of on-site childcare, of benefits like that. One of the other benefits I really liked when I spoke with a company recently was thinking about how they could offer housecleaning or landscaping benefits. And their thought was, “You know what? A lot of people would enjoy these benefits.” It turns out that there’s a lot of things we do outside of work that don’t give us a great sense of fulfillment or pleasure or joy. And if, as employers, we can look at what some of those things are and we can help take those off their plates, we can actually give back some work–life balance and ultimately flexibility to our employees. Roberta Fusaro: This next persona is also of particular interest to me because my older son falls into this category of the idealist. Who are the idealists and what are they looking for? What can companies do to bring them into the fold? Bonnie Dowling: It’s probably not surprising that the idealists are our younger folks. They’re aged 18 to 24. Many of them are students. They may be part-timers. They’re not encumbered by the same responsibilities that those of us who may be caregivers or well into our careers and have mortgages and other responsibilities, car payments, things like that—they’re not worried about those things. As a result, they’re focused on different things. Flexibility remains top of mind for them. Maybe it’s to work with their school schedule. Maybe it’s to pursue some of their passion areas. We know the generation entering the workforce right now, Gen Z, or the digital natives, are really thoughtful about the work–life balance piece and not wanting to work forever or work to live. They want to live and have a job. So the flexibility is really important; so is development and advancement. They want to see that there’s a career path that they can grow. They also want meaningful work. Purpose is very important to this group. Roberta Fusaro: The last persona on the list, but by no means the least, is a group we’re calling the relaxers. Can you talk a little bit more about this group? Bonnie Dowling: Many of them are the folks who have taken an early retirement. It’s people who have retired or they’re just not looking for work. They don’t really have a reason to come back. But we also know there are things that could bring them back. We also know, given the gap that we have, that we need to figure out a way to bring them back. They’re looking for something that’s purpose-driven, maybe something that relates to a legacy they would like to leave. They’re maybe not as motivated by compensation the way others are, because they probably have enough money to live comfortably for a while. That’s why they’ve retired and stepped out. They want the flexibility element. They’re going to want to work with friends. We call them the “Gronks” because of what happened with Rob Gronkowski, the American football player. He retired. His best friend, Tom Brady, called him and said, “Hey, you said you were retired. I was thinking about it too. But do you want to come back and win a Super Bowl?” And he came back to play. It was a short-term contract. He did a year-to-year approach. He got to work with his best friend, and he had the opportunity to do something big. Those are the personas, and those are the people we have to start thinking about, and how we can broaden our employee value proposition to encompass those different types of people coming in, and how we can focus on the elements of our employee value proposition that will be most appealing to those individual groups that we want to attract. Moving between personas Roberta Fusaro: Is there any overlap among the personas, or do you truly fall into one category
or another? Bonnie Dowling: There’s a lot of opportunity to move between them. If I think of myself as an example, I believe I’m probably a traditionalist at heart, because I haven’t quit my job. I have a lower risk tolerance. I want to know where my paycheck is coming from. However, I’m also a mom. I have a lot of responsibilities at home. And I’m burned out, like 42 percent of women out there, according to our Women in the Workplace research . Could I easily become a caregiver and step out? Yes, I could. There’s a lot of opportunity to move between these. Could you be both at once? You could probably be both at once as well. It’s a matter of how you’re prioritizing the different offerings of an employer and how you would think about what it would take to bring you back. Confronting the long-term attrition problem Roberta Fusaro: The framework of the personas is a great tool for companies to look at their employee value propositions. There are certain characteristics common across them all around flexibility, compensation, development, purpose. Apart from getting a better understanding of the talent pool through these personas, what else can companies do to address the attrition problem longer term? Bonnie Dowling: There are four different levers they can take. There is what our article addresses, which is, how do you attract talent that is out there and looking, or on the sidelines looking, to get back in? We also ask, how do you retain the talent you currently have? You need to create a culture that’s welcoming, sticky, and doesn’t mean bringing people back into the office. Think about what it is you can do, how you can make a concerted effort to show employees you care about who they are and build a culture even in a virtual environment. When executives tell me, “Aren’t you saying employees want to have their cake and eat it too? They want a strong culture, and they also want to be flexible and work from wherever, isn’t that oxymoronic?” I say, “No. If you’re dependent on being in person for your culture, because you think people need to see the posters in the elevators or see people in the water cooler area, then it’s lazy and it’s marketing, not a culture.” Culture is how we interact with each other on a daily basis. It’s how our managers lead. It’s how our individuals and colleagues look out for one another. That’s the core of culture, and you can do that virtually, or you can do that in person. You just have to make the effort and invest in it and prioritize it. There are two longer-term plays as well. When I think about the longer-term plays, I think one of them is, how do you reduce your demand for employees? How do you change the work you’re doing in a way that means you don’t need as many employees as you did previously? You have to rethink how you work. You have to look at some of the gains made over the course of the pandemic in terms of automation, in terms of use of things like the virtual check-in process when you go to a doctor’s appointment or the QR menus you see at restaurants that have started to be used more widely. The fourth one is, how do you increase the supply of workers for the long term? How are you getting the next generation excited about your industry? How are you partnering with schools, vocational programs, universities, colleges to offer the scholarships, to offer the flexible work schedules to attract them? All of those pieces are going to be really important to get people interested. It’s those four pieces—the two that are immediate around attracting and retaining what is available, and the two that are longer term, which are about reducing your actual demand as well as increasing the supply. Roberta Fusaro: Bonnie, I know I’ve learned a lot about where I fall into these personas and learned a lot about what companies can do to combat the Great Attrition. Thank you for taking the time and joining us today. Bonnie Dowling: Absolutely. It was wonderful, a real treat. Lucia Rahilly: Maybe another way to combat the Great Attrition is to have more fun at work. Is it possible? Let’s take a listen to Catherine Price, author of Power of Fun: How to Feel Alive Again , a book that was featured in our Author Talks series, who says fun should be a priority, not an afterthought. Catherine Price: At first glance, it might seem like the absolute worst time to be talking about a book about fun, given that we’re in year three of a global pandemic, and many people are feeling like fun is the last thing they could devote attention to. But what I’ve come to realize is that the opposite is true. Fun can boost our resilience and our spirits in a way that makes it easier for us to cope with whatever life may throw at us. We need to rethink how we think about fun: less as a treat we get to have only if everything’s already going great and more as a tool we can tap into to help ourselves weather the challenges life may present us with. One of the more interesting things I stumbled upon when I was trying to write a book about fun is that there’s not a very good definition of what fun truly is. I came up with the idea that fun, or as I call it “true fun,” is the confluence of three psychological states: playfulness, connection, and flow. Playfulness means you’re doing things just for the sake of doing them, and you’re not caring too much about the outcome. Connection is a fundamental element of fun because in the vast majority of situations or anecdotes people have shared with me, another person or another creature is involved. There are also cases in which you can be truly connected to an activity or to your authentic self or to your body. But often there’s a person involved, even for introverts. Flow is the psychological state in which you get so absorbed in your present experience you lose track of time, like an athlete in the midst of a game or a musician playing a piece of music or when you’re in the middle of a really engaging conversation. Making a point of noticing moments of playfulness, noticing moments of connection and noticing moments of flow can go a long way in helping us benefit psychologically from the moments of fun that we are already experiencing. We are constantly on high alert, but that’s very stressful and anxiety producing. One of my hopes for the book, in addition to it being a fun read, is to help us shift away from our natural tendency to focus on the negative and begin to appreciate some of the positive stuff that happens all the time, even in difficult times. It can be a real challenge to try to fabricate fun, especially in a group workplace setting. I’m not going to claim to have the perfect answer to that, because I do think fun is much like romance: if you try to force it too much, it’s not going to happen. What you can do, though, is set the stage for it. Fun is much like romance: if you try to force it too much, it’s not going to happen. What you can do, though, is set the stage for it. Catherine Price If you’re in a leadership role, one thing that’s conducive to fun is to allow your humanity to show. Anyone in a leadership position who’s able to make a joke at their own expense or have a more lighthearted, less professional, less jargony attitude, that’s going to help. About the author(s) Bonnie Dowling is an associate partner in McKinsey’s Denver office. Roberta Fusaro is an editorial director in the Waltham, Massachusetts, office, and Lucia Rahilly is global editorial director and deputy publisher of McKinsey Global Publishing and is based in the New York office. Explore a career with us Search Openings Related Articles Article - McKinsey Quarterly The Great Attrition is making hiring harder. Are you searching the right talent pools? Podcast From the great attrition to the great adaptation Article - McKinsey Quarterly Gone for now, or gone for good? How to play the new talent game and win back workers Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. الرياض | الشرق الأوسط | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content الرجوع إلى Locations ماكنزي في الشرق الأوسط الرياض الرجوع إلى Locations ماكنزي في الشرق الأوسط الرياض يعمل مكتب ماكنزي في المملكة العربية السعودية على مساعدة القادة السعوديين من أجل تحقيق كامل إمكانات المملكة المتمثلة في إرساء طبقة وسطى نابضة بالحياة تتمتع بمستوى معيشي مناسب وتتميز بمنظومة قائمة على قطاع خاص على مستوى عالٍ من التنافسية وقطاع عام معاصر. حققت المملكة العربية السعودية خلال السنوات الماضية نموًا هائلاً، ولكنها مع ذلك لا زالت تتمتع بحجم كبير من الإمكانات التي يمكن تحقيقها في المستقبل. وانطلاقًا من ذلك، فإننا ننظر إلى عملنا في المملكة كمصدر فخر وأهمية كبيرة بالنسبة لنا لأنه يركز على مساعدة المملكة على إطلاق العنان لإمكانات الشباب السعودي. ويشمل ذلك قطاعات التعليم والتوظيف وخلق فرص العمل ومساعدة الشركات السعودية على أن تحتل مكانةً ريادية على المستوى الإقليمي والعالمي والابتكار في مجالات الأعمال المصرفية الإسلامية والعمل الخيري والقطاع العام. يرتكز عملنا في المملكة بشكل رئيسي على الالتزام العميق برعاية وتعزيز الجيل القادم من القادة السعوديين. الوظائف العمل في الرياض نحرص على توظيف الكفاءات المتميزة من المملكة العربية السعودية ومختلف أنحاء الشرق الأوسط والعالم. ونقدم لموظفينا فرصة لا تضاهى للنمو على المستوى المهني والشخصي وفي نفس الوقت نمنحهم الفرصة لبناء علاقات قوية أثناء مساعدة المؤسسات الكبرى على تحويل أدائها. اطلع على الوظائف الشاغرة في مكتب الرياض البحث في الوظائف هل قدمت طلبًا؟ قم بالتسجيل هنا كن مستعدًا للمقابلة اعرف المزيد المسؤولية الاجتماعية برنامج قمم أطلقت شركة ماكنزي برنامج قمم سعيًا منها لمساعدة الجيل الجديد من الطلبة السعوديين على إطلاق طاقاتهم وتحقيق أقصى إمكاناتهم. نساعد عملائنا كل يوم على حل أصعب المشاكل التي تواجههم، وبناء قدراتهم، وتحقيق نتائج أفضل. اعرف المزيد عن موقعنا العالمي للوظائف مكتبنا الرياض الرياض برج المملكة الطابق 23 ص. ب. 54517 الرياض 11524 المملكة العربية السعودية رقم الهاتف: +966 (11) 212 3600 رقم الفاكس: +966 (11) 212 3800 https://maps.google.com/maps?q=24.713309,46.674385&z=15 اعرف المزيد عن مكاتبنا في الشرق الأوسط أبوظبي القاهرة الدوحة دبي المنامة الرياض أكثر من 140 مكتب حول العالم اعرف المزيد عن مكاتبنا في الشرق الأوسط تواصل مع ماكنزي الشرق الأوسط اتصل بنا The power of through-cycle M&A strategies | McKinsey Skip to main content The power of through-cycle M&A April 30, 2020 | Article By Jens Giersberg, Jan Krause , Jeff Rudnicki , and Andy West LinkedIn Twitter Facebook Email Article (PDF -391KB) The power of through-cycle M&A LinkedIn Twitter Facebook Email Article (PDF -391KB) How companies can use deal making to accelerate strategic shifts during the COVID-19 crisis. DOWNLOADS Article (PDF-391KB) Meeting the humanitarian challenge of COVID-19 is the imperative of our time. While healthcare workers and first responders battle the virus, risking their lives to save others, business leaders have a different but related mandate : to protect the health and safety of employees and customers while preserving people’s livelihoods. The companies that can quickly adjust their strategies to the economic and humanitarian shocks of the pandemic will be best positioned to emerge from the crisis healthy and profitable. Our experience and two decades of research show that a through-cycle mindset to M&A can enable and even accelerate these strategic shifts. History has shown that outperformers actively reserve cash and invest—organically and inorganically—through downturns. In this article, we examine how companies can use M&A to better weather the current economic storm and come back stronger in the “next normal.” A fog of uncertainty surrounds this moment, but our research into resilient companies reveals a few truths that can help businesses prepare to move when the mist lifts: continue to invest through downturns, especially in M&A; pursue programmatic M&A through economic cycles; and adjust corporate and M&A strategies rapidly to adapt to a changing world. We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Keep investing—even in downturns The global scale of the crisis, the wide ranges of disease progression, the likelihood that the next normal will require new strategies—these and many other factors are behind the fog of ambiguity enveloping industries today. To cut through this ambiguity, business leaders will need to explore
a range of scenarios and develop plans to navigate them. As we write this, few business leaders have the confidence to finance, approve, or execute transactions. Indeed, fewer than half as many transactions were completed in March 2020 than in March 2019, and many have been put on hold or pulled. The depth of this M&A pullback was highlighted the week of April 13, 2020—the first week since September 2004 that no deal larger than $1 billion was announced. 1 1. Kane Wu, “Coronavirus takes toll on global M&A as $1 billion deals disappear,” Reuters, April 20, 2020. M&A activity is likely to continue to be limited in the short term given the critical focus on employee safety, financing challenges, and volatile valuations. But we believe business leaders should heed lessons from M&A trends during the dot-com crash and the 2008 financial crisis (Exhibit 1). Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com The total value of transactions declined by between 40 percent and 60 percent, especially in financial-sponsor activity. M&A practitioners with through-cycle mindsets kept making deals, however, benefiting from lower valuations in the later stages of the crises and then accelerating their activity from 2009 through 2012. They also pursued a broader range of transaction types, including minority investments and partnerships; used more noncash structures; and accepted higher-leverage ratios. The COVID-19 crisis is unique, of course, and the current economic contraction will not evolve in exactly the same way previous downturns did. At some point, however, deal making will resume. The combination of unprecedented economic stimulus packages, low interest rates, and strong signaling from central banks may help cushion the blow and eventually jump start investment and consumer spending. Additionally, prices have fallen sharply for a range of commodities from oil and copper to lumber, which may help enable growth in many industries. Pursue programmatic M&A through cycles Sidebar LinkedIn Twitter Facebook Email Understanding the value of programmatic M&A Our more than two decades of research into the Global 2,000, the world’s largest public companies by market capitalization, shows that through-cycle “programmatic M&A”—making many relatively small transactions as part of deliberate and systematic M&A programs—delivers far more total returns to shareholders on average than other approaches, including organic growth and “big bang” deals, with less volatility and risk. The findings suggest that most companies should continue their active portfolio management, and, on average, a programmatic approach performs best, even in an economic downturn. The overall cycle of M&A is volatile; deal making typically declines during an economic shock and picks up again as the recovery begins to take hold. No one knows when the current crisis will end, or exactly how it will reshape markets or human behaviors. However, we do know that during the last downturn the companies that used a programmatic approach to M&A delivered excess returns with less volatility than did companies that used other approaches to M&A (Exhibit 2). This is consistent with more than 20 years of McKinsey research on programmatic M&A (see the box “Understanding the value of programmatic M&A”). Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com We have also found that, among companies practicing programmatic M&A, those who were more aggressive about reshaping their portfolios divested more in the last downturn. 2 2. Jeff Rudnicki, Kate Siegel, and Andy West, “ How lots of small M&A deals add up to big value ,” July 2019; and Martin Hirt, Kevin Laczkowski, and Mihir Mysore, “ Bubbles pop, downturns stop ,” May 2019. While volatility is high, our review of recent data also suggests that, from January 2019 through March 2020, the programmatic approach to M&A outperformed all others. Update corporate and M&A strategies—now Sidebar LinkedIn Twitter Facebook Email A checklist for maximizing the value of M&A By focusing on three C’s—competitive advantage, capabilities, and conviction—deal makers and executives can better manage their M&A program and use them as a means to weather the economic downturn resulting from COVID-19. Competitive advantage Determine the effect of COVID-19 shocks on overall strategy and business portfolio. Reevaluate two or three priority M&A themes to support the updated strategy. Redefine deal-screening criteria with updated M&A themes to refresh target lists. Capabilities Update “dry powder” analysis to account for changes that may limit the company’s ability to pursue transactions. Develop alternative deal structures to manage valuation volatility and liquidity challenges. Retool integration approaches based on refreshed M&A themes and COVID-19-induced execution challenges. Conviction Secure executive and board commitment to M&A to accelerate decision making when markets thaw. Build a through-cycle valuation perspective to enable transactions during the crisis. Review the full landscape of potential targets before rushing to assess distressed assets. Carefully choreograph outreach to targets and tailor the value proposition to acknowledge the crisis. The companies most likely to emerge strongly from the COVID-19 crisis will be those agile enough to adapt to their unique situations and pivot their M&A activities toward the most relevant and attractive opportunities before the next normal arrives . This will require a reassessment of their position in three areas: competitive advantage, capacity, and conviction. These three C’s are the foundation for maximizing the value of M&A (Exhibit 3) (see the box “A checklist for maximizing the value of M&A”). Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Rethinking competitive advantage The COVID-19 crisis will test every corporate strategy because its depth and scope exceed anything we have seen previously. Like the Great Depression and the world wars, it will drive lasting change in people’s and organizations’ perceptions and behaviors. In the business realm, leaders will need to determine what is fundamentally changing in terms of market performance and outlook: which market shocks are temporary, and which are permanent; which require immediate action to manage risk, and which create new long-term opportunity. M&A is a fundamental enabler of corporate strategy, and in our experience, the most successful senior leaders can describe exactly how and why M&A contributes to the company’s performance . They see the direct links between M&A strategy and corporate strategy, and they update and refine their M&A priorities to support specific corporate goals. As they think about deal making during the pandemic, executives should understand the areas in which M&A is the most effective tool to meet the company’s strategic goals and how the company’s distinct capabilities could add more value to M&A targets. The most effective companies will develop very detailed approaches for executing in priority areas, including identifying specific deal-screening criteria to narrow the range of potential targets (Exhibit 4). 3 3. Sophie Clarke, Robert Uhlaner, and Liz Wol, “ A blueprint for M&A success ,” April 2020. Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Assessing capacity in a changing environment Every crisis demands tremendous organizational capacity—to ensure safety, stability, continuity, and clear communications. M&A, too, requires capacity. Acquisitions require financing, of course, but companies also need the talent, systems, and governance to deliver on the promise of each new asset. In a challenging and rapidly changing environment such as this one, acquirers must conduct a thorough assessment of their capacity to execute deals and integrate or separate from assets. They should consider the following criteria when assessing M&A capacity: Update ‘dry powder’ analysis. Estimate how much capital can be reasonably deployed for M&A in the current environment, acknowledging that it has changed but that it should not be zero in most cases. An assessment of the current capital structure should include a look at leverage and equity ratios, debt covenants, rating requirements, and investor expectations. Financial capacity may change quickly, so this assessment should involve a range of scenarios and input from all members of the management team. Would you like to learn more about our Strategy & Corporate Finance Practice ? Visit our Corporate Finance page To expand M&A capacity, look toward various sources of financing—internal sources, as well as debt and equity financing. Internal financing in times of crisis might include explicitly prioritizing M&A over other previously approved capital investments or reducing capital allocations to shareholders, especially by slowing share buybacks. Investors are not likely to see dividend reductions as suitable M&A financing. Leaders raising debt or equity should carefully assess lenders’ and capital markets’ receptiveness and the achievable pricing, given expanding debt spreads and depressed equity valuations. Accelerated equity financing at lower levels can lead to higher dilution of current shareholders and therefore needs to be marketed to investors with a crystal-clear rationale and equity story. Sidebar LinkedIn Twitter Facebook Email An overview of deal structures Different types of deal structures can help balance risks and benefits between buyers and sellers. It is helpful to view potential structures along the following three dimensions: Upside for sellers Earn out structures. The deal emphasizes elements of the purchase price that are based on financial success after the sale—for instance, sales or profitability growth achieved. Upside for sellers and downside protection for buyers Seller to retain stake. The deal involves minority investments or “less than 100%” transactions that can be combined with strategic partnerships. Vendor notes. The seller provides a loan to the buyer to help finance parts of the purchase price; often combined with earn-out structure which determines final redemption. Option structures. Buyers and sellers are allowed to exercise various options in case financial metrics are over- or underdelivered after sale. Relative contributions. The transaction is based on cash flow and EBITDA contribution instead of a share-price exchange ratio or premium. Swaps and in-kind structures. In this noncash transaction, complete assets are traded or deals are centered on in-kind contributions (including shares). Downside protection for buyers Specific warranties. The seller provides expanded guarantees—for example, regarding the nature of business, historical financials, or litigation and environmental risks. Phased payments. The purchase price is paid in multiple installments, potentially linked to buyers achieving key milestones after sale. Develop alternative deal structures. Given the current volatility in valuations and companies’ liquidity challenges, acquirers may need to consider a broader range of deal structures (see the sidebar, “An overview of deal structures”). The optimal structure for a deal is one that helps both sides manage uncertainty and share risks and rewards. Some companies will forge new partnerships to access capital, particularly with asset-management and private-equity firms with dry powder and an appetite for deals. An important consideration with alternative deal structures is to make sure that terms do not undermine the acquirer’s plans to integrate the target. Revise integration approaches. To pursue new M&A themes and to manage crisis-induced execution challenges, many acquirers will need to update their integration tools and approaches. For instance, the business may need to refocus on value creation as leaders scramble to get back to precrisis operational levels. Deals may invite more regulatory scrutiny and take longer to approve. To that end, acquirers should consider readjusting their preclose-integration planning and timelines to stay flexible and manage potential delays. They will also need to actively manage the expectations of key stakeholders, including employees and capital markets. Financial and operational pressures could trigger the need for immediate restructuring of acquired targets. And finally, bridging corporate cultures, which is challenging in the best of times, will be difficult when in-person interactions are limited or impossible. Personalized approaches, such as scheduling more one-on-one and small-group videoconferences, will likely be the most successful ones. Senior leadership can use these channels to address critical integration questions while frontline leaders work virtually with their newly formed teams to get to know one another and manage day-to-day issues. Companies should plan for a second deliberate phase of cultural integration once the crisis wanes and employees begin to return to work as normal. Finding and building conviction Confidence and boldness are scarce in every crisis, but the ability to act quickly and decisively is the key to successful M&A. In our experience, the best acquirers commit to clear strategies, avoid deals that may look good financially but do not quite fit into those strategies, and prepare to execute before the fog lifts. Secure commitments to accelerate decision making. Senior managers and the board will need to gain agreement on the importance of M&A in the crisis and postcrisis strategy, even as they are addressing other urgent priorities. They should jointly clarify how the crisis requires a shift in strategy and how M&A and its specific themes will enable the company’s outperformance in the next normal. Getting ahead of the next stage of the coronavirus crisis Read the article To establish the capacity and focus needed for M&A, senior leaders should make the case for through-cycle M&A valuation and manage expectations by setting financial guardrails and defining boundary conditions. They should also identify who will lead the execution of M&A, from target identification and outreach to integration. To maintain alignment and flexibility, many leaders will conduct regular M&A working sessions with key decision makers in which they discuss potential targets and priorities. To remove decision-making roadblocks, some companies will streamline their governance and formal approval processes, set clear milestone deadlines, convene promptly when necessary rather than at a set cadence, and convey a sense of urgency based on early-mover advantages. Updates on M&A should be a recurring item on  executives’ and boards’ agendas. Build a through-cycle valuation perspective. A target’s lower valuation may attract a buyer, assuming no fundamental changes to the underlying business, but the owners of the target may be fixated on precrisis valuation (Exhibit 5). To make the transaction possible, the buyer may need to convey the additional through-cycle value-creation potential from the deal and raise the premium and propose creative structuring for the deal. Exhibit 5 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Develop comprehensive target lists for each M&A theme. The most successful acquirers look beyond obvious targets, including distressed assets, and carefully study the broader landscape to find the most suitable candidates in the context of their prioritized M&A themes. 4 4. An M&A theme is an area in which the company needs M&A to deliver its strategy and has the ability to add value to targets. They, of course, ensure that any acquisition is anchored in strategy. Buying a distressed asset that is inconsistent with strategy will likely not create the value expected and could distract from other, more strategic pursuits. Reach out to priority targets with unique, tailored value propositions. Acquirers should be forthright about why a deal makes sense now. Examples of tangible advantages may include a strong balance sheet, reliable infrastructure, resilient supply chain, trusted brand, and effective sales force. In addition to appealing financials, a compelling pitch usually features an exciting vision tailored carefully to the target, including a clear business case built on common goals, values, and a commitment to invest in the business. These conversations are the basis for proactive deal sourcing—going beyond discussions of assets and prices to cultivate relationships and present a compelling value proposition that spells out the deal’s benefits to the target and its stakeholders. Internally, business leaders should identify and gain agreement on what is nonnegotiable before beginning conversations with targets. A single senior leader should manage executive interactions with each target, using intermediaries sparingly. In the end, most successful deals are built on trust and shared values, not price per share. Uncertainty from COVID-19 and the human toll it is taking make this crisis much more worrisome than many previous global challenges. It’s unclear how long the pandemic will last, when a treatment or vaccine will be discovered or widely available, or how much economic pain we will experience at the global, national, local, or personal level. The situation will remain fluid, but we know some things for certain: the fog will lift, and when it does, the companies positioned to execute through-cycle M&A strategies quickly—those who have done the homework on their strategies and have strong balance sheets and a through-cycle mindset—will be able to deliver more value for shareholders, employees, customers, and the communities in which they do business, and can help speed the global recovery. About the author(s) Jens Giersberg and Jan Krause are both partners in McKinsey’s Cologne office, Jeff Rudnicki is a partner in the Boston office, and Andy West is a senior partner in the Madrid office. The authors wish to thank Patrick McCurdy, Philipp Espel, Manny Fragata, Tobias Lundberg, Kate Siegel, Robert Uhlaner, and Liz Wol, for their contributions to this article. Explore a career with us Search Openings Related Articles Article Safeguarding our lives and our livelihoods: The imperative of our time Article Getting ahead of the next stage of the coronavirus crisis Article - McKinsey Quarterly How lots of small M&A deals add up to big value Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Strategy & Corporate Finance consulting | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Strategy & Corporate Finance Building enduring value through bold strategies How we help clients Board Services Business Unit Strategy CEO Excellence Corporate Finance Corporate Strategy Digital Strategy Executive Transitions Growth & Innovation M&A Net Zero & ESG Strategy Regulatory Strategy Trends & Global Forces Our approach Empirical evidence Creative data mining enables us to produce privileged insights and finely calibrate strategies that we know work; these are not simply theoretical claims backed by a few cases but rather statistically valid ones based on large-sample learnings from nearly 100 years of experience. Bigger, bolder bets Uncertain times demand bold responses, yet many large corporations tend toward timidity and inertia. To be successful, companies must make bold moves to unlock new sources of growth. Making the right moves involves using granular analysis to choose the right submarkets, and upending the traditional approach to planning by reallocating resources—people, money, and management—to new areas. People—the social side of strategy Great strategy moves beyond analysis into an immersive journey where teams grapple with the issues, integrate a range of perspectives, and explode their own myths. This process can be challenging, but it creates the deep conviction people need if they are to commit to a bold new direction. Our thought leadership Book CEO Excellence From McKinsey & Company, an insight-packed, revelatory look at how the best CEOs do their jobs based on extensive interviews with... today’s most successful corporate leaders—including chiefs at Netflix, JPMorgan Chase, General Motors, and Sony. #CEOExcellenceBook Book Strategy Beyond the Hockey Stick The new book on how to beat the odds with a bold strategy Book - Practices Capabilities Valuation: Measuring and Managing the Value of Companies, 7th edition At the crossroads of corporate strategy and finance lies valuation. This book enables everyone, from the budding professional... to the seasoned manager, to excel at measuring and maximizing shareholder and company value. Impact stories Changing focus amid digital disruption How a digital transformation and entry into new business areas helped a high-tech firm thrive in the digital world. Charting a growth path for a newly public company How we helped a financial-services firm boost profitability and establish a digital and M&A strategy. More from this practice Noble Intelligence Harnessing artificial intelligence and other advanced technologies to address social and humanitarian crises McKinsey Regulatory Center We support you at all stages of the M&A transaction, including stakeholder management and postmerger integration. Strategy Analytics Center We help clients make better strategic decisions through a deeper understanding of corporate performance, value creation, macroeconomic forces, and global and local trends. McKinsey Academy McKinsey Academy helps organizations equip their people with the needed skills to achieve and sustain transformational impact. Corporate Performance Analytics Provides fast, sophisticated financial insights through automated benchmarking reports See all capabilities Featured insight Issue McKinsey on Finance, Number 79 February 1, 2022 – Perspectives on Corporate Finance and Strategy See all insights Our people Michael Birshan Senior Partner, London Global co-leader of our Strategy & Corporate Finance Practice, partnering with bold and determined leaders to set strategy, transform organizations,... Chris Bradley McKinsey Global Institute Director and Senior Partner, Sydney Leads research and works on strategic transformations for organizations across Australia and New Zealand’s media, banking, retail, and telecommunications... Carolyn Dewar Senior Partner, Bay Area Delivers large-scale performance-improvement programs that foster culture change and counsels senior executives making leadership transitions Dr. Laura Furstenthal Senior Partner, Bay Area Serves healthcare clients globally as well as not-for-profit organizations, governments, and Nobel laureates, guiding innovation in strategy,... Celia Huber Senior Partner, Bay Area Leads the North American Board Services practice area and advises organizations on strategies to help them thrive within the context of global... Anna Koivuniemi Partner, Amsterdam Leads growth strategy and analytics globally, helping clients to make strategic choices and leverage digital, data, and strong commercial skills... Ishaan Seth Senior Partner, New York Global coleader of our Strategy & Corporate Finance Practice; works with clients in financial services and private equity, including investment... Ben Stretch Partner, Sydney Leads McKinsey’s Strategy & Corporate Finance Practice in Australia and New Zealand, counselling senior executives on strategy, acquisitions,... More people Connect with our Strategy & Corporate Finance Practice Contact LinkedIn @McKStrategy @McKonFinance Medellín Office | Colombia | Colombia | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content McKinsey in Colombia Medellín Helping unleash the next wave of growth and innovation Medellín is known as “the city of eternal spring”, and McKinsey is proud to be helping its economy blossom. Our office, opened in 2017, supports both private- and public-sector clients to shape bold strategies for growth, achieve operational excellence, and bring out the best in their people. Our team includes top talent drawn from local universities, and experts from across our global firm. Our people Carlos Enrique Buitrago Partner, 
                                        Quito Coleads McKinsey’s Operations Practice in Latin America; helps clients across the region by delivering transformational improvements... Jaime Morales Partner, 
                                        Medellín Helps companies across sectors transform and grow by developing new markets; optimizing sales, operations, and technology; and... Careers Working in Medellín In the heart of magical Antioquia, we work in one of McKinsey’s newest offices with views of shining skyscrapers juxtaposed with jungle-covered mountains. Entrepreneurial, diverse, and international, our team of pioneers is bringing the best of McKinsey to a city that embodies urban transformation and modernization. Strong links from the office to local communities ensure that we contribute to public- and private-sector boards, universities, and charitable initiatives alike. The story doesn’t end there—outside the office we can be found paragliding, diving, debating, and of course, dancing the night away. All visitors receive a grand Colombian welcome! Explore jobs in Medellín Search jobs Already started an application? Sign in here Prepare for your interview Learn more Every day we help our clients solve their toughest problems, build capabilities, and create better outcomes. Learn more on our Global Careers site Our location Medellín Medellín Carrera 42 # 3 Sur – 81 Edificio Milla de Oro, Torre 1 Oficina 404 Medellín 050022 Voice: +57(4) 319 6200 https://maps.google.com/maps?q=6.201562,-75.572037&z=15 Learn More About Our Locations Bogotá Medellín 130+ offices worldwide Explore our locations Connect with McKinsey Colombia Contact Romania | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Welcome to McKinsey Romania We work with Romania’s leading companies to drive growth, reshape operations, build leaders, and transform the economy. Since 2006, we have worked with a range of companies—from telecommunications to financial services to oil and gas—to drive Romania’s economic transformation while enabling all their work with the right technologies. Combining global expertise with deep local knowledge, we help our clients shape ambitious growth strategies and deliver breakthrough performance. We attract some of Romania’s brightest talent and are passionate about building the capabilities of our people, our clients, and across society. We strive to achieve lasting and meaningful impact in all we do—for our clients and for Romania as a whole. 10+ years of McKinsey presence in Romania 160+ projects completed across all economic sectors 80 percent of consultants hold PhDs, MBAs, or master’s degrees Our people Alex Filip Managing Partner, 
                                        Bucharest Brings extensive experience serving clients across key industries, helping them tackle issues related to digital transformation,... Jurica Novak Senior Partner, 
 Romania Connect with McKinsey Romania Contact Negotiating a better joint venture | McKinsey Skip to main content Negotiating a better joint venture April 27, 2016 | Article By Eileen Kelly Rinaudo and Jason Roswig LinkedIn Twitter Facebook Email Article (PDF -3MB) Negotiating a better joint venture LinkedIn Twitter Facebook Email Article (PDF -3MB) As important as it is to secure the right terms for a shared enterprise, it is just as critical to form a sustainable relationship. DOWNLOADS Article (PDF-3MB) In the fast-paced world of deal making, joint ventures (JVs) are a conundrum. Slow in the making, often with complicated structures and shared management teams, they seem out of place in a volatile era marked by buzzwords that hype agility and nimble strategic moves. Yet there they are, more than 1,500 JV deals completed annually over the past ten years, including around 10 percent of them characterized as large JVs, with an initial value of more than $250 million. Their volume seems likely to endure—more than two-thirds of executives surveyed in 2014 reported that they expect to do more JVs in the future. 1 1. Eileen Kelly Rinaudo and Robert Uhlaner, “ Joint ventures on the rise ,” McKinsey on Finance , November 2014. This McKinsey Global Survey was in the field from March 11 to March 21, 2014, and garnered 1,263 responses from C-level and senior executives representing the full range of regions, industries, company sizes, and functional specialties. Of them, 982 executives had personal experience leading or managing joint ventures But JVs are not always embraced without reservation. In fact, we encounter many executives who express significant concerns, often when they’re wrapped up in the uncertainty of JV negotiations. Given how much longer those negotiations can last compared to traditional acquisitions, this is both understandable and alarming. One global conglomerate we’ve observed advises its US-based headquarters to expect JV negotiations to last three to six times longer than M&A negotiations. That’s a long time for doubt to creep in, particularly if the competitive context justifying a venture might shift in the meantime. How can executives build healthier partner relationships to give future JVs the best odds of success? Our review of a series of long-standing partnerships—supported by our 2014 survey and a series of structured interviews with JV partners 2 2. We interviewed 45 joint-venture managers. —identified three principles that made a difference in deal negotiations: investing more time and effort up front, working harder to cultivate and sustain the JV relationship, and standardizing key processes and learning mechanisms. Invest more up front As business negotiations go, JVs are marathons, not sprints. In their rush to complete a deal quickly and begin capturing value, inexperienced JV planners neglect the foundational steps of planning. Commonly, they jump too quickly into high-stakes discussions on specific deal terms such as how ownership is divided, who nominates key leaders, and what intellectual-property protections will be put in place. What they leave aside is an explicit understanding of how well those terms match the objectives of the deal. In fact, most companies need to invest more time in the early phases of deal planning and preparing for negotiations. Our research has shown that many planners focus more than half of their negotiating time hammering out specific deal terms that should be addressed late in negotiations and only 20 percent of their time on the JV structure and business model, which should be addressed first. In contrast, those same planners believe that the phases of the process devoted to internal alignment and the business model represent 60 percent of the total value at risk, while the phase devoted to deal terms accounts for only 10 percent (Exhibit 1). Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com That disconnect between time spent and value derived reinforces damaging habits. Deal terms are important, but they are difficult to correctly perceive and negotiate without a clear articulation of broader issues including deal objectives, market considerations, and walk-away points. Negotiators who lack that foundation are poorly prepared to discuss deal terms. The cost can often be measured in time. For example, negotiations slow considerably when negotiators fixate on specific, preconceived deal terms even though other solutions could also work or when they belabor negotiations on all possible considerations instead of covering the most likely ones. Cost can also be measured by long-term damage to the JV. When negotiators fail to examine a potential partner’s deeper motives or to consider the regulatory landscape fully, companies can end up with deal terms that don’t adequately govern an agreement—and that can carry substantial costs. For example, after a European company formed a JV to manufacture equipment in China, it unexpectedly learned that local regulators would compel it to transfer a larger equity stake to its Chinese partner, which threatened the deal’s
feasibility. If the European company’s negotiators had conducted a more rigorous up-front process, they likely would have discovered that requirement. Instead, the venture’s launch was delayed, and the European company’s governance rights were diminished—consequences that might have been avoided. Companies can avoid or at least mitigate such problems by investing more time in the early stages of planning. One US agricultural company requires extensive up-front business planning to confirm internal alignment and identify the motives of each counterparty. Planners there credit their rigorous preparation phase for making negotiations smoother. Would you like to learn more about our Strategy & Corporate Finance Practice ? Visit our Transactions page That’s consistent with our experience. We’ve found that companies benefit when they set up internal checks and balances to ensure that these foundational issues are articulated and confirmed internally before negotiations with partners begin in earnest. They should also engage potential partners in early discussions to confirm that they all agree on the goals of a joint endeavor, on their expectations of changes in the market over time, and on how the JV should plan to adapt as the market evolves. One global energy company learned this lesson the hard way when its partners in an existing JV objected that a new venture completed by the energy company would, over time, hurt the existing JV’s business prospects. As a result, a foreign court ordered the energy company to pay extensive damages for an initiative that never even launched. For most companies, a good starting point is for planners to force a tough and thorough self-review to identify their own objectives, goals, and—even more difficult—their strengths and weaknesses as JV partners. Where possible, they should also convince a potential partner’s leadership to do the same, lest they get mired in internal misconceptions in the future. Cultivate a trusting relationship Negotiating JVs differs from negotiating mergers or acquisitions because the end goal is a sustainable, ongoing, trust-based relationship, not a one-time deal. Not surprisingly, a significant portion of our survey’s respondents indicated that the level of honesty and trust between the parent companies had a significant impact on the partnership’s overall success (Exhibit 2). Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Positive initial meetings are important to establishing trust, but planners need to do more. Regular and ongoing business and social interactions with critical parent leadership-team members, including management off-site events and frequent, engaged board meetings, can help maintain trust and communication, reveal the breadth of motivating factors that influence a partner, and nurture a strong relationship even after negotiations conclude. As one energy executive
observed, it is frequently only after many hours together in a “smoky room,” spread over the days, weeks, and months of negotiation, that the true motives of potential partners become clear. Understanding partner motives and securing
mutual commitment to a deal beyond its financials will help ensure that all parties share the same expectations of ongoing JV operations. In our interviews, numerous executives expressed concern about nontraditional objectives that may be motivating potential JV partners. These include sharing capital to upgrade facilities, achieving a relationship with a previously inaccessible third party, or increasing employment opportunities for a specific region. Such objectives often work to the disadvantage of a JV partner, as managers at a global conglomerate discovered. They negotiated a deal with a regional player that included transferring core technology into the JV in order to qualify for lucrative government contracts. Conglomerate executives at first applauded the deal, though the planners expressed concern that their partner’s motives might not be consistent at all levels of its organization. The venture subsequently reached a tipping point when, during an industry conference, the regional company’s senior executives boasted that they would start selling products based on the global conglomerate’s technology, but at a fraction of the price. This forced deal teams on both sides to revisit the partnership’s objectives to reaffirm the relationship’s durability. Negotiators who understand a partner’s motivation, business needs, and capabilities well before closing a deal will be better positioned to establish a strong, candid relationship with shared, explicit expectations. Thorough research can highlight things that wouldn’t necessarily surface during negotiations but that could affect the partner’s involvement with the JV. For example, one energy company avoided a potential misstep after scrutinizing a partner company’s relationships with distributors before coinvesting in a local manufacturing operation. That analysis made it clear that the partner company’s CEO intended to use his own distribution company to exclusively channel products into a lucrative
sales territory. After the energy company escalated its concerns, its partner moved ahead with the venture anyway but did not use the CEO’s distribution company. How the best acquirers excel at integration Read the article Standardize processes and learning mechanisms Unlike dedicated M&A teams that develop negotiating skills over multiple deals, JV teams tend to change from deal to deal, often due to shifting team-member roles and responsibilities or low JV deal flow. That creates little institutional memory around key processes, approaches for managing critical issues, and even partnership-specific negotiating skills. All of these things can be proactively managed, even if deal terms cannot. Yet our survey of JV practitioners found that less than a quarter have a JV design-and implementation playbook—the kind of simple tool that most companies with M&A programs have had for years to reduce strain for the internal team
and to ease discussions with potential partners. Without that kind of institutional knowledge, inexperienced teams often see JV negotiations as zero-sum games; they rigidly calculate wins and losses on every negotiating point. That leaves them with little flexibility to appreciate the needs of a partner interested in entering into a commercial agreement or reaching consensus on the terms of a mutually beneficial JV. The result can be a weak or ineffective deal. For example, one global company faced challenges investing in a regional JV because it focused too emphatically on legalistic deal terms to protect its own interests. That created an adversarial tone in the negotiations and undermined the collaboration needed to allow both companies and the JV to succeed. It also prolonged the process, to the frustration of the JV partners. For most JVs, long-term success also requires an agreement process that is transparent and follows patterns of conversation established from the outset. At its core, this simply means communicating with all parties about how, when, and what to communicate. The eventual pattern of communication may vary from deal to deal, and not all parties will like it. That’s OK. Just laying it out keeps expectations aligned, focuses conversations, and reduces time-consuming delays. Otherwise, internal approval processes can cause bottlenecks, and not having the right people in the room can bring momentum to a standstill. Standardized processes are especially helpful once a deal is under way, when adapting and restructuring can strengthen a partnership and increase financial returns—as long as the relationship is strong and the process has clearly allowed for adaptation. One aerospace partnership ensured all parties continued to agree on the goals of the JV by contractually committing to a standardized annual evaluation process. This included valuing each partner’s contributions to ensure that the risks and rewards for each partner remain consistent with the original objectives of the deal. In the event that one partner’s contributions did not match the other’s, the terms of the agreement required the lagging partner to increase its contribution. Together with a management team in which the CEO position is swapped on a regular basis, both partners have been able to maintain a decades-long relationship. With so many companies planning to increase their JV activity in coming years, it’s worth investing the time in negotiations and planning to ensure the value of these ventures. About the author(s) Eileen Kelly Rinaudo is a senior expert in McKinsey’s New York office, where Jason Roswig is a consultant. Explore a career with us Search Openings Related Articles Article Joint ventures on the rise Article Avoiding blind spots in your next joint venture Survey How M&A practitioners enable their success Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. An opportunity to reorder the healthcare industry | McKinsey Skip to main content The gathering storm: An opportunity to reorder the healthcare industry September 23, 2022 | Article By Daniel Brown, Addie Fleron, Shubham Singhal , and Drew Ungerman LinkedIn Twitter Facebook Email Article (9 pages) The gathering storm: An opportunity to reorder the healthcare industry LinkedIn Twitter Facebook Email Article (9 pages) Leaders will redesign their organizations for speed, accelerate productivity improvements, reshape their portfolio and innovate new business models, and reallocate constrained resources. DOWNLOADS Article (9 pages) The once-in-a-century pandemic thrust the healthcare industry into the teeth of the storm. The combination of accelerating affordability challenges, access issues exacerbated by clinical staff shortages and COVID-19, and limited population-wide progress on outcomes is ominous. This gathering storm has the potential to reorder the healthcare industry and put nearly half of the profit pools at risk. Those who thrive will tap into the $1 trillion of known improvement opportunities by redesigning their organizations for speed, accelerating productivity improvements, reshaping their portfolio, innovating new business models to refashion care, and reallocating constrained resources. The healthcare industry has lagged behind other industries in applying these practices; players that are able to do so in this crisis could set themselves up for success in the coming years. Fate whispers to the warrior, ‘You cannot withstand the storm.’ The warrior whispers back, ‘I am the storm.’ Source unknown Reorder or be reordered. That is the rallying cry for industry leaders. The healthcare industry faces an acceleration in costs of nearly $600 billion in 2027, which could make healthcare less affordable and threaten the sustainability of industry margins. However, a path to weather the storm exists—the staggering $1 trillion opportunity to create value and improve healthcare by transforming the delivery of care, improving clinical productivity, applying technology, and simplifying administrative procedures. 1 1. This opportunity does not reflect any changes to baseline trends in the social determinants of health or personal health-promoting behavior. Any improvements in these areas would be incremental. What’s more, this level of opportunity is based on innovations already in use and available to executives today. The imperative for companies that seek to thrive in coming years will be scaling up these innovations much more quickly than they currently do. Our research from the last recession (2007–09) shows that challenging times for an industry can create a significant separation between resilient companies and others, meaningfully reordering the sector (Exhibit 1). The “resilients”—the top-quintile performers in each sector—did better at the outset of the downturn and widened the performance gap in subsequent years, delivering a cumulative total-return-to-shareholders lead of more than 150 percentage points, compared with the non-resilients, by 2017. 2 2. Martin Hirt, Kevin Laczkowski, and Mihir Mysore, “ Bubbles pop, downturns stop ,” McKinsey, May 21, 2019. For non-resilients, this lead was tough to reverse: nearly 70 percent of resilients remained top-quintile performers at the end of the decade, with few non-resilients joining them. 3 3. “ Bubbles pop, downturns stop ,” May 21, 2019. This analysis is based on the returns of publicly traded companies, which are used as a proxy for performance across a downturn. Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com The coming years will provide a test for leaders of incumbents and disruptors across the healthcare industry. If history serves as a guide, companies that rise to the occasion will probably be rewarded with sustained overperformance, creating strategic distance from their competitors and establishing themselves as recognized leaders in improving healthcare. In the healthcare sector, resilients are likely to be organizations that deploy four actions faster and more effectively than their peers. Action 1: Redesign for speed The pandemic forced executives in all sectors to make extensive changes to increase the flexibility and speed of decision making in their organizations. Nowhere were these changes more critical than in healthcare, which served as the nation’s front line of defense against COVID-19. As society transitions toward managing COVID-19 as an endemic disease, healthcare leaders might take this opportunity to identify which changes from the past two years are working and which have outlived their usefulness. Of particular importance is for leaders to be even more bold to sustain the gains made during the pandemic and further increase the speed of decision making and execution. Our research shows that a concerted effort to become faster pays off (Exhibit 2). Fast organizations outperform others by a wide margin on a range of outcomes, including profitability, operational resilience, organizational health, and growth. 4 4. Aaron De Smet, Elizabeth Mygatt, Iyad Sheikh, and Brooke Weddle, “ The need for speed in the post-COVID-19 era—and how to achieve it ,” McKinsey, September 9, 2020. Often, these advantages are even more significant in difficult times, when decisions have more important consequences, and it is necessary to outpace competitors at every phase of the process. That kind of speed will be essential during the next few years. The devastating effects of cost increases eroding affordability and margins require scaling-up speeds (for example, executing a six-fold increase in the number of patients in risk-bearing, value-based care arrangements in the next five years) unlike those the industry has ever attempted. Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com But increasing an organization’s speed isn’t easy. Even most executives at sector-leading operators can identify at least a few hurdles that prevent them from moving even faster. Organizational silos, unclear strategies, and slow processes frequently interfere with attempts to make decisions and get work done more quickly. To increase speed and improve resilience, leaders could start by asking these questions: Identify the bottlenecks. What are the top one to three bottlenecks slowing down the organization? What has prevented the organization from addressing them, and what can you do to remove them now? Enhance the operating model. Are roles and responsibilities for key processes identified clearly? Have unnecessary stage gates that stymie decision making been removed? Actively monitor potential risks. Have trigger points been set to enable immediate action when thresholds for key decisions are crossed? Action 2: Double down on productivity Increasing efficiency is a perennial objective of healthcare executives, but for most organizations, incremental improvements probably cannot meet the moment. To survive the coming storm, leaders must shift their mindset and adopt much bolder aspirations to raise productivity. Of course, bold actions must be thoughtfully prioritized to improve not only the costs but also the quality of care, access, and the patients’ experience. Our research shows that challenging times can create opportunities for step changes in productivity and that, as we have noted, companies making faster and bolder moves during downturns perform better over the long run (Exhibit 3). In the last recession, the resilients heeded the warning signs earlier, cutting operating costs as a percentage of revenue by an average of 50 basis points at the start of the downturn and by 350 basis points through the recovery. These moves gave resilients a substantial head start over the non-resilients, where costs rose during the downturn and fell only slightly in the recovery. 5 5. “ Bubbles pop, downturns stop ,” May 21, 2019. Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Simply put, transformations are vital in this environment. Providers The need for improvement is particularly urgent for providers, many of which face a one- to two-year lag between the elevated costs they incur today and the opportunity to negotiate higher reimbursement rates from payers. On top of this, providers are experiencing poor year-to-date performance and a souring outlook for investment income, which has served as a significant buoy for health system finances over the past decade. To manage through this difficult period, providers have an opportunity to reimagine how things get done in back-office functions and the delivery of clinical care and to prioritize the most critical work. Providers can pursue transformations that address a comprehensive set of value levers, including clinical operations, external spending, and general and administrative expenses. They might also use current labor shortages and increased competition for talent as an opportunity to reduce the amount of labor needed to get work done: for example, to help teams work at top of license, providers could roll out innovative technologies (including automation) and use tried-and-true playbooks for staffing, scheduling, and redesigning models of care. Successful initiatives could improve not only financial performance but also the patient and caregiver experience, while increasing access and quality. Payers To avoid unsustainable price increases for purchasers, payers too will need to accelerate their efforts to cut medical and administrative costs. In particular, payers could play a leadership role in transforming the delivery of care—for example, by significantly increasing the number of value-based care programs, enabling and scaling up new care models in lower-acuity and more convenient sites of care (including homes and virtual care), and reimagining care pathways to make them more effective and efficient. At the same time, payers should also improve their productivity—for example, by redesigning internal processes, deploying new technology (including automation), and focusing on performance management. Beyond their own four walls, payers should seek ways to help reduce the 25 percent of national health expenditures spent on administrative expenses. Known interventions could lower the level to 18 percent, a goal that payers are well placed to pursue in their role as orchestrators of the healthcare system. Most institutions have undertaken performance improvement journeys, but few successfully assemble all the elements needed to ensure lasting change. In fact, our research finds that 70 percent of performance transformations fail. However, the odds of success increase dramatically—raising the success rate to nearly 80 percent—when organizations apply a proven playbook. 6 6. Michael Bucy, Bill Schaninger, Kate VanAkin, and Brooke Weddle, “ Losing from day one: Why even successful transformations fall short ,” McKinsey, December 7, 2021. Action 3: Adopt a growth mindset In downturns, resilient companies seize the opportunity to distance themselves from competitors through investments in strategic growth. 7 7. Neha Patel and Shubham Singhal, “ The future of US healthcare: What’s next for the industry post-COVID-19 ,” McKinsey, July 19, 2022. This strategy is doubly necessary in healthcare, where profit pools are rapidly shifting—in many cases, away from the core segments of the sector’s incumbents. Unfortunately for those incumbents, the competition for attractive profit pools is fierce. Increasingly well-capitalized disruptors, for example, are rapidly deploying new business models to capture the fastest-growing and highest-margin segments. These disruptors are often unburdened by legacy constraints, bureaucracy, or the need to manage a separate core business. Some are backed by substantial venture-capital and private-equity funding. As a result, disruptors often get to market faster than incumbents accustomed to incrementally evolving business models. But incumbents do not have to cede these opportunities. Although disruptors may have speed, incumbents may have their own natural advantages. These include existing relationships and the trust of patients and members, the ability to quickly scale up what works across markets, time-honed operational discipline at scale, and, in some cases, opportunities for diversified growth to strengthen the core business as well. 8 8. Annie Kurdziel, Prashanth Reddy, and Lara Sanfilippo, “ Transformative healthcare growth through diversification ,” McKinsey, August 9, 2019. Several incumbents have already begun to see success through diversification. Large payers, for instance, have created diversified business units operating in care delivery, data and analytics, pharmacy services, care and utilization management, and other technology and service businesses. Some providers have created very profitable units to manage revenue cycles and enable value-based care. Our analysis has shown that diversified healthcare companies can deliver higher returns. Although this analysis is based on the returns of publicly traded companies, in our experience private not-for-profit healthcare organizations also often see financial benefits from diversification (Exhibit 4). Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Where exactly might the sector’s leaders look for growth? For providers, profit pools are continuing to shift away from core acute-care segments and facilities 9 9. For example, the EBITDAs of general acute-care hospitals are expected to increase at a CAGR of less than 5 percent through 2025. The EBITDAs of skilled nursing facilities, inpatient rehabilitation facilities, and long-term acute-care hospitals are expected to decline over the same period. and toward higher-value, lower-cost ambulatory, virtual, and home sites for care. These changes are driven by the preferences of patients and physicians and by sectoral shifts, including the increased adoption of value-based care models. Many health systems are already far along in developing a robust ambulatory footprint. Still, in these spaces it will probably be important to focus on buying, building, or partnering with innovators—for example, primary-care disruptors, risk-bearing management service organizations (MSOs), and virtual-care companies (Exhibit 5). Exhibit 5 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Payers face a similar imperative to diversify and reinvent their business models. Profit pools are shifting away from individual, small-group, and administrative-services-only insurance plans and toward government segments: the Medicaid, Medicare Advantage, and Medicare supplements segments are forecast to grow at a CAGR of more than 10 percent through 2025. Moreover, the most successful managed-care models increasingly center on value-based–care arrangements with meaningful risk, the orchestration (and sometimes ownership) of nonacute care (for example, through risk-bearing MSOs), the integration of pharmacy services, and member engagement across the care journey. These models are improving costs, the quality of care, and the experience of members while expanding the total profit pool for payers. Our research has found that payers investing in such next-generation managed-care models achieve higher financial returns than their peers by delivering better value to the healthcare system. 10 10. Emily Clark, Jennifer Rost, and Anna Stolyarova, “ Innovation and value: What payer-led managed-care models may look like ,” McKinsey, December 2, 2021. To succeed in achieving diversified growth, organizations need a set of competencies that may be new—and different from those required in the day-to-day business. These new competencies include programmatic M&A, partnerships, effective integration, and rapid business building. Programmatic M&A and partnerships. Two decades of McKinsey research across sectors shows that programmatic M&A is more likely than large deals to deliver excess TSR at lower levels of risk. 11 11. “ How one approach to M&A is more likely to create value than all others ,” McKinsey, October 13, 2021. In healthcare, most innovative companies are small or midsize, and programmatic M&A is often the optimal approach for diversifying efficiently or for acquiring the capabilities needed to reinvent business models. Many of these assets are in high demand and command high multiples that may place them out of reach for all but the largest players. But even when an acquisition isn’t feasible, a range of partnership models could be structured to achieve as much value as M&A. These models typically require at least as much attention to derive value and must avoid a range of common pitfalls that derail negotiations and the long-term capture of value. 12 12. “ Overcoming the cost of healthcare transformation through partnerships ,” McKinsey, August 11, 2022. Effective integration. Although the transformation playbook is essential to unlock value in the ordinary course of business, it is particularly salient during M&A integrations, to ensure that the anticipated value of the deal thesis is realized. 13 13. “ Unlocking value in large-deal healthcare M&A: How to achieve transformation during integration ,” McKinsey, September 14, 2021. Further, programmatic acquirers need to develop an integration competency that is “always on” and makes it possible to scale up and integrate an acquired company’s core operations quickly. Rapid business building. Across sectors, business building is a top priority for growth, and healthcare is no exception. 14 14. Ari Libarikian and Shubham Singhal, “ Leap to the future of healthcare: Reinvent through business building ,” McKinsey, April 9, 2021; Shaun Collins, Ralf Dreischmeier, Ari Libarikian, and Upasana Unni, “ Why business building is the new priority for growth ,” McKinsey, December 10, 2020. Developing a strong business-building capability may prove essential if attractive acquisition targets or potential partners do not exist or are not economical and the skills and experiences required to rapidly scale a new enterprise are quite different from the typical capabilities of healthcare incumbents. Business building is also essential when large incumbents buy innovative small to midsize companies but need to enlarge them many times over to handle the scale of the incumbents’ volume. However, the skills and experience required to scale a new enterprise rapidly are quite different from those typical of healthcare incumbents. Action 4: Reallocate constrained resources Our research shows that across economic contexts, companies that actively reallocate resources outperform those that don’t. 15 15. Massimo Garbuio, Tim Koller, and Zane Williams, “ Admit it, your investments are stuck in neutral ,” McKinsey, July 10, 2019. In challenging times, such a reallocation is more important than ever. Many organizations struggle to reallocate at the necessary pace. Successful reallocators follow a tested portfolio of processes that aim to seed high-growth areas with the resources necessary to succeed, while avoiding retrenchment in the core business. Maintain absolute clarity on the objectives for capital allocation. The most successful reallocators take a clean-sheet, not incremental, approach to allocating strategic (as opposed to maintenance) capital. They focus on identifying the minimum allocation of maintenance capital to sustain the core business. Take a dynamic approach to budgeting. To act quickly when markets shift or new opportunities arise, organizations should see budgets as rolling, not fixed. Remove budget anchors to avoid rubber-stamping the same allocations every year. And have clear ground rules for early termination to stop underperforming projects. Align talent to value. Ensure that the best talent focuses on the most important growth areas. To sum up, four sets of actions will help the healthcare sector weather the storm and outperform through the recovery. First, assess the speed of your organization to ensure that you can make hard decisions faster than your peers do. Second, launch (or recommit your organization to) a bold performance transformation that protects the core business, create business optionality on the balance sheet, and prepare to reinvest in growth. Third, do invest in growth proactively through programmatic M&A and partnerships, effective integrations, and rapid business building. Put a particular focus on diversification and innovative business models aligned with the $1 trillion opportunity. Finally, reallocate organizational resources to realize the value creation agenda—for example, by taking a clean-sheet approach to capital and budget allocations. The opportunities are known and the approaches we describe are proven. The imperative is strong leadership. Healthcare leaders must set clear priorities, adopt proven approaches for the necessary transformation, use M&A, build new businesses, and inspire action. About the author(s) Daniel Brown is a consultant in McKinsey’s New Jersey office, Addie Fleron is an associate partner in the Chicago office, Shubham Singhal is a senior partner in the Detroit office, and Drew Ungerman is a senior partner in the Dallas office. Explore a career with us Search Openings Related Articles Article The gathering storm: The uncertain future of US healthcare Article The gathering storm: The transformative impact of inflation on the healthcare sector Article The gathering storm: The affordability challenge of endemic COVID-19 Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Rural economic development strategies for local leaders | McKinsey Skip to main content Rural rising: Economic development strategies for America’s heartland March 30, 2022 | Article By Mike Kerlin , Neil O’Farrell, Rachel Riley, and Rachel Schaff LinkedIn Twitter Facebook Email Article (10 pages) Rural rising: Economic development strategies for America’s heartland LinkedIn Twitter Facebook Email Article (10 pages) There is no one-size-fits-all economic development strategy for rural communities. How can local leaders—including governments, businesses, and individuals—put rural regions on track to thrive? DOWNLOADS Article (10 pages) In downtown Clarksdale, Mississippi, in a repurposed freight depot built in 1918 for the Yazoo and Mississippi Valley Railroad, sits the Delta Blues Museum. The state’s oldest music museum, it is central to the growing tourism industry in the Mississippi Delta, “the land where the blues began”—once home to John Lee Hooker and Muddy Waters. Yet on March 18, 2020, as the COVID-19 crisis escalated across the United States, the museum was forced to temporarily close its doors. Tourism across the country slowed to a trickle, and Clarksdale’s Coahoma County—85 miles from Memphis, 77 percent Black, and with 35 percent of its population living in poverty as of 2019—suddenly lost one of its main sources of income and employment. 1 1. “S1701: Poverty status in the past 12 months,” American Community Survey, US Census Bureau, 2019. By April 2020, the county’s unemployment rate had reached about 20 percent. 2 2. “Unemployment rate in Coahoma County, MS,” retrieved from Federal Reserve Economic Data (FRED), Federal Reserve Bank of St. Louis,
    March 8, 2022. Meanwhile, about 1,000 miles northwest, in rural Chase County, Nebraska, the unemployment rate in April 2020 was only 2.2 percent. Businesses struggled to fill positions and attract workers; the poverty rate in Chase County was lower than the US average and remains so today. 3 3. “Unemployment rate in Coahoma County, MS,” retrieved from Federal Reserve Economic Data (FRED), Federal Reserve Bank of St. Louis, March 8, 2022. As these stories show, rural America is not one geographical unit but a mosaic of different landscapes, people, and economic realities. 4 4. America at work: A national mosaic and roadmap for tomorrow , Walmart, February 2019. It includes agricultural powerhouses, postindustrial towns, and popular tourism enclaves. Some rural communities are relatively close to major cities, while others are hundreds of miles from the nearest urban hub. Some have thriving workforces and a handful of economic anchors, while others face declining populations and some of the lowest living standards in the country. Some benefit from endowments such as energy resources and beautiful landscapes, while others have few natural amenities. Rural America is not one geographical unit but a mosaic of different landscapes, people, and economic realities. Below, we examine the types of rural communities in the United States and suggest that attention to three foundational elements—sectors, workforce, and community and connectivity—can promote economic success. We then outline a data-driven approach to economic development that can be tailored to meet the needs of different communities and share examples of initiatives that have led to positive outcomes in rural communities throughout North America. Tracking growth across rural America’s five community archetypes In collaboration with Walmart, we’ve identified five archetypes of rural American communities (Exhibit 1). 5 5. America at work: A national mosaic and roadmap for tomorrow , Walmart, February 2019. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes Americana. The largest rural community archetype, comprising 879 counties and 40 million Americans, Americana counties have slightly lower GDP and educational outcomes than urban areas. They are relatively close to major cities and often include several major employers. Distressed Americana. Distressed Americana communities comprise 18 million people living in 973 counties (many in the South) facing high levels of poverty, low labor force participation, and low educational attainment. Historically, these communities have been hubs for agriculture, extractive industries, and manufacturing. Their decline has mirrored the struggles in these sectors. Rural Service Hubs. Rural Service Hubs are so named because the areas (often close to highways or railways) are home to manufacturing and service industries. Because these hub communities typically serve surrounding counties that are more rural, they tend to specialize in industries such as retail and healthcare. Great Escapes. Great Escapes are the smallest but most well off of the rural archetypes, home to wealthy enclaves and tourist destinations. They comprise 14 counties and 300,000 people. While the focus on tourism in Great Escapes communities results in many low-paying service jobs, their GDP, household income, and educational attainment outpace their rural peers. Resource-Rich Regions. This category comprises 177 counties that are home to almost one million people. As the name suggests, these communities are defined by economic reliance on oil and gas or mining, often alongside high rates of agricultural production. Due in part to the value of the resources, household income, GDP per capita, and educational attainment in Resource-Rich Regions tend to be higher than average. Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Over the past ten years, the populations of all archetypes except for Distressed Americana have grown (Exhibit 2). Resource-Rich Regions in places such as West Texas and North Dakota have seen some of the fastest growth. For example, since 2010, the populations of McKenzie County, North Dakota, and Loving County, Texas, have grown by 134 percent and 104 percent respectively, while median household incomes have increased by nearly half in nominal terms. 6 6. Data Buffet, Moody’s Analytics. Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Yet while the population of Loving County soared, Concho County, Texas, another Resource-Rich Region, witnessed a 33 percent decline in population over the past decade. Approximately two-thirds of Resource-Rich Region counties faced similar, though often less precipitous, declines. 7 7. Data Buffet, Moody’s Analytics. Counties where residents typically have access to world-class natural amenities, which are often among the Great Escapes, have been among the most uniformly successful since 2010. The appropriately named Summit County, Colorado, is home to one of the greatest concentrations of ski resorts in the world, featuring Breckenridge, Copper Mountain, Keystone, and Arapahoe Basin. Over the past decade, the county’s population has grown by 11 percent and median household income has increased by 54 percent. 8 8. Data Buffet, Moody’s Analytics. Gallatin County, Montana, home to Bozeman, is a Rural Service Hub, though it also features the world-class natural amenities common to Great Escapes. It contains Big Sky Resort and is one of the gateways to Yellowstone National Park. The county, particularly the city of Bozeman, has seen a significant influx of remote workers during the pandemic, which may have contributed to a jump in housing prices of more than one-third since the beginning of 2020. 9 9. “Gallatin County home values,” Zillow, updated on January 31, 2022. Meanwhile, Pender County, an Americana region on the southern coast of North Carolina, achieved 22 percent population growth from 2010 to 2020 while positioning itself as a logistics hub. Pender Commerce Park, a 450-acre industrial center developed as part of a partnership between Pender County and Wilmington Business Development, attracted FedEx Freight in 2018. 10 10. “FedEx Freight coming to Pender Commerce Park,” Pender County, North Carolina, February 5, 2018. Rural counties’ wide range of economic performance over the past decade reinforces that there is no one-size-fits-all playbook for growth. Instead, we have identified some of the fundamental characteristics that thriving counties tend to share, even as the appearance or impact of the characteristics varies from place to place. Elements of a thriving rural community Rural communities require three interconnected, baseline elements to thrive: sectors, workforce, and community and connectivity (Exhibit 3). Rural economic development initiatives typically tie into one or more of these key elements. Sectors. Sectors refer to stable or growing tradable industries that bring wealth into communities, create employment opportunities, and carry strong multiplier effects that support the overall economy. Thriving rural communities play to their region’s strengths, supporting sectors such as agriculture, manufacturing, energy, tourism, and postsecondary education. Workforce. People are the lifeblood of any community. A healthy, skilled workforce is the most important factor in attracting and retaining employers in key sectors. 11 11. Ron Starner, “More than some like it hot,” Site Selection , January 2018. In addition, workers spread wealth and create additional jobs by buying goods and services within their communities. Community and connectivity. The most intangible element, community and connectivity includes services and amenities critical to quality of life, such as transportation infrastructure and access to broadband, healthcare, childcare, and arts and culture. Because these assets support the workforce, they are essential to developing thriving sectors. Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com While thriving communities are succeeding across all three elements, more narrow or focused efforts can still catalyze economic growth. For example, even if job creation is low or GDP growth has plateaued in a community, improving residents’ quality of life can slow outmigration and attract thriving sectors in the future. Creating an economic development strategy Creating an economic development strategy for a rural area is similar to doing so in other places. It is a multistep process that requires assessing the current state of the region, identifying the value proposition, evaluating existing programs, and establishing partnerships and rural hubs. When those steps have been taken, communities will be in a position to prioritize specific initiatives. Assess the current state of the region Before engaging in an economic development strategy, it is important to understand the current state of a region, its competitive position, and its strengths and challenges. This requires using quantitative data from sources such as the US Census Bureau, the Bureau of Labor Statistics, as well as the National Center for Education Statistics and qualitative data from sources such as stakeholder interviews to assess regional performance across a variety of metrics. The framework in Exhibit 3 provides a starting point. Regions can be assessed by sector, including factors such as employment rate, GDP, specialization, and growth by industry; workforce, including a demographic breakdown, employment by occupation, and educational attainment; and community and connectivity, including factors such as transportation infrastructure and access to broadband, childcare, and healthcare. With these data, policy makers can understand their region’s strengths and challenges relative to other regions and begin to focus on assets, or competitive advantage, and potential barriers to development. Identify the value proposition After the diagnostic phase has resulted in a picture of a region’s strengths and challenges, the next step is formulating the value proposition, which is part of a strengths-based approach to economic development. The value proposition is about creating a regional story line that answers questions such as: Why would someone live here? Why would a company locate here? Why would someone visit? The value proposition can take many forms across sectors, the workforce, and community and connectivity. For instance, it may be a high-performing local talent pool, a knack for retaining and growing local businesses, an ability to build partnerships to attract investment, or distinctive industry clusters. The value proposition for residents might include a strong local community, a high quality of life, or access to natural amenities. The most effective economic development strategies leverage and develop a region’s strengths and reinforce its value proposition. Evaluate existing programs and initiatives Any one region can be affected by multiple programs and initiatives, including those from federal, state, and local governments and from groups such as chambers of commerce and business improvement districts. Policy makers may want to take stock of existing programs before developing new initiatives to avoid reinventing the wheel. Key questions to ask include: What does this program cover? What are its strengths and weaknesses? Can it be improved? Is it possible to increase engagement? Successful economic development strategies often leverage existing efforts or improve them incrementally by updating programs or increasing participation. An analysis of a region’s current programs also reveals genuine gaps that can be addressed with new initiatives. Establish partnerships and rural hubs Rural regions often include multiple stakeholders, such as governments, nonprofits, and educational institutions, that have a—sometimes overlapping—hand in the three foundational elements of economic development noted above. In addition, multiple communities within a broader region may have shared economic needs. Partnerships in rural areas can therefore allow communities to direct limited resources and expertise to shared initiatives. When regions and institutions band together, they create economies of scale, also called rural hubs. Designing rural economic development initiatives Rural regions are not monoliths, so rural economic development strategies will vary. The approach outlined above will help leaders identify their region’s unique strengths, challenges, and assets that can be formed into cohesive value propositions. That said, many broad economic development initiatives can be tailored to meet the needs of different regions. Below is a nonexhaustive list of initiatives that may apply to rural regions, based on their specific assets and needs. Launch ‘big push’ investment The idea of the “big push” is to funnel a significant amount of investment into a particular area of need to create a sustainable, long-term, virtuous cycle of economic growth. This can take many forms but is most frequently associated with the attraction of a major employer or the construction of large-scale infrastructure. Due to its size, big-push investment usually requires involvement and funding from an overarching government body, such as the state or federal government. An example of big-push investment in electric vehicles can be found in Tennessee. The state has offered Ford Motors and its partner, South Korea–based SK Innovation, hundreds of millions of dollars in incentives to develop BlueOval City, a site for the production of electric pickup trucks and advanced batteries. Leaders expect the project to create nearly 6,000 jobs in Stanton, Tennessee, a town within Distressed Americana Haywood County. 12 12. Morgan Watkins, “Here are the incentives Kentucky and Tennessee used to lure Ford’s new factories,” Louisville Courier Journal , October 11, 2021; “Ford to lead America’s shift to electric vehicles with new mega campus in Tennessee and twin battery plants in Kentucky; $11.4B investment to create 11,000 new jobs and power new lineup of advanced EVs,” Ford Motor Company, September 27, 2021. Embrace placemaking Residents want to live in communities that are safe, interesting, and attractive. Placemaking means creating those environments. It is, by one definition, “the process of creating quality places that people want to live, work, play, and learn in.” 13 13. Robert Steuteville, “Four types of placemaking,” Better Cities & Towns, October 2014. Funding for placemaking efforts can come from a variety of sources, including private groups and local, state, or federal governments. For instance, the US Department of Agriculture’s Rural Placemaking Innovation Challenge made available $3 million (with a maximum grant of $250,000) to rural areas for technical assistance related to placemaking. 14 14. “Rural Placemaking Innovation Challenge,” US Department of Agriculture, updated on July 26, 2021. Like their urban counterparts, many rural cities and towns have seen success in creating business improvement districts (BIDs), small-scale economic development organizations often funded by local stakeholders, such as businesses. BIDs deliver services in a particular area, often at the neighborhood or “Main Street” level. The services might include street cleaning, public safety, beautification, or events. One example of placemaking comes from Douglas, Georgia, a city of roughly 12,000 people about 115 miles northwest of Jacksonville, Florida. It is the county seat of Coffee County, characterized as Distressed Americana. 15 15. How small towns and cities can use local assets to rebuild their economies: Lessons from successful places , US Environmental Protection Agency, May 2015. In the late 1980s, Douglas faced downtown vacancy rates of about 25 percent. In response, the city restored historic facades (funded by an initial $10,000 from the city and the local industrial development authority), replaced sidewalks, and added lighting. Today, Douglas’s downtown is a community gathering place where festivals take place regularly. Walking paths connect the downtown area to local community colleges. By 2012, the vacancy rate had dropped to 6 percent, thanks in part to 12 newly opened businesses downtown. Surrounding areas of the county gained nearly 8,000 new residents between 1990 and 2000, increasing the county’s growth rate to 26 percent—up from 11 percent between 1980 and 1990. 16 16. US Census Bureau, “Resident population in Coffee County, GA,” retrieved from Federal Reserve Economic Data (FRED), Federal Reserve Bank of St. Louis, updated on May 5, 2021. Develop tourism infrastructure Tourism is technically an export sector—it draws in spending from outside the region to generate economic growth. Some rural communities can leverage existing assets, such as state or national parks, to capture tourism value. Others can use their environment or location to create a reason for tourists to visit. Crosby, Minnesota, is 125 miles north of Minneapolis and home to just shy of 3,000 people. The town was a hub for iron ore mining until the industry collapsed a half-century ago. In the 1980s, leaders from Cuyuna Range Economic Development Inc., a regional economic development organization, and other stakeholders petitioned the state to create a recreation area on the former mine sites and surrounding land. 17 17. Dan Kraker, “From mining to biking: How Minnesota’s Cuyuna Range became an off-road cycling destination,” Minnesota Public Radio,
    October 7, 2016. The Cuyuna Country State Recreation Area was officially established in 1993. In 2011, it became the state’s first mountain bike park, featuring 25 miles of trails. Since 2011, 15 new businesses—including a brewery, a yoga studio, and a farm-to-table restaurant—have opened in Crosby, largely serving the tens of thousands of annual visitors to the trail system. Once the trails reach completion at 75 miles, leaders anticipate a local economic impact of $21 million. 18 18. Nicholas Hunt, “How biking is saving small-town USA,” Outside , May 15, 2017. Attract, retain, and expand small and medium-size businesses Sixty-five percent of workers in nonmetropolitan areas are employed by small and medium-size enterprises (SMEs), a higher share than in the nation as a whole. 19 19. Hanna Love and Mike Powe, “Rural small businesses need local solutions to survive,” Brookings Institution, December 1, 2020. These companies provide benefits to communities beyond direct employment. SMEs generate local wealth, because profits go to the owner, who is more likely to live and spend locally than shareholders of a large corporation. 20 20. Devra Gartenstein, “Reasons why small businesses are important,” Houston Chronicle , January 28, 2019. Rural communities with a relatively high share of SMEs also tend to have better health outcomes. 21 21. Troy Blanchard, Carson Mencken, and Charles Tolbert, “The health and wealth of US counties: How the small business environment impacts alternative measures of development,” Cambridge Journal of Regions, Economy and Society , March 2012, Volume 5, Number 1. While attracting new businesses generates local excitement, expanding and retaining SMEs has a higher return on investment, in part because existing businesses do not require tax incentives to move to the area. Local businesses are stalwarts of the community, paying economic dividends through local taxes and job creation. Leaders can initiate a business retention and expansion strategy by collecting qualitative data from interviews with entrepreneurs about expansion plans and business challenges. In addition, economic development practitioners can support SMEs by helping them access loans and grants, navigate regulatory requirements, write business plans, plan for succession, and connect to large companies doing procurement in the region. Local businesses are stalwarts of the community, paying economic dividends through local taxes and job creation. Northern Development Initiative Trust, an economic development agency representing rural Northern British Columbia, has made SME support a core part of its mission. The agency offers programs such as the Competitiveness Consulting Rebate, which helps SMEs recover up to 50 percent of costs (to a yearly maximum of CA $30,000 [US $23,400]) 22 22. Converted on March 7, 2022; “Competitiveness Consulting Rebate,” Northern Development Initiative Trust, accessed December 15, 2021. for external business consulting on services such as website creation, marketing strategy, and workplace health and safety plans. Thus far, the program has allotted more than CA $5.5 million (US $4.3 million) to SMEs across 552 projects. A complementary program, the Northern Industries Innovation Fund, supports regional SMEs’ innovation projects, such as providing funds to develop a technical process to adapt forestry equipment for drilling. 23 23. “Northern Industries Innovation Fund,” Northern Development Initiative Trust, accessed December 15, 2021. Attract remote workers Many companies have adopted long-term remote work strategies prompted by the COVID-19 pandemic and employees’ desire for flexibility. These strategies primarily affect knowledge workers, who tend to be highly educated and high earning. Historically, remote workers have flocked to smaller communities with natural amenities. A 2018 Gallup poll showed that 27 percent of Americans would prefer to live in a rural area. 24 24. Frank Newport, “Americans big on idea of living in the country,” Gallup News, December 7, 2018. This presents an opportunity for rural communities to make strategic investments to attract remote workers. Some investments, such as strong broadband infrastructure, may be a requirement. Others, such as assets related to quality of life or outdoor recreation, may carry varying weight depending on the region. Some regions offer direct financial incentives to remote workers who relocate. For example, the Shoals region includes Lauderdale and Colbert Counties in rural Northwest Alabama. Remote Shoals, a partnership launched in 2019 between the Shoals Chamber of Commerce and the Shoals Economic Development Authority, offers participants a stipend of up to $10,000 to move to the region and work remotely for at least
12 months. 25 25. “Work remotely? Get $10,000 to do your job from the Shoals,” Shoals Economic Development Authority, accessed December 15, 2021. The program received more than 200 applications from 33 states in 2019 and 500 applications in 2020. 26 26. Jared Lindzon, “Cities offer cash as they compete for new residents amid remote work boom,” Fast Company , June 22, 2020; Anna Eubanks, “Remote work possibilities draw traffic to the Shoals,” Shoals Chamber of Commerce, January 22, 2021. By March 2021, the total payroll of those in the program was $1.8 million. 27 27. DeAndria Turner, “Remote Shoals thriving in the midst of COVID-19,” WAFF 48 News, March 24, 2021. Increase access to healthcare According to the Centers for Disease Control and Prevention, significant disparities in health outcomes exist between urban and rural America, with residents of rural communities more likely to die from ailments such as heart disease and cancer. 28 28. “About Rural Health,” Centers for Disease Control and Prevention, updated on August 2, 2017. Limited access to healthcare in rural regions is one of the drivers of this imbalance. 29 29. “Rural health disparities,” Rural Health Information Hub, updated on April 22, 2019. However, many rural regions are working to address this challenge by making it easier for patients to access care—virtually or in person—and by training the next generation of rural healthcare workers. Project ECHO, based at the University of New Mexico, uses a combination of telemedicine, case-based learning, and web-based disease management tools to offer treatment for people with chronic diseases at more than 250 sites across the state. 30 30. Martha Hostetter, “Case study: Project ECHO expands access to specialty care for rural patients,” Commonwealth Fund,
    accessed December 15, 2021. The program saves many rural residents from long trips to hospitals in more urban areas to receive specialized care. In Alabama, the University of Alabama at Birmingham and Tuskegee University, in partnership with rural community clinics, train registered nurses in three identified areas of health professional shortages. The effort became particularly relevant during the COVID-19 crisis. 31 31. Jennifer Lollar, “$2.8M grant will establish primary care RN workforce,” University of Alabama at Birmingham, August 29, 2018. Rural America is indeed a mosaic. From a distance, it is often idealized but misrepresented. Upon closer examination, it reveals a diversity of colors and images. It is not one place but thousands—each community with its own identity, culture, strengths, and challenges. Some rural regions are thriving, while others have yet to fully capture their potential value. But all rural areas could benefit from an economic development plan that strengthens sectors, the workforce, and community and connectivity. About the author(s) Mike Kerlin is a partner in McKinsey’s Philadelphia office; Neil O’Farrell is a specialist in the New York office; Rachel Riley is an associate partner in the Washington, DC, office; and Rachel Schaff is a client capabilities manager in the Waltham office. Explore a career with us Search Openings Related Articles Article How state and local governments win at attracting companies Article America 2022 in charts: An economic opportunity snapshot Report The case for inclusive growth Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. US manufacturing: The next frontier for sustainable, inclusive growth | McKinsey Skip to main content US manufacturing: The next frontier for sustainable, inclusive growth April 21, 2022 | Podcast By Eric Chewning and André Dua LinkedIn Twitter Facebook Email Article (9 pages) US manufacturing: The next frontier for sustainable, inclusive growth LinkedIn Twitter Facebook Email Article (9 pages) US manufacturing has an outsize impact on the nation’s economy. How can business leaders harness innovation to revitalize the manufacturing sector and drive inclusive economic growth? DOWNLOADS Article (9 pages) In the third episode of McKinsey’s new Future of America podcast, McKinsey’s André Dua and Eric Chewning explore the surprising ways that revitalizing US manufacturing could accelerate economic equality for more Americans. An edited version of the conversation follows. 00:00 Audio US manufacturing: The next frontier for sustainable, inclusive growth André Dua: Welcome to the third episode of McKinsey’s Future of America podcast, where we’ll explore how we can build a future that drives sustainable and inclusive growth. This isn’t about trade-offs. We reject the “or” and embrace the “and.” Join us in conversation with leaders who are accelerating progress to grow, broaden, and sustain prosperity for more Americans. I’m your host for today, André Dua. I’m a senior partner with McKinsey, the managing partner of our Miami office, and a member of our McKinsey Global Institute Council. I regularly publish insights regarding inclusive growth and economic opportunity, including about McKinsey’s semiannual American Opportunity Survey, which explores Americans’ perceptions of who is and who isn’t served by our economy. Today, I’m joined by my colleague Eric Chewning, a partner in our Washington, DC, office and the coleader of McKinsey’s Aerospace & Defense Practice in the Americas. Eric has two decades of experience advising decision makers in national security, and he has deep expertise in improving US manufacturing and helping companies drive sustainable and inclusive growth. Eric, welcome, and thanks for being here today. Eric Chewning: Thanks for the invitation to join the podcast. André Dua: Of course. Let’s start by having you tell our listeners about your background, because you’ve really had a very interesting and varied career. Eric Chewning: Yes, it’s definitely allowed me to see the US manufacturing ecosystem from a couple different perspectives. To start off, I was an investment banker, where I covered industrial companies. Then I enlisted in the army after September 11th, and so I served for four years, did a deployment to Iraq in 2004–05, and then came out of the army, went to business school, and found my way into consulting, and ultimately here at the firm with McKinsey. I ended up, in 2017, going over to the Department of Defense [DOD], where I was the deputy assistant secretary for Industrial Base Policy. In that role at DoD, I managed the industrial base, as well as manufacturing portfolios, and then went on to be the chief of staff for two of our US defense secretaries. Then I came back to the firm in April of 2020, where I now colead McKinsey’s Aerospace & Defense Practice in the Americas. Subscribe to the Future of America podcast Google Podcasts Apple Podcasts Spotify Stitcher RSS André Dua: Before we jump into it, I want to ask one thing about your career, because it’s a pretty interesting move. When you came out of the army, what made you decide to go to business school at that stage? That’s quite a left turn. Eric Chewning: Yeah, in a lot of ways, I sort of did the reverse. Most folks were in the military first and then found their way into consulting or banking, and I started off in banking and then went into the army. I always knew service to our country was important, and when I had the opportunity to serve, I did. But I knew it was something that I wanted to do as part of a career, not make a career. Going to business school was the opportunity to sort of reinsert myself back into the business ecosystem and refamiliarize myself with what corporate America was like. André Dua: Right, but it really seems like the different strands of your career have come together, which is understanding financial systems, understanding the military, then business, then working at DOD, and so forth. It’s really interesting how those things have been integrated. Today, we’re here to discuss inclusive growth, and specifically we’re going to talk about sustainable, inclusive growth in the context of US manufacturing. But before we dive into that, I want to just zoom out a little bit. Let’s talk about manufacturing more broadly, because I think many of our listeners are probably paying more attention to manufacturing than they normally do, partly because they see the impact of supply chain disruptions on their own experience as consumers. There are shortages of goods, goods take longer to get to them, and so forth. Can you talk about the current state of US manufacturing and how important US manufacturing is for our economy? Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes Eric Chewning: I’d be happy to. At the headline level, US manufacturing accounts for about $2.3 trillion of GDP. It employs about 12 million people and supports hundreds of local economies across the country. Now, those headline numbers don’t really capture the outsize impact of manufacturing. Although it only accounts for about 11 percent of our GDP and 8 percent of direct employment, it drives 20 percent of our nation’s capital investment, 30 percent of our productivity growth, 60 percent of our exports, and over 70 percent of business R&D. And it also generates important spillover effects that help impact the broader economic activity in related sectors. I saw this firsthand when I was at DOD: the impact of US manufacturing but then also, candidly, the decline of US manufacturing in some ways. We’ve lost about 25 percent of manufacturing firms since 1997. That reflects closures as well as fewer manufacturing start-ups, and over that period we lost about 4.5 million manufacturing jobs. André Dua: You mentioned a lot of statistics, which, overall, I take to mean that, relative to its contribution to GDP and to direct employment, there’s actually significantly more impact from the manufacturing sector. One statistic really caught my attention, which was that it contributes 30 percent of productivity growth. That may be a little counterintuitive for folks. I wonder if you could say a little bit more about why the manufacturing sector drives so much productivity growth? Eric Chewning: Manufacturing is a critical enabler of what we would call technical innovation. So think about improvements to processes that underline how we make things. As we get more efficient in how we make certain activities, it not only creates additional jobs but it also drives overall productivity. But I think it’s important to note, despite those productivity growths, the overall decline of manufacturing in some sectors has created challenges. And you mentioned, André, in the beginning, the impact that folks have felt through COVID-19 and the microchip shortage now—if you wanted to go buy a car, for example. My wife was looking at new cars this week, and the wait times she was quoted were almost a year. When you think about it, that is something that I never experienced before. When I was at DOD, we saw this in a couple ways. There was, for example, a sole-source dependency, meaning there’s just one single supplier for a component of some munitions and missiles. We’re down to four large domestic suppliers for complex alloy castings that you would use for fixed-wing aircraft or helicopters. Some of it even as simple as our large-caliber gun barrels. Tank tubes are all manufactured in a single factory. And you see those sorts of impacts not just in the broader economy but in the supply chains that we rely on for national security. André Dua: Let’s maybe stick on that for a minute, because I think listeners are probably interested in some of the effects of that. On the one hand, we’ve got COVID-19. On the other hand, we’ve got what some people believe is a potential decoupling of the economy into one which is very US- and Europe-led and one which maybe had China at the center of it, particularly on the technology side. In addition to that, we’ve got some disruptions caused by the conflict in Ukraine. That has all sorts of implications for energy, and I think, at the same time, we do hear a lot of discussion about the supply chain and in particular the need to move from a focus on supply chain efficiency to supply chain resiliency, meaning one that can handle these sorts of disturbances and shocks. Let me ask you: What do you think all of these different trends and disruptions mean for the geographic footprint of manufacturing? Because you just described that the geographic footprint in the US has declined, from a direct-job perspective, and that there are fewer suppliers of things, but do you think we might see a resurgence of US manufacturing? Say a little bit about what’s going to happen in manufacturing in light of all this. Eric Chewning: When I think about manufacturing, it really hits at the intersection of economic and national security. And if we think about four core goals that, as we think about the future of America, I think manufacturing plays an important role. Manufacturing has historically made outsize contributions to productivity and overall economic health for the country. If we’re going to continue to drive GDP expansion, manufacturing is going to have to play an important role. Eric Chewning The first is around boosting productivity and economic growth. As we talked about, manufacturing has historically made outsize contributions to productivity and overall economic health for the country. And if we’re going to continue to drive GDP expansion, manufacturing is going to have to play an important role. The second thing, and this, André, is where you were headed: manufacturing is critically important for supporting jobs and incomes for workers and communities, particularly workers and communities not on the coasts. So while manufacturing may not provide the kind of mass employment it once did, no other sector plays the same role in supporting middle-income jobs across the country, especially outside of large cities. As you think about the opportunity with manufacturing, manufacturing makes up the largest economic employer in 500 counties across the United States, so it’s a broad-based driver for economic growth. Another important thing, I think, is the role of manufacturing enhancing overall innovation and competitiveness. Those are the positive externalities we talked about in the beginning. And then there’s this idea that you’re touching on, which is: How do we think about the role of manufacturing in driving national security and resilience? An interesting statistic there from my prior life is that about 70 percent of value-added manufacturing consumed in the United States comes from North America. And when you compare that to other large pure economies, just to pick China as an example: 90 percent of China’s value-added manufacturing comes from within their region. If you think about that, this means that the US is potentially more susceptible to global supply chain disruption than other economies. André Dua: I want to pick up on one of those, because I think it’s a nice segue. You mentioned that one of the benefits of manufacturing in the US is that it does support jobs for workers and communities that are not on the coasts, and a lot of middle-income jobs in particular. Let’s talk a little bit about the impact that US manufacturing has, or can have, on the twin topics of inclusion and sustainability. Eric Chewning: Sure. There are a lot of ways to define inclusion, and if you just pick one, let’s talk about skill-based inequality in the United States. Nine out of ten American adults have a high school degree or a GED, and about 50 percent have bachelor’s degrees or higher. It’s important to remember that you’ve got to make sure our economy is generating those high-quality jobs for folks who don’t have bachelor’s degrees. And manufacturing plays a unique role in doing that, because often manufacturing jobs are skill based, not degree based. So you can get involved in them through apprenticeship programs or licensing programs, and they create great economic opportunity for folks who may not live in the coasts. They’re in the middle of the country, but then they also have access to high-paying, good jobs without the need for a college degree. André Dua: Let me follow up a little bit. One of the questions that people have is related to the extent to which different industries are providing opportunity for people of different backgrounds. And you mentioned one, which is people of different educational backgrounds. What’s your sense of what’s going on in manufacturing as it relates to providing opportunities to people of different genders and racial backgrounds? How does manufacturing do relative to other sectors, and are there interesting things going on there that we should be aware of? Eric Chewning: I do. When you think about the opportunity in manufacturing, because there’s a general recognition that job growth in the United States, driven through manufacturing, there is the opportunity to incorporate folks from diverse backgrounds. I know individual manufacturers are very focused on that. I think, too, this idea around manufacturing creating broad-based economic growth—and, again, I think it’s important to think about that broad-based economic growth not just through a lens around gender or race but to think about it more broadly in terms of what it can do for gender, what it can do for race—but then what it can do for different educational backgrounds and geographic backgrounds as well to really get that broad-based economic growth. André Dua: One of the big issues that obviously has come to the fore again, partly because of the Russia–Ukraine conflict, are the set of issues around climate change and sustainability. Now, obviously these have been getting back on the agenda. There’s a real sense that time is running out in some ways, and there’s a lot of talk about how different industries get to net zero in terms of their contribution to emissions. How is manufacturing handling this, or how are companies that are in manufacturing handling this? How seriously are they taking this, and what opportunities or challenges do you think are presented by these sets of issues? Eric Chewning: It’ll be interesting to see what the SEC [US Securities and Exchange Commission] ultimately does around its accounting for carbon within a supply chain, because if some of those rules do go forward, you could see an impact. A shortening of supply chains in a colocation, to reshore our supply chains to address some of the carbon footprint issues, might be a natural outcome for that. You can’t bucket all manufacturers with one label. You have to think about different manufacturing archetypes, and as we research this, there are really four. There’s a set of manufacturing activities that are scale-based where competitiveness is driven by standardization and specialization for mass production. I think of a steel mill as an archetypal example there. There are others where they’re driven by learning-curve effects, processing engineering, and exponential growth through learning. A semiconductor fab would fall into that category. And the last two manufacturing archetypes are one around R&D design and one around flexibility. An R&D-based one would be a fabless semiconductor design house. It’s this flexible archetype that I think potentially has the most value for the situation you’re describing, because manufacturing competitiveness revolves around the ability to do high-value, low-volume production using flexible capacity colocated near the sources of consumption. You could imagine that smaller-batch production, colocated near a city, would really begin to fit a type of manufacturing archetype that would be supportive of the type of environment you’re describing. André Dua: Let’s look ahead into the future a little bit. Pull out your crystal ball. What’s next for US manufacturing? Where are we headed? Eric Chewning: It’s a great question, because I think we are certainly in a position now where there’s a lot of discussion, to your earlier point, around “What role might reshoring have here in the United States?” or reshoring back to North America broadly. So whether it’s the rush through Mexico, and how that might occur, as I look at it, I really put the focus on: At a national level, what are the policies we want to be able to put in place to drive manufacturing competitiveness writ large? Recognizing that, because manufacturing remains one of the main economic engines and primary employer for over 500 counties here in the United States, what do we need to do to enhance competitiveness overall, and as a result, you have “lift all boats”? A lot of that’s going to come down to our ability to drive forward Industry 4.0 technologies to further increase productivity enhancements. André Dua: Eric, I wanted to talk a little bit about economic opportunity in America. It does seem clear that the economy is recovering in a number of respects. Employment is up, wages are up, and they’re up particularly higher among lower-income Americans. GDP growth has recovered to some extent, and yet what’s interesting is that, at the same time, many Americans say they don’t feel like there is great economic opportunity available to them, their families, and in the country overall. A big piece of this is the ability to reskill and upskill to prepare yourself and your family for the future economy. Can you talk a little bit about what you’re seeing happening in reskilling and upskilling, both in manufacturing and across the economy? Eric Chewning: This idea around a human-capital investment is increasingly important, purely as you look at some of the dislocation in jobs. So you’ve got a couple things going on. You’ve got a certain segment of the population that retired during the COVID-19 pandemic. And then you’ve got the creation of these new digital jobs where clearly there’s a human-capital need for us to fill. We’ll need to get those types of skills in place. I’ll give you a really good example. A lot has been made about the need for reshoring certain semiconductor fabs back to the US. And the CHIPS Act [Creating Helpful Incentives for Producing Semiconductors for America Act] was a good example. The $52 billion investment is going through Congress, but also large chip manufacturers have announced large investments, hundreds of billions of dollars worth: Intel, TSMC, Samsung. Once we’ve sort of poured the concrete, then you get those facilities all in place, where are we going to find the engineers and the lab techs and all the workers to run those? As we’ve looked at this, you’re talking about tens of thousands of jobs where we haven’t really created a pipeline for adjusting what those needs are. I think that’s part of this reskilling challenge—it’s not just giving folks sort of a sense of optimism, which it does, but it’s beyond that. It’s also, what are we doing to make sure we’re creating human capital to fit the jobs we need to have in the future? André Dua: One of the things you and I have talked about in the past are some of the different sectors you’ve been involved in. One I’ve always found particularly interesting, because in a way it’s a microcosm of so many different issues in that sort of subindustry, and that’s shipbuilding. It’s a bit of a departure from the last question, but I would be interested to hear you describe what is going on in the shipbuilding industry in a particular geography in America, and what can we learn from that, and what are you seeing? Eric Chewning: When I was at DOD, one of the areas where I spent a lot of time was on Industrial Base around shipbuilding. In part because we’re at a point now where we’re increasing the amount of investment that the country’s making in ships—as an example, we’re going to move to producing three submarines a year rather than two. Now, that doesn’t sound like a lot, but when you think about it, it’s a 50 percent production increase in submarines. So what does that really mean? The other thing that’s neat about shipbuilding, particularly when talking about submarines, is it’s this unique combination of old-school metal bending coupled with high-tech digital engineering and really high-tension skills, like nuclear propulsion. How do you create a workforce that’s able to handle all of those things? And as we move to that production pace of three submarines a year, it’s creating a tremendous strain on the workforce and the supply chain. So one of the things that we worked on very aggressively was expansion of the necessary skills. Often you’re talking about trade skills: welders, fitters, plumbers, very, very important jobs. We talked about it before. They’re important jobs that don’t necessarily require a college degree. How are we generating that human capital, particularly in the areas where our shipyards are? Within the United States, as it comes to submarines, we’ve got two producers: the Newport News business down in Hampton Roads, Virginia, and then the General Dynamics Electric Boat facility up in Groton, Connecticut. I’ll just focus on Virginia for a minute: the Hampton Road region where the shipping yards are colocated have some of the highest areas of average poverty rates in the commonwealth. We’re in this situation where we’re going to need jobs. We’re going to need jobs that are skill-based, don’t necessarily require college degrees, and are colocated in communities where they could use the jobs. So that’s a natural-made opportunity, and folks are taking advantage of that. In this instance, Huntington Ingalls, which is the largest employer in Virginia—it’s the largest military shipbuilder—is working with the commonwealth of Virginia and DOD to try and get folks into apprentice schools and training programs for those jobs. I know Connecticut and Rhode Island are doing something similar for General Dynamics Electric Boat in Connecticut. I think that’s a really interesting example of how you get public and private actors together to really drive human-capital development for manufacturing jobs. Could’ve gone on to national security. André Dua: Yeah, I think it’s also an example of the more sophisticated conversation we need to have about postsecondary education. We tend to think very simplistically of universities, maybe community colleges, but we’re not talking about all the different segments and types of institution that are needed often for jobs that don’t require traditional university degrees. We’re also not talking about more partnerships between government, the private sector, and these educational institutions to meet specific needs. So I think there’s an opportunity for a much more sophisticated discussion. Let’s stick on the topic of defense for a minute, because there’s another big issue in the world which people are thinking about, which is inflation. The reason I want to bring these two issues together is we’re at a moment where, because of this conflict, countries—Germany being one of them—are rethinking their total commitment to defense spending. But, obviously, everyone is doing so in the context of a higher-inflation environment. I’ll be interested to hear your thoughts on what this means in terms of defense affordability and where we go from here in meeting defense needs in this kind of economic environment. Eric Chewning: It’s a great question. We just published a piece on this, looking at—the president’s ’23 budget requests to Congress as it relates to defense and the impact, the difference—inflation rates would have on overall DOD buying power . The assumptions that were in the budget requests were sort of in an inflationary environment. Whenever you do defense forecasting, they do it over what’s called the FYDP, or the Future Years Defense Program. The shipbuilding plan ends up being a 30-year plan. And they try and create visibility so folks can kind of see what outlays are going to look like. We’re going to need jobs that are skill-based, don’t necessarily require college degrees, and are colocated in communities where they could use the jobs. That’s a natural-made opportunity. Eric Chewning In this instance, the inflationary assumptions were around 2 to 3 percent, but we’re just coming off a quarter where inflationary assumptions were 7 percent. And when you look at the way DOD experiences inflation, it’s usually 20 basis points higher than the rest of the economy. They’re particularly susceptible to increases in fuel prices. So we just said, if you actually play out this delta in inflationary assumptions—and you could see during the ’70s the stagflation period extended periods of higher rates of inflation—you’re really, really going to be eroding the DOD’s buying power, as it comes to modernization of equipment. That then puts a new onus on industry as they think about how. André Dua: I want to come back to a little bit of your career. Last year, you wrote about lessons that we can learn from the military to tackle the COVID-19 crisis. I’d like to see whether you think there are also lessons that we can learn from the military as leaders think about the issues of how to spur sustainable, inclusive growth in the economy and in their organizations. The military has a unique way of doing things, and presumably there are some interesting insights that we can all learn from that. Eric Chewning: For me, my time in the US army was the most inclusive organization that I’ve ever been a part of. And I know for folks who haven’t served, that may sound a bit counterintuitive. But as I think about my own experience with my unit, particularly the one I served with when we were in Iraq, we had a Japanese American as our battalion commander. Our company commanders were a Native American, a Puerto Rican, and a White person, like myself. All the soldiers came from every different conceivable walk of life. We had Ivy League students. We had West Point students. We had enlisted kids right out of high school. You had me, who just came from an investment-banking career. You had people with master’s degrees through GEDs, and it was a great group of folks who were united by a common mission. They had a shared sense of purpose. But each individual leader, from the commander on down, created a culture of trust. I think when corporate leaders talk about sustainable, inclusive growth, there’s the potential for the discussions to veer off in certain directions. It’s important that we maintain cultures of trust and a common sense of purpose, because I think when you’re able to do that within an organization, you’re going to get much better outcomes. I think that was certainly true for the teams I’ve worked with in the military. André Dua: Let’s stick with this topic of sustainable, inclusive growth for a second. I’m curious for your thoughts on what actions you think leaders could take generally to build a more sustainable and inclusive future for all Americans. I think it’s maybe worth recapping why we think this matters. Without growth, there can’t be more opportunity for more people. Without sustainability, we don’t necessarily bequeath to future generations a world which continues to be prosperous and has many of its natural assets. And without inclusivity, not everyone has their opportunity to participate and make a better life for their family. So within that context, what do you think leaders should be doing to build a sustainable and inclusive future for Americans? Eric Chewning: I always think the power of example is important, and this idea around, how can you create alignment on a vision and then make folks feel comfortable—that there’s a place for them in the future that you’re outlining? And in my mind, in any type of transformative change effort, that’s sort of the beginnings of the bare-minimum table stakes that, as a leader, you have got to be able to put in place. And I think that’s true with SIG [sustainable, inclusive growth]. The nice thing about sustainable, inclusive growth is that it’s amorphous enough where you can describe it in a way and describe the value proposition in a tailored context. But it’s real enough that people can kind of see the type of future you’re describing, one where there’s economic opportunity, one where that economic opportunity is broad based, and everyone has a chance to benefit from it, and it’s done in a way where you’re creating a healthier community. And it’s hard to find someone who’s going to disagree with that value proposition as you hit each of the
specific points. André Dua: Eric, thanks for sharing your insights with us today. I really appreciate it. I wanted to reflect on a couple things that I found really interesting. First, it just made me realize how differently we need to think about US manufacturing forward because it’s so important. But the reason we need to think about it differently is we are entering a world in which resilience is increasingly important. And then there are some really significant national-security implications of not having the right manufacturing footprint. So I think my first big takeaway is, it’s time to have an elevated dialogue about US manufacturing. The second thing I take away from this discussion, Eric, is the absolute importance of talent. There resides in American citizens and residents an enormous amount of talent. As a nation, we’ll be well served to invest in that talent and to find ways to pull them into productive work, because it’s going to be a huge unlock for this country. I really think that’s an important thing that really ran through a lot of the conversation. Eric, we like to wrap up these Future of America episodes with a few quick questions. We always ask the same three, so I’m going to hit them now. Is there a book or article that you’ve recently read that excites you about a more sustainable and inclusive future? Eric Chewning: First of all, great summary. In line with that, the book I’ll reference was written by a friend of mine, Matt Kroenig at Georgetown [University]. And he wrote a book on The Return of Great Power Rivalry , and the book looks at historical competitions where relatively democratic versus more autocratic governments competed, going back to Athens and Sparta, all the way through the US–Soviet Cold War. He makes this argument that countries with relatively more open government and inclusive societies sustain long-term advantage in the international system because they get economic growth, because diplomatically they’re able to be more inclusive when it comes to management of allies and coalitions, and that they’ve got better military strength, because the societies hang better together and they’re more innovative in their decision making. I like this idea around SIG as a national competitive advantage, not just for companies but for societies. That’s an interesting kind of thought experiment to explore. André Dua: Second question: What makes you optimistic that we can achieve sustainable and inclusive growth? Eric Chewning: If you ask everyday folks, “Do you want a future for America where your kids have opportunity? Do you want one where there’s broadbased economic growth for everyone? And do you want a healthier community?” They’re going to say yes to all three of those questions. Then you say, “OK, that’s great. We’ve sort of aligned on kind of the type of future we want to have. Now let’s take the next step and talk about how we get there.” Often, alignment around a vision is sometimes the hardest part, but it can be the most important. There’s real work to be done after that, but you’re part of the way there. André Dua: Lastly, what’s the one thing that listeners can do to help promote sustainable and inclusive growth themselves? Eric Chewning: In theme with our discussion, I’m going to say, just support domestic manufacturing. André Dua: Thank you, Eric. That was Eric Chewning, a partner in our Washington, DC, office and the coleader of McKinsey’s Aerospace & Defense Practice in the Americas. And I’m André Dua. You’ve been listening to McKinsey’s Future of America podcast series. Thanks for joining us. About the author(s) Eric Chewning Partner, Washington DC LinkedIn André Dua Senior Partner, Miami LinkedIn Explore a career with us Search Openings Related Articles Commentary Want to drive inclusive economic growth? Start with manufacturing Discussion Paper - McKinsey Global Institute Building a more competitive US manufacturing sector Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Race in the workplace: The frontline experience | McKinsey Skip to main content Race in the workplace: The frontline experience July 30, 2022 | Report Three of four frontline workers want to be promoted, but less than one in four achieves it. Companies can take targeted actions to better support these workers. LinkedIn Twitter Facebook Email Full Report (46 pages) Race in the workplace: The frontline experience LinkedIn Twitter Facebook Email Full Report (46 pages) DOWNLOADS Full Report (46 pages) The vast majority of Americans are introduced to the workforce through frontline jobs—whether waiting tables, stocking store shelves, or folding clothes. Approximately 70 percent of the current US workforce is concentrated in frontline jobs. 1 1. McKinsey analysis based on Bureau of Labor Statistics, 2019 data. Too often, however, frontline jobs are both a starting point and an end point for workers. Our research found this challenge is especially true for frontline workers of color, who face an array of impediments to moving up the ladder. Sidebar LinkedIn Twitter Facebook Email About the authors This report is a collaborative effort by Chris Copeland, Bryan Hancock , Sofia Soto, Monne Williams , and Lareina Yee . The research in this report was conducted in collaboration with Walmart, the W.K. Kellogg Foundation, PolicyLink, the People & Organizational Performance Practice at McKinsey, and the McKinsey Institute for Black Economic Mobility. This report shines a light on the experiences of frontline workers of color, the pathways from the front line to the middle class, and the skills workers need to advance. It also offers steps companies could take to improve job quality and better support frontline workers of color to develop and progress in their careers. The frontline experience for workers of color With 112 million workers, the frontline workforce is massive but not a monolith. Frontline professionals (for example, school teachers and registered nurses) number 17 million workers who earn an average annual salary of $54,000. 2 2. McKinsey analysis based on Bureau of Labor Statistics, 2019 data. Frontline hourly and salaried roles (such as retail salespeople, cooks, and store managers) are filled by 95 million workers who earn an average annual income of $33,000. Our research focused on this second category. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes The front line is a vital part of nearly all sectors of the economy. They are the public face of many organizations, working in industries from healthcare to transportation and logistics to foodservice. They make tremendous contributions to the US economy, including carrying the nation through the pandemic. Yet despite these contributions, frontline workers experience the greatest hardship from economic disruption. Workers of color, who are overrepresented in the US frontline workforce, feel these challenges more acutely than their White counterparts. At many companies, frontline roles are a revolving door with low pay and little advancement, leaving workers of color without a path to move up the ladder. As a result, more employees of color are in roles with lower job quality (for example, jobs that lack healthcare benefits or don’t pay a living wage 3 3. Marcela Escobar, “6 job quality metrics every company should know,” Brookings Institution, October 27, 2021. ). Our research found employees in frontline roles report the worst experience. Frontline hourly employees are nearly 20 percent less likely than corporate employees to believe that DEI policies are effective (Exhibit 1). Just one-third of workers in the bottom 10 percent of income had jobs with paid sick leave. 4 4. "Employee benefits in the United States," Bureau of Labor Statistics, March 2019. In addition, 45 percent of hourly employees don’t believe their company encourages them to take advantage of work-life policies (for example, leave of absence and parental leave) without jeopardizing their employment or career advancement. These factors have a dramatic influence on their job experience and views of their company. Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Frontline workers of color do not feel included in the workplace Video Frontline hourly employees report the lowest overall feelings of inclusion 5 5. Inclusion consists of behaviors such as allyship, mutual respect, and advocacy as well as conditions such as shared prosperity and fair participation. of all employees in the workforce, and differences in inclusion emerge as they climb the corporate ladder (Exhibit 2). While all groups feel more connected at higher levels of their organization, Black employees experience lower inclusion than their peers at most levels. This pattern essentially sets up a no-win situation for Black frontline workers: shared stressors in the front line or feelings of isolation as they move up the ladder. Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com ‘We’ve had some really bright, talented Black individuals, but the culture just will not allow them to advance in positions that are beyond the bottom of the totem pole.’ Sidebar The Black frontline experience: Overcoming the trust deficit Continue to next section LinkedIn Twitter Facebook Email Fast facts 13 percent of the frontline workforce Top industries: healthcare support, protective services, and community and social services Our Race in the Workplace Survey found that among all frontline staff, Black workers have the largest enterprise trust deficit—defined as a gap in perception on attributes such as acceptance, fairness, and authenticity. This trust deficit affects nearly every facet of the Black frontline experience. For example, Black workers report the highest job satisfaction and lowest intention to leave. Yet they also experience the highest attrition rates. What is causing them to look to other organizations for opportunities? For many, it’s a lack of career development and advancement potential. Fully 84 percent of Black employees indicated a desire to be promoted, but just 62 percent perceived an opportunity to advance. They also don’t view their work environment as a meritocracy. A contributing factor is that Black frontline workers earn on average 25 percent less than their White peers. The lack of trust manifests itself in numerous ways. Black workers are less likely to feel supported, encouraged, and treated as professional equals by their peers. Fifty-one percent of Black employees have at least one mentor, but just 38 percent said they have at least one sponsor, meaning many Black workers lack valuable direction and support on career paths and development opportunities. In addition, Black frontline employees report feeling included in the workplace less than any other racial group. As poor as the experience is for Black frontline workers, it also represents a high-water mark. As they move up the ladder, they report feeling less included and supported than their peers. Video Frontline hourly workers perceive a pervasive lack of fairness in promotions Our research explored the perceptions of frontline hourly employees regarding fairness and transparency in promotions. They are more likely than their salaried peers to feel their organization is inconsistent when it comes to promoting employees on merit and performance (Exhibit 3). Just 39 percent of hourly respondents believe their employer takes an objective, empirical view of performance and promotion. In addition, our analysis highlighted frontline workers of color want to advance but lack access to opportunities by a sizable margin compared with White workers. Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Sidebar Asian workers: A tale of divergent experiences Continue to next section LinkedIn Twitter Facebook Email Fast facts 1 1. Bureau of Labor Statistics (OES and QCEW); Moody’s Analytics; American Community Survey, 2019. 5 percent of frontline workforce Top industries: Accommodation and foodservice, manufacturing, healthcare At first glance, Asian frontline workers appear to be poised for rapid advancement. They have the highest levels of education: 21 percent have earned at least a bachelor’s degree, compared with 16 percent of White workers, 10 percent of Black workers, and 8 percent of Latino workers. They also report higher levels of inclusion than other workers of color and are more likely to believe their workplace is fair and impartial. Yet despite these advantages, Asians must overcome many of the same challenges as other workers of color: They consistently make less money than their White peers for the same job. They don’t feel supported, with more than half lacking a sponsor. And they report the highest levels of burnout among all frontline employees. Moreover, Asian workers are often stereotyped as being diligent and quiet, perceptions that could be a factor preventing them from moving into management positions. 2 2. Adeel Hassan, “Confronting Asian-American stereotypes,” New York Times , June 23, 2018. They experience a steady decline in representation by level, falling from 9 percent of frontline hourly roles to 5 percent of salaried positions. The picture is complicated further by the fact that Asian workers represent a diverse array of nationalities and subgroups, 3 3. The subgroups are South Asian (Bangladeshi, Bhutanese, Indian, Nepalese, Pakistani, or Sri Lankan), East Asian (Chinese, Japanese, Korean, or Mongolian), Southeast Asian (Burmese, Cambodian, Filipino, Hmong, Indonesian, Laotian, Malaysian, Thai, or Vietnamese), and Pacific Islander (Pacific Islander: People who are ethnically Fijian, Guamanian/Chamorro, Native Hawaiian, Micronesian, Samoan, Tongan). East and Southeast Asians make up 74 percent of Asian frontline workers, while South Asians are more represented in non-frontline roles. each with its own distinct experience. For example, Southeast Asians report some of the lowest inclusion scores, while East and South Asians report among the highest. Companies seeking to devise effective programs and provide support for Asian workers must dig beneath the top-line classifications to truly understand the specific challenges each subgroup faces. Video Workers of color are held back by low levels of sponsorship The level of support that workers receive on the job can have a direct impact on their career prospects. Our research found more than half of all frontline employees of color have at least one mentor in the workplace—in line with their White peers. But these relationships aren’t translating into sponsorships (Exhibit 4). Black and Latino frontline employees report the lowest levels of sponsorship: the majority (nearly six in ten) have no sponsor at all, with Black frontline employees seeing especially low levels. Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com This pattern appears to have a direct impact on career advancement. Our analysis found employees are five times more likely to get a promotion if they have four or more sponsors. In effect, every sponsor translates to a roughly 10 percent increase in an employee’s chance of getting a promotion. Sidebar Latino workers: Resilience in the face of challenges Continue to next section LinkedIn Twitter Facebook Email Fast facts 14 percent of frontline workforce Top industries: Agriculture, construction, and forestry, fishing, and hunting Latino workers, who are more likely than any other race to be in frontline roles, face several daunting challenges when it comes to careers. They have the second lowest wages of all groups, earning 22 percent less than White frontline workers, in part because of the occupations in which they are concentrated. They are the largest immigrant group of any race, with 85 percent of immigrants on the front line. 1 1. American Community Survey, 2019; US Census Bureau, accessed July 27, 2022. They also have the lowest education levels of any race: just 36 percent have some postsecondary schooling. Perhaps unsurprisingly, Latino frontline employees don’t feel included or supported on the job: they are significantly less likely than White employees to believe their employer allows self-expression and has adequate protective mechanisms in place. In greater numbers than other groups, Latino frontline employees cite a lack of support for employee well-being as a key reason for leaving their jobs. They also report the second-lowest level of sponsorship, a crucial element in advancement. Despite these challenges, Latino employees in our sample have achieved a higher share of promotions from frontline jobs to entry-level corporate roles than other employees of color. They also report levels in inclusiveness in line with their White peers when they are promoted to corporate roles. Video Creating a pathway for advancement from the front line Our analysis assessed occupations by their value and role in career advancement and identified concrete pathways for advancement. We found it is possible for workers to move from initial frontline jobs and the accompanying economic insecurity to roles with middle-class incomes and better career prospects. A recent McKinsey research effort sought to identify the progression of frontline workers as well as the attributes and experiences that enable their advancement . 6 6. “ Unlocking experience-based job progressions for millions of workers ,” McKinsey, June 2, 2021. We analyzed historical job progressions of four million workers without four-year degrees, also known as individuals Skilled Through Alternative Routes (STARs), 7 7. STARs: Skilled through alternative routes,” Opportunity@Work, accessed July 28, 2022. who successfully transitioned from low- to higher-wage occupations and identified the jobs that created pathways to higher-paying roles. The career journey was segmented into five job categories 8 8. Most frontline jobs are concentrated in the Origin and Mid-wage Origins categories, but about 20 percent are Target jobs. : Origin occupations are frontline jobs with pay of less than $37,000. Wait staff, retail salespeople, and maintenance workers are common Origin roles. Mid-wage Origin roles , such as automotive master mechanics and chefs provide a small increase in pay ($37,000–$42,000) for workers who transition from Origin roles. Roles in Historical Adjacencies , such as cargo and freight agents, electricians, and nurse practitioners, offer workers a bump in pay (to more than $42,000), but they have limited success as pathways to higher-wage occupations. Gateway occupations give workers the opportunity to build skills and experience and earn more than $42,000 a year. Gateway roles include food service managers, vocational nurses, and radiologic assistants. Target occupations are jobs with middle to higher wages (more than $42,000) that are resilient to automation. Companies often hire for these roles based on job experience, not just credentials. Examples include sales managers, social workers, and critical-care nurses. Our analysis identified 77 Gateway occupations that are especially effective at unlocking job progressions into Target jobs (Exhibit 5). These roles can help workers develop new skills (for example, a customer service representative moving into an IT role) or broaden existing skills gained through work experience (such as a medical assistant moving into a more advanced role in healthcare). Exhibit 5 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com A closer look at Origin roles reveals unequal access to the jobs that provide a path to higher-paying jobs. Origin roles in which workers of color are overrepresented (for example, security guards and light-truck drivers) have pathways to higher-paying roles, but workers of color are not promoted to these jobs (Exhibit 6). Exhibit 6 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com The existence of these pathways should be cause for optimism. However, workers of color must also overcome entrenched biases within the workplace that limit opportunities and impede their advancement. For instance, many of the roles with pathways lean more on interpersonal skills than on formal education. Since the assessment of these skills is inherently more subjective, it allows bias to have a greater influence on promotion decisions. ‘While there are opportunities at times for advancement, my peers often doubt me and my skill set is judged, even though I share the same qualifications as everyone else.’ A road map for change Our research and analysis have clearly demonstrated that frontline workers want to develop, progress, pursue new opportunities, and find roles that are more fulfilling. In that regard, companies have the potential to mobilize and develop their front line, which could create benefits for workers and organizations alike. Yet companies must provide more support to frontline workers to enable their career advancement. This will require organizations to reassess their traditional approach to the front line and directly address three myths. Myth 1: Frontline workers are free to move up the corporate ladder. Reality: More than 70 percent of frontline workers want to be promoted within their companies, but only 4 percent make the leap to corporate. While relatively few frontline workers get promoted to corporate jobs, it’s not for lack of interest. By acknowledging this truth, companies can turn their focus to increasing that number. Formalize paths for advancement from the front line to higher-paying roles Overhaul the frontline talent management system Establish a talent market program Myth 2: Frontline workers are not qualified for higher-level roles. Reality: 70 percent of job progressions to Gateways hinge on transferable interpersonal skills, which are best learned through experience. Companies can support upward mobility by emphasizing on-the-job experience, and creating more transparency around opportunities and skills needed. Define the skills frontline workers need for higher-level roles Reward experience rather than relying on credentials Identify Gateway jobs within the organization and remove artificial barriers to promotion Myth 3: High rates of turnover are just the way it goes on the front line. Reality: Companies have a responsibility to create a better employee experience. Companies can improve the workplace environment to make the worker experience more positive and sustainable. Give a voice to frontline workers “Raise the floor” on the frontline experience Invest in frontline managers ‘My sponsors have allowed me to rotate jobs and learn about different departments. I’ve been able to learn new types of skills.’ For tens of millions of workers of color, frontline jobs have the potential to be far more than just a livelihood. Companies that truly invest in the front line can make these roles a starting point for a fruitful career, a clear path to the middle class, and a way to transition to interesting new jobs across industries. Much work remains to fulfill this aspiration. But for workers who have been traditionally overlooked, the time is long past due for companies to provide more opportunities and support. About the author(s) Chris Copeland is an associate partner and Monne Williams is a partner in McKinsey’s Atlanta office, Bryan Hancock is a partner in the Washington, DC, office, Sofia Soto is a consultant in the New York office, and Lareina Yee is a senior partner in the San Francisco office. The research team comprised Diana Nleya, Layaill Mustafa, Noah Furlonge-Walker, and Ranti Odujinrin. Jackie Wong, an engagement manager based in Philadelphia, provided the team with additional research support. The research in this report was conducted in collaboration with Walmart, the W.K. Kellogg Foundation, PolicyLink, and McKinsey’s Institute for Black Economic Mobility, with video production by Osato Dixon . Explore a career with us Search Openings Related Articles Article - McKinsey Quarterly One move companies can take to improve diversity Article - McKinsey Quarterly The Black experience at work in charts Article Women in the Workplace 2021 Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Shanghai |  | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content McKinsey Client Capabilities Network Shanghai Shanghai is an international metropolis at the forefront of Asian economic development. The Client Capabilities Hub in Shanghai is part of the McKinsey Greater China office and home to over 250 client capabilities professionals. Their specialized and cross-industry knowledge span automotive, basic materials, consumer goods and retail, chemicals, energy, financial institutions, high tech, media, telecommunications, pharmaceuticals, and travel and logistics. In particular, the advanced analytics group serves clients in big data, data mining, and data optimization, playing a critical role in delivering and developing knowledge related to the region’s most important sectors. Our People Director, Client Capabilities Hub, Shanghai Nuo Dai Leads the Client Capabilities Hub in Shanghai, driving strategic planning, capability building, operational management, recruiting, and training. Careers Working in Shanghai We are a tight-knit, supportive community of recent graduates and experienced professionals. Many colleagues are involved in various office initiatives, such as charity work with local organizations. Our people not only build knowledge, capabilities, and new skills but also grow into well-rounded leaders through mentoring and training opportunities in communication, people and project management, leadership, and problem solving. Explore careers at McKinsey Explore jobs in Shanghai Search jobs Already started an application? Sign in here Prepare for your interview Learn more Every day we help our clients solve their toughest problems, build capabilities, and create better outcomes. Learn more on our Global Careers site Our location Shanghai Client Capabilities Hub Shanghai Client Capabilities Hub 21F, 3 Corporate Avenue 168 Hubin Road Shanghai 200020 Voice: +86 (21) 6385 8888 Fax: +86 (21) 6386 2000 https://maps.google.com/maps?q=31.2208534,121.4772106&z=15 Our Client Capabilities Hub locations Atlanta Brussels Chennai Dallas Dubai Gurugram Johannesburg Lisbon Riyadh San José Shanghai Waltham Wroclaw Connect with the McKinsey Client Capabilities Network Contact Belgium and Luxembourg | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Welcome to McKinsey Belgium and Luxembourg Our diversity brings together the best minds to tackle the most complex problems. McKinsey Belgium & Luxembourg works with global companies, multinationals, and emerging leaders in and outside of BeLux. We help businesses build solutions and capabilities for lasting performance improvements across financial services, basic materials, transportation, pharmaceuticals & medical products, telecommunications, consumer goods, technology, and more. 40+ years in BeLux 40+ countries where BeLux staff served clients in 1 year 30+ nationalities represented in our office Inspiring Belgium With the world in constant flux, staying informed about the latest insights and trends is a real challenge. Inspiring Belgium includes curated content on key topics to help regional leaders prepare for the future. Learn more Our people Frederic Vandenberghe Senior Partner, 
                                        Brussels Leads our offices in Belgium and Luxemburg, helping clients across Europe in the financial services, business services, B2C, and... Mieke Van Oostende Senior Partner, 
                                        Brussels Serves clients in the financial-services sector, particularly on strategy, organization, and risk management, with a global reputation... Michel Van Hoey Senior Partner, 
 Luxembourg Connect with McKinsey BeLux Contact Accelerating the transition to net-zero travel | McKinsey Skip to main content Accelerating the transition to net-zero travel September 20, 2022 | Report LinkedIn Twitter Facebook Email Full Report (35 pages) Accelerating the transition to net-zero travel LinkedIn Twitter Facebook Email Full Report (35 pages) Despite the best intentions, many travel companies are struggling to reduce their emissions fast enough. A report by McKinsey and Skift provides four practical recommendations. DOWNLOADS Special Report Full Report (35 pages) The worsening effects of climate change have made decarbonization a top priority for many industries, including global travel and tourism, which accounts for between 8 and 11 percent of the world’s emissions. 1 1. A net zero roadmap for travel & tourism: Proposing a new target framework for the travel & tourism sector , World Travel & Tourism Council, November 2021. If nothing is done, the sector’s carbon emissions will only rise as it grows. Travel activity is expected to soar by 85 percent from 2016 to 2030. 2 2. “Tourism’s carbon emissions measured in landmark report launched at COP25,” UNWTO, December 4, 2019. As more consumers, employees, regulators, and investors consider reducing their trips, they are also ramping up the pressure for the sector to reduce its carbon emissions. In response, more travel companies have pledged to reach net zero. But obstacles stand in the way. The range of decarbonization technologies in the market is limited, and what’s available is expensive. Sidebar LinkedIn Twitter Facebook Email About the authors This article is a collaborative effort by Danielle Bozarth , Olivier Cheret, Vik Krishnan , Mackenzie Murphy, and Jules Seeley , representing views from McKinsey’s Travel, Logistics & Infrastructure practice. While demand reduction may be part of the answer, there are many practical steps travel companies can take right now to accelerate their journey toward greater sustainability—and potentially create value while doing so. McKinsey and Skift Research have put together a report exploring four high-priority areas for travel companies to focus their decarbonization efforts to catalyze the most meaningful outcomes for the environment, their customers, and themselves. This article provides a summary of the report’s key insights. Identify and sequence decarbonization initiatives Being familiar with the relevant decarbonization levers for your business is a good start, but it’s not enough. Many travel companies struggle with implementation and balancing tradeoffs. A robust plan factoring in various potential pathways is critical for success in a quest as complex as deep decarbonization. The marginal abatement cost curve (MACC) pathway framework provides a cost-benefit analysis of the individual levers and phasing plans, helping companies identify strategic tradeoffs between the various implementation pathways over time. Companies can use the framework as a tool to validate climate-related targets and time their decarbonization initiatives. By assessing the feasibility and tradeoffs between multiple potential paths, companies could be better equipped to select the most appropriate pathway for them to reach their net-zero goals. Would you like to learn more about our Travel, Logistics & Infrastructure Practice ? Partner to accelerate decarbonization of business travel Business travel represents 30 percent of all travel spend, making it an important segment for travel companies. 3 3. Getting back to business: Navigating the safe return of meetings and their role in economic recovery , US Travel Association, November 2020. As more organizations—businesses and non-profits alike—set more ambitious emissions-reduction goals for themselves, they’ll almost certainly be reevaluating their travel habits. This opens up opportunities for travel companies to enter into robust decarbonization partnerships with their corporate clients. The top 100 highest-spending organizations of business air travel spent around $12 billion on domestic and international bookings in the US in 2019. Forty-five of them—together accounting for around half of this business air travel spend—have not only made public decarbonization declarations, but have also committed to a target year of 2030 or before. These organizations would be eagerly seeking out ways to reduce their emissions and are likely to be more receptive to enter into decarbonization agreements with travel companies. To help organizations that travel achieve their net-zero goals, travel companies could expand their range of sustainability-focused product and service offerings. As examples, car rental companies could encourage clients to pay for electric vehicles, while hotels may command a premium for corporate packages that promote low-carbon and sustainable practices. Travel companies could also devise other ways to support these organizations in their transition toward greater sustainability. These could include designing booking platforms to nudge users to make more sustainable choices while making reservations, helping employees track their emissions, and providing greater data transparency for organizations to accurately measure their carbon emissions from corporate travel. Close the “say-do” gap among leisure travelers Studies suggest that leisure travelers—who generate 70 percent of all travel revenues—are also supportive of decarbonization. 4 4. Getting back to business: Navigating the safe return of meetings and their role in economic recovery , US Travel Association, November 2020. A recent McKinsey survey indicates that 40 percent of travelers globally say they are willing to pay at least two percent more for carbon-neutral flight tickets. However, Skift’s latest consumer survey has exposed a “say-do” gap; only 14 percent of travelers state that they actually paid more for sustainable options when they travel. 5 5. Mishal Ahmad, Frederik Franz, Tomas Nauclér, and Daniel Riefer, “ Opportunities for industry leaders as new travelers take to the skies ,” McKinsey, April 5, 2022; “US travel tracker survey,” Skift Research, August 2022. There are a number of reasons for this "say-do" gap, including a lack of clear sustainability-related information, a scarcity of sustainable booking options, and high cost barriers. Many customers also struggle with balancing competing priorities when purchasing a travel service or product. Travel companies can bridge this gap and help consumers choose more sustainable actions by focusing on three levers: offering more sustainability-focused travel products and services visibly within the booking journey, presenting decarbonization information in compelling ways to engage and resonate with customers, and using behavioral science techniques to encourage leisure travelers to make sustainable purchase decisions (for instance by actively promoting the most environmentally friendly options first). A methodical approach is necessary to execute these initiatives while keeping the booking journey seamless for the customer. A dedicated task force armed with digital capabilities, consumer insights, and sustainability expertise could be set up to test and refine these changes for smaller sample groups in each customer segment before implementation is scaled up. Rebooting customer experience to bring back the magic of travel Read the report Build new sustainable travel options for the future Instead of reacting to the latest developments in the transition toward net zero, the travel sector can be a proactive force and pioneer sustainable products and services. Not only will doing so reduce the industry’s carbon emissions, but it will also enable individual travel companies to create new sources of revenue that could be redirected to other green business opportunities. There are as many possibilities as there are types of travel companies. An airline may consider playing a more active role in the sustainable aviation fuel (SAF) value chain, perhaps by investing in production plants to increase the supply of alternative fuels or launching a green airline that deploys smaller alternative propulsion aircraft. Hotel and lodging companies can launch green hotels or brands, applying the latest design approaches and green technologies. Larger and more established travel agencies and booking platforms could provide standardized environmental, social, and governance (ESG) measurement services to smaller travel companies that lack such capabilities or brands focused on green travel. Finally, credit card companies could partner with airlines and hotels to offer customers a consolidated view of their emissions across travel providers, giving them additional rewards when they choose more sustainable options. Launching these new businesses will likely require companies to create special initiatives outside of the current core of the business. The teams leading them will need to be empowered to experiment with and learn from short iterations and pilots without the pressure to be immediately profitable. Fortunately, the first examples of travel companies applying green business-building principles show promising results, which should pave the way for others to follow suit. The full report concludes by offering a checklist for travel companies to track their readiness in each of the four strategies and to maximize their chances for a successful—and profitable—journey to net zero. Companies that act now could gain a competitive advantage over their peers, while those that stand by and wait may find their value proposition erode as they get left behind. Download the full report About the author(s) Danielle Bozarth is a senior partner in McKinsey’s New York office, Olivier Chéret is a senior expert in the Montreal office, Vik Krishnan is a partner in the San Francisco office, Mackenzie Murphy is a consultant in the Seattle office, and Jules Seeley is a senior partner in the Boston office. The authors wish to thank Seth Borko, Wouter Geerts, Melissa Lukasiewicz, and Haixia Wang for their contributions to the report. Explore a career with us Search Openings Related Articles Report Rebooting customer experience to bring back the magic of travel Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Switzerland | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Welcome to McKinsey Switzerland We combine McKinsey’s global expertise with deep local insights to help our clients accelerate sustainable and inclusive growth. Working at the heart of the world’s most competitive economy, we serve clients in both the private and public sectors on challenges in operations, organization, strategy, and technology. Our offices are located in Zurich and Geneva, and our consultants work across the dynamic financial, pharmaceutical, and manufacturing sectors. At a local and global level, we combine bold strategies and transformative technologies to help organizations innovate more sustainably, achieve lasting gains in performance, and build workforces that will thrive for generations. >60 years ago McKinsey opened its first office in Switzerland >40 nationalities are represented across our Swiss offices >320 projects conducted over the past year Our people Michael Steinmann Managing Partner Switzerland, 
                                        Zurich Leads McKinsey’s work in Switzerland, advises leading life-science and industrial companies on strategy, organization, innovation... Sylvain Johansson Senior Partner, 
                                        Geneva Leads McKinsey's work in Geneva; supports financial-services institutions in corporate and business-unit strategy, transformation,... Meet our people Careers Working in Switzerland A career at McKinsey is an opportunity to work with remarkable people, help shape growth, build important institutions in Switzerland—and learn and grow constantly. We offer a variety of paths for students, graduates and experienced professionals. Learn more Featured Initiatives McKinsey supports leadership, growth, and the entrepreneurial spirit. We seek out partnerships that focus on knowledge, innovation, and giving back to the community. Swiss Institute of International Studies Founded in 1943, the Swiss Institute of International Studies (SIAF) is a politically and economically independent center of excellence based in Zurich. The institute seeks to serve as a platform for critical debate and thought leadership on current world topics, attracting world-renowned and top-class guest speakers from academia, business, culture, and politics. Learn more on the SIAF site >>venture>> foundation >>venture>> is Switzerland’s leading and largest annual start-up competition. Established in 1997, it is governed by the not-for-profit >>venture>> foundation with the purpose of empowering entrepreneurs and fostering innovation across Switzerland. Participants can apply each year under one of the following five industry verticals: finance and insurance, health and nutrition, industrial engineering and hardware, information and communication technologies, and retail and customer services. Learn more on the Venture site McKinsey for Children The McKinsey for Children charity was established in 1995 to support children in need by improving their prospects in childhood and guiding them along the path to a successful adult life. All supported projects are located in regions characterized by poverty and are devoted to helping children around the world by providing education, meals, or shelter. Learn more Featured Insights Article Klimastandort Schweiz: Schweizer Unternehmen als globale Treiber für Netto-Null July 10, 2022 – Die Welt ist nicht auf Kurs, Netto-Null bis 2050 zu erreichen. Die Schweiz ist hier keine Ausnahme. Obwohl die Schweiz mit ihrer fortschrittlichen Wirtschaft im Inland <0.1% der globalen Emissionen verursacht, spielt sie als internationales Wirtschafts- und Finanzzentrum eine bedeutende Rolle in der Dekarbonisierung. Report Making up lost ground: How Switzerland’s second-pillar pension funds can improve their investment performance March 1, 2020 – Taking action in three key areas would allow Swiss pension funds to capture significant benefits without taking undue risks or making big leaps of faith. Report Creating value, finding focus: Global Insurance Report 2022 February 15, 2022 – The insurance industry struggles to create economic profit. But amid COVID-19’s enduring changes, opportunities await. Collection The net-zero transition: What it would cost, what it could bring January 25, 2022 – This McKinsey report offers a detailed look at the economic and societal impact of the transition to net-zero carbon emissions... by 2050. Article
            
 Switzerland Connect with McKinsey Switzerland Contact LinkedIn Twitter Instagram Facebook YouTube Economic conditions outlook, September 2022 | McKinsey Skip to main content Economic conditions outlook, September 2022 September 29, 2022 | Survey LinkedIn Twitter Facebook Email Article (6 pages) Economic conditions outlook, September 2022 LinkedIn Twitter Facebook Email Article (6 pages) In stormy weather, survey respondents maintain realism about the global economy. While geopolitical conflicts and inflation remain top of mind, concerns about energy volatility predominate in Europe. View the surveys Economic conditions outlook, September 2022 Economic conditions outlook, June 2022 Economic conditions outlook, March 2022 Survey Economic conditions outlook, September 2022 In September, respondents in most regions cite inflation as the main risk to growth in their home economies for the second quarter, according to the latest McKinsey Global Survey on economic conditions. 1 1. The online survey was in the field from August 29 to September 2, 2022, and garnered responses from 1,247 participants representing the full range of regions, industries, company sizes, functional specialties, and tenures. To adjust for differences in response rates, the data are weighted by the contribution of each respondent’s nation to global GDP. Geopolitical instability and conflicts remain a top concern as well, most often cited as the greatest risk to global growth over the next 12 months. Responses assessing the global economy are primarily downbeat, as they were in the last survey. Regional divergence in outlooks has emerged, as respondents in Europe express deeper concerns over energy price volatility and more somber views about their domestic economies. Respondents in North America, on the other hand, were less negative about their countries’ current economies than in the previous survey. DOWNLOADS Article (6 pages) Regional differences also appear when private-sector respondents report on the cost increases that are most affecting their companies. Respondents in Europe most often cite the impact of rising energy prices, while those in India and North America tend to point toward wage increases. Overall, nine out of ten respondents say their companies have seen cost increases in the past six months, and a majority have raised the prices of their products or services. Most also foresee their organizations’ operating expenses increasing in the coming months. Views on global conditions remain downbeat After a particularly negative assessment of economic conditions in the June survey, responses to the latest survey are almost as gloomy (Exhibit 1). Looking toward the future, pessimism remains consistent with the previous findings, with about half of respondents expecting global conditions to weaken in the next six months. Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Respondents’ takes on the global economy vary significantly by region, however. Those in Europe and North America offer a grim view of both current and future global conditions, whereas those in Greater China 2 2. Includes respondents in Hong Kong and Taiwan. are primarily positive about the present and the future. Overall, for the third quarter this year, geopolitical instability and conflicts remain the most-cited risk to global economic growth, and inflation remains the second-most-cited threat. In a change from June, volatile energy prices have superseded supply chain disruptions as the third-most-cited global risk. Inflation remains top of mind—except in Europe and Greater China Respondents’ concerns about supply chain disruptions as domestic economic risks have also diminished since the previous survey. Supply chain challenges are now the fifth-most-cited risk to respondents’ home economies, surpassed by concerns about rising interest rates. Inflation remains the most-cited risk to domestic economies for the second quarter, followed by volatile energy prices and geopolitical instability and conflicts. In all locations but Europe and Greater China, inflation is the most-cited threat to respondents’ economies over the next 12 months (Exhibit 2). In Europe, volatile energy prices and inflation are the growth risks cited most often, with geopolitical instability or conflicts a more distant third. In Greater China, the COVID-19 pandemic remains the most reported risk, cited by nearly half of respondents for the second quarter in a row. Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com As unease heightens in Europe, optimism builds in North America Similar to the June survey, four in ten respondents say economic conditions in their countries have improved over the past six months. However, the findings show new regional divergence (Exhibit 3). Responses in Europe are more downbeat than earlier this year, with more than three-quarters of respondents now reporting that their economies have worsened. At the same time, in North America—where sentiment was closely aligned with Europe’s in the previous two quarters—respondents have become more positive since the previous survey. In Greater China, India, and Asia–Pacific, a majority say their economies have improved. But in Asia–Pacific, optimism has faltered. Respondents there are much less likely than in the previous survey to say that their countries’ economies have improved. Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Expectations about the next six months also vary by region. Respondents in Europe and Asia–Pacific are less likely than in June to expect their countries’ economies to improve, while respondents in other developing markets have become more hopeful. Overall, respondents are about as likely to expect their countries’ economies to improve as to worsen in the next six months, as was also true in the previous survey. Concerns mount over companies’ prospects The latest survey asked private-sector respondents about the challenges their companies are facing and their expectations for the coming months. Nine in ten respondents say their companies have experienced cost increases in the past six months. The largest share of responses point to rising energy prices—which include electricity as well as fuel—as having the biggest impact, followed by increases in the costs of materials. The concerns over various types of cost increases vary by region (Exhibit 4). In Europe, respondents primarily point to rising energy costs, whereas wage increases are of top concern in India and North America. Consistent across all regions, respondents say their companies have raised the prices of their products or services in the past six months. Looking ahead, 71 percent of respondents expect their companies’ operating expenses to be greater next year than they were last year. Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com What’s more, expectations for companies’ profits and customer demand are the most downbeat that they have been since July 2020. Just 51 percent expect profits to increase, down from 65 percent six months ago. The same share—51 percent—expect demand for their companies’ goods or services to increase. Sidebar LinkedIn Twitter Facebook Email A note on the state of globalization For the first time, the McKinsey Global Survey on economic conditions asked respondents how they expect cross-border trade levels in the next three to five years to compare with the levels in the past three to five years. Forty-five percent say they expect lower trade levels, while 38 percent expect increased international trade. Respondents in developed economies are much more likely than emerging-economy respondents to expect trade levels to decline. We also asked respondents how they expected their companies’ supply chains to change over the same period. Forty-four percent say their companies’ supply chains will either become more regional or more local, concentrating within the countries or municipalities in which they operate. 1 1. Twenty-seven percent expect their companies’ supply chains to become more regional, and 17 percent expect them to become more local. The other 32 percent expect
    no change or answered, “don’t know.” By comparison, just 24 percent of respondents say their companies’ supply chains will become more global—that is, interregional. While concerns over the effects of supply chain disruptions on global and domestic growth have eased since the previous survey, those disruptions remain top of mind as a risk to company growth for the second quarter (for more on how respondents expect their supply chains to change, see sidebar, “A note on the state of globalization”). Furthermore, a majority of respondents working in manufacturing—including those in automotive and assembly, aerospace and defense, advanced electronics, and semiconductors—or retail report that their companies’ inventory levels are not ideal. One-third say they have too much inventory, while 21 percent say levels are too low. Looking specifically within the consumer goods and retail sector, respondents are just as likely to report too little inventory as too much, while a plurality say their inventory levels are about right. Of the respondents in all manufacturing and retail industries reporting nonoptimal levels, nearly three-quarters expect their organization to achieve optimal levels within the next 12 months. Download Economic conditions outlook, September 2022 (PDF–407  KB). ABOUT THE AUTHORS The survey content and analysis were developed by Jeffrey Condon , a senior knowledge expert in McKinsey’s Atlanta office; Krzysztof Kwiatkowski and Vivien Singer , both capabilities and insights experts at the Waltham Client Capabilities Hub; and Sven Smit , the chair and director of the McKinsey Global Institute and a senior partner in the Amsterdam office. This article was edited by Heather Hanselman, an editor in the Atlanta office. Survey Economic conditions outlook, June 2022 Just one quarter after geopolitical conflicts and instability overtook the COVID-19 pandemic as the leading risk to economic growth, survey respondents’ concerns over inflation now exceed their worries about the effects of geopolitical issues on their countries’ economies. In the latest McKinsey Global Survey on economic conditions, respondents also see inflation as a growing threat to the global economy and continue to view geopolitical instability and supply chain disruptions among the top threats to both global and domestic growth. 3 3. The online survey was in the field from June 6 to June 10, 2022, and garnered responses from 899 participants representing the full range of regions, industries, company sizes, functional specialties, and tenures. To adjust for differences in response rates, the data are weighted by the contribution of each respondent’s nation to global GDP. Most Popular Insights To slow down attrition, pay closer attention to what workers really need The gathering storm: An opportunity to reorder the healthcare industry Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle Accelerating the transition to net-zero travel Data ethics: What it means and what it takes Amid this disruption-crowded environment, respondents report uneasy views on economic conditions, both globally and in their respective countries. For the fourth quarter in a row, respondents to our latest survey—conducted the first full week in June—are less likely than those in the previous survey to say economic conditions have improved. Overall, pessimism about the second half of 2022 is on par with the early months of the pandemic in 2020. Exceptionally, however, the mood is much more positive among respondents in Asia–Pacific and Greater China, who report improvements and continue to be upbeat about their economic prospects. Inflation, geopolitical, and supply chain concerns all loom large Respondents’ views of the top threats to their home economies have shifted since March 2022, 4 4. The March 2022 survey was the first survey since December 2019 in which the COVID-19 pandemic was not one of the top five most-cited risks to domestic growth. From March 2020 through December 2021, the pandemic was the most-cited risk all but once. In the latest survey, it is the seventh-most-cited risk. and they now most often cite inflation as a risk over the next year (Exhibit 1). While geopolitical conflicts were top of mind in the previous quarter’s survey, which ran four days after Russia had invaded Ukraine, respondents are now nearly half as likely to cite geopolitical issues as a risk to their countries’ economies. Geopolitical conflicts and instability remain an outsize concern in Europe, where 50 percent list it among their top risks. But even in Europe, inflation is the risk cited most often—as it is in every geography except Greater China. 5 5. “Greater China” includes respondents in Hong Kong and Taiwan. There, respondents most often point to the COVID-19 pandemic. Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Sidebar LinkedIn Twitter Facebook Email Respondents predict extended disruption related to the Ukraine invasion We asked survey respondents about their expectations for how the war in Ukraine might affect lives and livelihoods outside the conflict zone . When asked about the war’s effects on the global economy, a plurality of respondents—37 percent—select a scenario called 2B, in which hostilities either end or are easing within the next six months and the global response is moderate, with a continued exit from stimulus policies related to the COVID-19 pandemic, reduced decarbonization goals, and a restart of fossil-fuel investments (exhibit). Exhibit We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Geopolitical instability remains the top-cited threat to the global economy (see sidebar, “Respondents predict extended disruption related to the Ukraine invasion”), as it was in the March survey, and inflation has overtaken volatile energy prices to become the second-most-cited concern. Supply chain disruptions round out the top three global risks, followed by volatile energy prices and rising interest rates. For the second survey in a row, more than three-quarters of respondents expect interest rates in their countries to increase in the next six months. 6 6. In comparison, in the September 2021 survey, 51 percent of respondents said they expected interest rates in their countries to increase, and 64 percent said the same in the December 2021 survey. Respondents also see supply chain disruptions as major obstacles for their companies’ growth. In the latest survey, that answer choice has overtaken geopolitical instability as the most-cited risk to companies’ growth. These supply chain concerns—and those about the changing trade environment and relationships—are much more common among respondents who say at least some of their companies’ essential materials 7 7. We define “essential materials” as any components that are necessary to produce new products or services. are produced in China than among those who don’t source materials from China. Would you like to learn more about our Strategy & Corporate Finance Practice ? Visit our Trends & Global Forces page Respondents are largely pessimistic about the global economy but more positive about their countries’ prospects Nearly two-thirds of respondents say the global economy is worse now than it was six months ago—the highest share to say so since the June 2020 survey . That appraisal is much more negative than what respondents predicted six months ago: in our December 2021 survey, nearly six in ten respondents expected to see economic improvements over that time period. At the same time, respondents’ takes on both current and future conditions in the global economy have grown progressively gloomier since June 2021, with half of all respondents expecting conditions to worsen in the second half of 2022 (Exhibit 2). Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com The findings about respondents’ respective countries also have grown more somber over the past year (Exhibit 3). For the first time since the September 2020 survey, respondents are more likely to say economic conditions in their countries have worsened than improved over the past six months. Views vary widely by region, however. In both Asia–Pacific and Greater China, about two-thirds of respondents say their countries’ economies have improved. The responses from Europe and North America are much more downcast: just one in five respondents in each region report recent improvements in their economies. Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com That said, respondents’ expectations for their home countries over the next six months are somewhat more hopeful than their outlook on the global economy: 39 percent expect their economies to improve in the near future. However, this is the first survey since the one in September 2020 in which less than half of respondents expect improvements in their home economies. Now, they are just as likely to expect economic conditions will improve as decline. McKinsey Global Surveys Explore the collection Most respondents in Asia–Pacific and Greater China expect their economies to improve in the second half of 2022, although overall optimism has declined since the previous survey (Exhibit 4). Over the same time period, respondents in Europe and North America have become much more pessimistic about the future. Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Download Economic conditions outlook, June 2022 (PDF– KB). ABOUT THE AUTHORS The survey content and analysis were developed by Krzysztof Kwiatkowski and Vivien Singer, capabilities and insights experts in McKinsey’s Waltham, Massachusetts, office, and Sven Smit , the chair and a director of the McKinsey Global Institute and a senior partner in the Amsterdam office. This article was edited by Heather Hanselman, an editor in the Atlanta office. Survey Economic conditions outlook, March 2022 Worries about geopolitical conflicts, among other risks to growth, now exceed executives’ concerns about the COVID-19 pandemic. Overall economic optimism continues to decline. Geopolitical instability is now cited as the top risk to both global and domestic economies in our latest McKinsey Global Survey on economic conditions. 8 8. The online survey was in the field from February 28 to March 4, 2022, and garnered responses from 785 participants representing the full range of regions, industries, company sizes, functional specialties, and tenures. To adjust for differences in response rates, the data are weighted by the contribution of each respondent’s nation to global GDP. That’s the consensus among executives worldwide, who have cited the COVID-19 pandemic as a leading risk to growth for the past two years. Our quarterly survey was launched four days after the invasion of Ukraine, and executives express uncertainty and concern about its impact on the economy. About three-quarters of respondents cite geopolitical conflicts as a top risk to global growth in the near term, up from one-third who said so in the previous quarter . Meanwhile, the share of respondents citing the pandemic as a top risk fell from 57 to 12 percent, as much larger percentages now identify energy prices and inflation as threats to the global economy. At the same time, overall sentiment about the economy remains largely positive, but it continues to trend downward. For the third quarter in a row, respondents are less likely than in the previous one to report that economic conditions in their respective countries and across the globe are improving. They are also less likely to believe that either global or domestic conditions will improve in the months ahead. The near-term economic outlook is especially gloomy among respondents in developed economies, whose views are increasingly downbeat compared with their emerging-economy peers. Would you like to learn more about our Strategy & Corporate Finance Practice ? Visit our Trends & Global Forces page Geopolitical conflict overshadows all other risks to growth According to the survey results, executives expect that the economic effects of the invasion of Ukraine will be strongly felt. Seventy-six percent of all respondents cite geopolitical instability and/or conflicts as a risk to global economic growth over the next 12 months, and 57 percent cite it as a threat to growth in their home economies (Exhibit 1). Exhibit 1 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Executives see geopolitical instability as the top risk to both global and domestic growth in every geography except Greater China, 9 9. Includes Hong Kong and Taiwan. where respondents most often cite the COVID-19 pandemic. Thirty-nine percent of respondents there say the pandemic is a threat to domestic growth, compared with 5 percent of all other respondents. Nearly two years after COVID-19 was declared a global pandemic, 10 10. “Timeline: WHO’s COVID-19 response,” World Health Organization, updated January 25, 2021. this is the first time our respondents have not cited the pandemic as the top risk to growth in the global economy (Exhibit 2). Exhibit 2 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Overall sentiment continues to wane While respondents tend to report improving—rather than worsening—conditions in the global economy and in their home countries, the percentages of executives saying so continue to decrease over time (Exhibit 3). Exhibit 3 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com Their outlook for the next six months is even more downbeat, especially for the global economy (Exhibit 4). Forty-three percent of respondents believe the global economy will improve over the next six months, a share that’s nearly equal to the 40 percent who think conditions will worsen. This month’s result also marks the first time since July 2020 that less than a majority of respondents feel optimistic about the global economy’s prospects. Exhibit 4 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com And while executives overwhelmingly cite geopolitical conflicts as a risk to economic growth, rising interest rates are a growing concern as well. Interest rates are among the top five risks to near-term growth in the global economy (for the second survey in a row) and in respondents’ home countries—and the share of respondents expecting a significant increase in near-term interest rates has more than doubled since the previous quarter. Across regions, executives in North America and in Europe are the most likely to expect interest rates to rise rather than hold steady or decrease. McKinsey Global Surveys Explore the collection The divide between developed and emerging economies grows For the third quarter in a row, the survey results suggest a widening gap in optimism between developed-economy and emerging-economy respondents. In developed economies—where respondents cite geopolitical conflicts as a risk to growth more often than their peers do—sentiment is declining at a faster rate than in emerging economies. Only 52 percent of developed-economy respondents, versus 73 percent of their emerging-economy peers, say economic conditions at home have improved in recent months. In our two previous surveys, the gap was much smaller (Exhibit 5). Exhibit 5 We strive to provide individuals with disabilities equal access to our website. If you would like information about this content we will be happy to work with you. Please email us at: McKinsey_Website_Accessibility@mckinsey.com This trend is also evident in respondents’ views on the global economy. This month, just 39 percent of developed-economy respondents say global economic conditions have improved in recent months, compared with 68 percent in emerging economies. Respondents in developed economies also report a more downbeat outlook for the coming months: only 36 percent believe conditions in the global economy will improve in the near term, versus 55 percent of their emerging-economy peers. Download Economic conditions outlook, March 2022 (PDF–422 KB). About the author(s) The survey content and analysis were developed by Alan FitzGerald , a director of client capabilities in McKinsey’s New York office; Vivien Singer , a capabilities and insights expert at the Waltham Client Capabilities Hub; and Sven Smit , the chair and director of the McKinsey Global Institute and a senior partner in the Amsterdam office. This article was edited by Daniella Seiler, an executive editor in the New York office. Explore a career with us Search Openings Related Articles Survey Economic conditions outlook, 2021 Survey The coronavirus effect on global economic sentiment Article McKinsey Global Economics Intelligence Sign up for emails on new {PRACTICE_NAME} articles Never miss an insight. We'll email you when new articles are published on this topic. Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} email alerts. Business Resilience | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Your browser doesn't support HTML5 video. Business Resilience The world is experiencing a level of disruption and business risk not seen in generations.  Some companies freeze and fail, while others innovate, advance, and even thrive. The difference is resilience. Institutional Resilience Financial Resilience | Operational Resilience | Technological Resilience | Organizational Resilience | Reputational Resilience | Business Model Article How to build geopolitical resilience amid a fragmenting global order September 8, 2022 – Organizations need to strategically invest in capabilities, people, processes, structures, and technology to navigate the risks arising from an evolving and fraught geopolitical landscape. Institutional Resilience Article War in Ukraine: Twelve disruptions changing the world May 9, 2022 – The war is devastating lives and roiling markets. Here we track the disruptions that seem likely to shape lives and livelihoods, beyond the immediate crisis. Interview How Volkswagen board member Hiltrud Werner finds resilience April 15, 2022 – Resilience is top of mind for Hiltrud Werner in her role as a member of Volkswagen’s board of management as COVID-19, new regulations, and the energy transition test the automaker’s mettle. Article From risk management to strategic resilience March 9, 2022 – Senior executives at leading companies reveal their commitment to move from defensive risk management to a forward-looking stance based on strategic resilience. Interview What resilience means to Nextdoor CEO Sarah Friar March 4, 2022 – For Nextdoor CEO Sarah Friar, resilience requires diverse perspectives, local solutions, playbooks to help anticipate the unexpected, and a regard for unforeseen consequences. Interview How CEO Alain Bejjani, an emerging-markets leader, models resilience February 28, 2022 – To manage through the pandemic, the CEO of Dubai-based Majid Al Futtaim has leaned on servant leadership, multistakeholder engagement, and his long-term goals. Article Three keys to a resilient postpandemic recovery February 18, 2022 – The path to sustainable, inclusive growth lies in building resilience now. Article The postpandemic board agenda: Redefining corporate resilience August 3, 2021 – As boards move beyond crisis management, survey results suggest that specific risks and organizational issues are increasingly top of mind. Podcast The role of boards in fostering resilience June 9, 2021 – The lessons learned from the current crisis can help corporate boards make the organizations they serve stronger. Article Next-generation nowcasting to improve decision making in a crisis May 20, 2021 – Traditional nowcasting has served its purpose well, but the COVID-19 crisis proved challenging for most models. A next-generation approach supports critical decision making and strategy moving forward. Article The resilience imperative: Succeeding in uncertain times May 17, 2021 – Strengthening institutional resilience has never been more important. Survey How boards have risen to the COVID-19 challenge, and what’s next April 29, 2021 – According to a new survey, the COVID-19 crisis has accelerated operational changes and stronger collaboration between directors and management that are key to a board’s success. Article Rethinking resilience: Ten priorities for governments November 30, 2020 – Due to the pandemic, governments are under increased pressure to act quickly and at scale. By reimagining how they work, they can create resilient societies and public services for the next normal. Article Meeting the future: Dynamic risk management for uncertain times November 17, 2020 – The world is changing in fundamental ways, leading to dramatic shifts in the landscape of risks faced by businesses. Financial Resilience Article Something’s coming: How US companies can build resilience, survive a downturn, and thrive in the next cycle September 16, 2022 – The US economy continues to throw off mixed signals. But one thing is becoming clear: executives should prepare for an extended... period of higher interest rates. Article Managing financial crime risk in digital payments June 24, 2022 – To face down the financial-crime threat, payments service providers can learn from banks while utilizing their own advanced technological... skills. Interview NatWest Chairman Sir Howard Davies on operational resilience in banking April 28, 2022 – Sir Howard Davies discusses the ways regulations, interest rates, monetary policy, climate risk, and economic threats are affecting... the resilience of the banking system. Article Financial institutions and nonfinancial risk: How corporates build resilience February 28, 2022 – As nonfinancial companies move from enterprise risk management to a resilience-based approach, their experience in nonfinancial... risk can provide a model for banks. Article Holistic customer assistance through digital-first collections May 21, 2021 – With families continuing to struggle to make ends meet, lenders that find the right combinations of digital-first customer support... will experience the benefits, including longer-term customer loyalty and insurance against being left in the wake of more ambitious peers. Article Meeting the future: Dynamic risk management for uncertain times November 17, 2020 – The world is changing in fundamental ways, leading to dramatic shifts in the landscape of risks faced by businesses. Operational Resilience Article Using analytics to address inflation risks and strengthen competitive positioning August 18, 2022 – In the new inflationary environment, company leaders can protect their business and gain competitive advantage by deploying analytics-aided strategies. Article Speed and resilience: Five priorities for the next five months March 29, 2021 – As businesses recover from COVID-19-related disruption and reimagine themselves for the next normal, they need to ask—and answer—five questions. Article Four ways pharma companies can make their supply chains more resilient September 23, 2021 – Pharmaceutical supply chains have become global and complex. With more outsourcing, new modalities, and novel ways to reach patients, it’s critical to ensure that they can withstand shocks. Article Building the  internal-audit function of the future February 18, 2021 – Due to the pandemic, many organizations have prioritized short-term priorities for internal-audit functions, but now is also the time to recalibrate for potential long-term uncertainty and complexity. Video The need for resiliency February 9, 2021 – Here’s how leading companies can build a business that’s ready for tomorrow. Article The resilience imperative for medtech supply chains December 18, 2020 – The COVID-19 pandemic highlighted issues with supply chains around the world. As medtech companies prepare for the next normal, they have a mandate to rebuild their supply chains with resilience as a new priority. Report
            
                    - McKinsey Global Institute Risk, resilience, and rebalancing in global value chains August 6, 2020 – Companies need an understanding of their exposure, vulnerabilities, and potential losses to inform resilience strategies. Article
            
                    - McKinsey Global Institute COVID-19 and climate change expose dangers of unstable supply chains August 28, 2020 – With the world undergoing its second “once in a generation” disruption in 12 years, a reappraisal of supply chains is in order. Technological Resilience Survey Why digital trust truly matters September 12, 2022 – Consumer faith in cybersecurity, data privacy, and responsible AI hinges on what companies do today—and establishing this digital trust just might lead to business growth. Article Securing your organization by recruiting, hiring, and retaining cybersecurity talent to reduce cyberrisk June 29, 2022 – Shed the conventional methods. Talent-to-value protection defines the most important cybersecurity roles that demonstrate the greatest reduction in risk for the enterprise. Article Model risk management 2.0 evolves to address continued uncertainty of risk-related events March 9, 2022 – Organizations this year plan to enhance their MRM framework capabilities—including risk culture, standards, and procedures—and to upgrade their validation resources with MRM 2.0 firmly on the agenda. Interactive The data-driven enterprise of 2025 January 28, 2022 – Rapidly accelerating technology advances, the recognized value of data, and increasing data literacy are changing what it means to be “data driven.” Podcast Cyber resilience: Protecting America’s digital infrastructure September 28, 2021 – Faced with rising cyberthreats, government and the private sector will need to improve their digital hygiene while also preparing for the next wave of cyber adversaries. Article Fast forward: How cloud computing could transform risk management September 17, 2021 – Cloud-based computing will provide tangible benefits for banking risk management functions, but risk leaders face significant challenges migrating their systems and activities from on-premises to the cloud. Article Building cyber resilience in national critical infrastructure June 30, 2021 – Recent cyberattacks focus attention on the vulnerabilities of operations technology to web-based threats. Blog Post IT resilience for the digital age May 11, 2021 – Incumbents are experiencing more frequent and more severe service disruptions as they accelerate their digital capabilities. To build IT resilience, they must adopt seven core practices. Article International personal-data transfer amid regulatory upheaval March 3, 2021 – The Schrems II ruling created a noteworthy disruption in the data-privacy environment. Here, we outline a risk-based approach to managing uncertainty from the ruling. Article Derisking digital and analytics transformations January 5, 2021 – While the benefits of digitization and advanced analytics are well documented, the risk challenges often remain hidden. Article Derisking AI by design: How to build risk management into AI development August 13, 2020 – The compliance and reputational risks of artificial intelligence pose a challenge to traditional risk-management functions. Derisking by design can help. Article Building a cloud-ready operating model for agility and resiliency March 19, 2021 – Four operating-model changes can help companies accelerate the journey to cloud. Organizational Resilience Interview Aon CEO Greg Case: Resilience protects, but it also promotes growth April 5, 2022 – As CEO of a company whose raison d’être is helping firms make better decisions, Greg Case has wide-ranging knowledge and a nuanced perspective on resilience and its importance in volatile times. Article
            
                    - McKinsey Quarterly ‘Great Attrition’ or ‘Great Attraction’? The choice is yours September 8, 2021 – A record number of employees are quitting or thinking about doing so. Organizations that take the time to learn why—and act thoughtfully—will have an edge in attracting and retaining talent. Podcast Culture in the hybrid workplace June 11, 2021 – Hybrid work is happening. Your culture will need to catch up—fast. Article Speed and resilience: Five priorities for the next five months March 29, 2021 – As businesses recover from COVID-19-related disruption and reimagine themselves for the next normal, they need to ask—and answer—five questions. Interview
            
                    - McKinsey Quarterly Driving for resilience: An interview with the CEO of India’s Tata Motors November 24, 2020 – Guenter Butschek, CEO of Tata Motors, shares his perspectives on the COVID-19-era challenges of leading a major global automotive company based in India, and the opportunities just up the road. Interview Resilience in a crisis: An interview with Professor Edward I. Altman November 19, 2020 – One of the leading researchers in corporate financial health discusses what executives can do to help their companies endure the financial stresses of crisis times. Article When nothing is normal: Managing in extreme uncertainty November 2, 2020 – In this uniquely severe global crisis, leaders need new operating models to respond quickly to the rapidly shifting environment and sustain their organizations through the trials ahead. Article Doing vs being: Practical lessons on building an agile culture August 4, 2020 – Four global success stories offer insights and lessons learned on achieving organizational agility. Reputational Resilience Article Strengthening institutional risk and integrity culture November 2, 2020 – Many of the costliest risk and integrity failures have cultural  weaknesses at their core. Here is how leading institutions are strengthening their culture and sustaining the change. Article
            
                    - McKinsey Quarterly One move companies can take to improve diversity April 9, 2021 – Many companies are trying to increase their hiring of diverse employees. Opening hubs, factories, and other worksites where Black people live is a powerful but overlooked lever. Business Model Webinar Finding tomorrow’s ‘resilients’: Propelling companies into the next wave of growth October 28, 2020 – Leaders can learn from the last global recession and plan for several economic outcomes, including, as McKinsey senior partners Cindy Levy and Bob Sternfels suggest, taking thoughtful actions to balance growth, margins, and optionality. Article Four regions, four stories of recovery and resilience March 24, 2021 – An assessment of shareholder returns in the world’s four largest capital markets reveals how the global stock market fared in the face of the COVID-19 pandemic. Article Strategic resilience during the COVID-19 crisis March 2, 2021 – Organizations that rapidly reinvented themselves in response to the pandemic can provide lessons on resilience. Article The emerging resilients: Achieving ‘escape velocity’ October 6, 2020 – The experience of the fast movers out of the last recession teaches leaders emerging from this one to take thoughtful actions to balance growth, margins, and optionality. Related Collections The Next Normal: Emerging stronger from the coronavirus pandemic Explore McKinsey’s curated collection on the business outlook for 2021 and beyond—plus insights about how companies can emerge stronger from the coronavirus crisis. McKinsey on Risk, Number 12, April 2022 April 4, 2022 – Gearing up for life beyond the crisis, organizations face an array of risks. In our latest issue of McKinsey on Risk, we offer our unique perspectives about the imperatives of strengthening institutional resilience. Featured Video Video Staying resilient during an economic downturn While the last downturn was severe, a group of companies we've called "resilients" flourished. How did they do it? Industry perspectives Article How retailers can build resilience ahead of a recession March 10, 2020 – Recent history shows how retailers can prepare for a downturn to create competitive advantage. Article Resilience in transport and logistics February 4, 2020 – The transportation-and-logistics sector is especially susceptible to economic shocks. Here’s how to prepare your operations... for a smoother ride. Article Down but not out: How automakers can create value in an uncertain future September 25, 2019 – After delivering record profits in 2017, global automakers have faced several challenges over the past 12 months. Can the industry... find a sustainable path to value creation in this disruptive environment? Interactive Beating the cycle: Building resilience in chemicals September 18, 2019 – Economic downturns take their toll—and chemical companies are  no exception. Our research finds companies that increased resilience... in the last downturn made bold moves. Article Resilience in TMT: Winning in downturns October 1, 2019 – Economic downturns hold substantial opportunities for companies in the technology, media, and telecommunications (TMT) sector.... By starting now to build an action plan and execute no-regret moves, companies can put themselves on a path to emerge resilient through the next slowdown. Article Recession and resilience: Preparing healthcare leaders for the next economic downturn July 11, 2019 – Economic ebbs and flows are called “cycles” for a reason. The challenge for healthcare leaders is not whether the... next downturn will occur—it’s whether you’re ready for it. Connect with Business Resilience Contact us Sign up for our {PRACTICE_NAME} newsletter Don't miss this roundup of our newest and most distinctive insights Something went wrong. Please try again later. Subscribed to {PRACTICE_NAME} Manama | Middle East | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content McKinsey in the Middle East Manama McKinsey’s Manama office works with some of Bahrain’s leading institutions to shape strategies, deliver high performance, and nurture talent. Careers Working in Manama We recruit some of the most talented people in Bahrain, and from across the Middle East and the world. A career at McKinsey provides unparalleled professional and personal growth and the opportunity to build powerful networks while helping major organisations transform their performance. Explore jobs in Manama Search jobs Already started an application? Sign in here Prepare for your interview Learn more Every day we help our clients solve their toughest problems, build capabilities, and create better outcomes. Learn more on our Global Careers site Our location Manama Manama Financial Harbour (West), Level 41 Office No. 4123 King Faisal Highway Manama, Bahrain Voice: +973 (17) 502 830 Fax: +973 (17) 502 832 https://maps.google.com/maps?q=26.237451,50.573333&z=15 Learn more about our Middle East locations Abu Dhabi Cairo Doha Dubai Manama Riyadh 130+ offices worldwide Explore our locations Connect with McKinsey in the Middle East Contact Search Jobs | Careers | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content My Dashboard Inclusive-Growth | New at McKinsey Blog | McKinsey & Company https://www.mckinsey.com 915b5091-0d7e-44d2-a8c4-cf08267e52fe Skip to main content Back to New at McKinsey Blog Inclusive Growth Browse by:

    '''
]

reloaded_results = tf.sigmoid(reloaded_model(tf.constant(examples)))
original_results = tf.sigmoid(classifier_model(tf.constant(examples)))

print('Results from the saved model:')
print(reloaded_results)
print('Results from the model in memory:')
print(original_results)


Results from the saved model:
tf.Tensor([[0.039 0.    0.994]], shape=(1, 3), dtype=float32)
Results from the model in memory:
tf.Tensor([[0.039 0.    0.994]], shape=(1, 3), dtype=float32)


In [16]:
results = data_test.copy(True)
results['Result'] = 'Failed'
results['Predict'] = '0'

def get_max_label(result, label_map):
    m = max(result)
    return label_map[[i for i, j in enumerate(result) if j == m][0]]

count = 0
for index, row in results.iterrows():
    try:
        result = tf.sigmoid(reloaded_model(tf.constant([row['text']])))[0]
        result_label = get_max_label(result, label_map)
        is_pass = result_label == row['category']
        results.at[index, 'Predict'] = result_label
        if is_pass:
            count += 1
            results.at[index, 'Result'] = 'Pass'
        results.to_csv('results.csv', index=False)
    except:
        print('ERROR:', index)

print(count / len(results))

0.8205128205128205
